In [ ]:
sizeOfRoom=[]
h=int(input('please enter the room height in cm'))
w=int(input('please enter the room width in cm'))
sizeOfRoom.append(h)
sizeOfRoom.append(w)

please enter the room height in cm500
please enter the room width in cm500


In [ ]:
!pip install keras_retinanet

     |████████████████████████████████| 71kB 5.4MB/s 
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=163869 sha256=0772bf1cf87dd75d6c3696a9eaf7b64043449aa57880301df7956dc7f883354a
  Stored in directory: /root/.cache/pip/wheels/79/1d/fc/496708301dbd84bc2faa258d24d82f39fe46d9701d52287373
  Created wheel for keras-resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20487 sha256=7da2a40d559baa52804d5713e9f2dd9c2899a079505708a1bff3cc27e3d02348
  Stored in directory: /root/.cache/pip/wheels/5f/09/a5/497a30fd9ad9964e98a1254d1e164bcd1b8a5eda36197ecb3c
Successfully built keras-retinanet keras-resnet


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
%tensorflow_version

Currently selected TF version: 1.x
Available versions:
* 1.x
* 2.x


In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
    # show images inline
%matplotlib inline

    # automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2
import tensorflow as tf

# import keras
#import tf.keras
from tf import keras

    # import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

from gpu import setup_gpu

    # import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: ignored

In [ ]:
def get_num_pixels(filepath):
    from PIL import Image
    width, height = Image.open(filepath).size
    return width,height

In [ ]:
def loadingModel():
    
    gpu = 0

    setup_gpu(gpu)
    model = models.load_model('/content/drive/My Drive/snapshots/resnet50_csv_50(2).h5', backbone_name='resnet50')

    model = models.convert_model(model)
    
    return model

In [ ]:
def predictObjects(imagePath, model, numofWindows, numofDoors):
    image = read_image_bgr(imagePath)
    labels_to_names = {0: 'door', 1: 'window'}
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    # print('width, height: ',get_num_pixels(imagePath))
    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale
    objectsCoor={}
    num1=numofWindows
    num2=numofDoors
    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break
        # else:
        #     l=True
        #     j=0
        #     #the object
        #     o=''
        #      #check if there are previous objects in the picture
        #     if len(objectsCoor)!=0:
        #         print('len object coor',len(objectsCoor))
        #         #if it is true, it means we have to break, otherwise we delete the dictionary element
                
        #          #get the objects names
        #         objs=list(objectsCoor.keys())

        #         for obj in objs:
        #             o=obj
        #             k=0
        #             #check if there is a previous object of the same kind of the current detected one
        #             nn=str(labels_to_names[label])
        #             mm=str(obj)
        #             if nn in mm :
        #                  #get the coordinates of the current detected object
                        
        #                 b=box.astype(int)
        #                 c=b.tolist()
        #                 m=0
        #                  #m<4 because each object has 4 coordinates: left, down, right, up
        #                 while m<4:
        #                     #if the one of the coordinates of the current object is smaller than the coordinates of the other object, it means it's contained in it
                            
        #                     if c[m]<objectsCoor[obj][m]:
        #                         j+=1
        #                     m+=1
        #                 #if there are more than two intersection we don't have to search anymore
        #                 if j>2 and j!=4:
        #                     l=False
        #                 #if the intersections are less than 2
        #                 if l:
        #                     j=0
        #                     m=0
        #                     while m<4:
        #                         if c[m]>objectsCoor[obj][m]:
        #                             j+=1
        #                         m+=1
        #                     if j>2 and j!=4:
        #                         l=True

        #     if j>2 and j!=4:
        #         if l:
        #             del objectsCoor[o]
        #             if 'window' in o:
        #                 num1-=1
        #             elif 'door' in o:
        #                 num2-=1
        #         else:
        #             break
                
                   
                
        color = label_color(label)
        b = box.astype(int)
        #to concatenate the number of objects with their names because we need them in following steps
        if labels_to_names[label]=='window':
            num1=num1+1
            name=labels_to_names[label]+str(num1)
        elif labels_to_names[label]=='door':
            num2=num2+1
            name=labels_to_names[label]+str(num2)

        print('name ',name,b)
        objectsCoor[name]=b
        draw_box(draw, b, color=color)
        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)

    
    
    plt.figure(figsize=(15, 15))
    plt.axis('off')
    plt.imshow(draw)
    plt.show()
    #returns the objects names with their horizontal coordinates, and the number of detected windows and doors
    return objectsCoor, num1, num2

In [ ]:
def extractingSectors(i,model,Sectors,y,numofwindows,numofdoors):
    imagePath=input()
    w, h=get_num_pixels(imagePath)
    print('w',w)
    objectsCoor, numofWindows, numofDoors=predictObjects(imagePath,model,numofwindows,numofdoors)
    objects={}
    
    var=True
    if len(objectsCoor)==0:
        print(f'wall num {i} has no windows nor walls')

    elif len(objectsCoor)==1:
        imgcoor=[]
        objimgcoor=[]
        objectNames=list(objectsCoor.keys())
        print('objects names are: ',objectNames)
        objectName=objectNames[0]
        coor=objectsCoor[objectName]
        #objimgcoor holds the returned horizontal coodinates of the object
        objimgcoor.append(coor[0])
        objimgcoor.append(coor[2])
        #calculate how many sectors the image can be divided into, according to the number of sectors of the room
        imageSectors=w*Sectors/y
        #to find out start sector and end sector of the object but in image sectors
        startSector=objimgcoor[0]/50
        endSector=objimgcoor[1]/50
        
        if imageSectors< Sectors:
            print('please upload a valid image with the right measurements')
            var=False
            return ####
        else:
            #each sector in image equals x sector in real sectors
            x=imageSectors/Sectors
            startSector=int(startSector/x)
            endSector=int(endSector/x)
            #first element is the wall number, second one is the start sector, the third is the end
            objects[objectName]=[i,startSector,endSector]
            
    #if we have more than one object 
    elif len(objectsCoor)>1:
        listOfObjectsNames=list(objectsCoor.keys())
        for j in listOfObjectsNames:
            objimgcoor=[]
            objectName=j
            print('object name and its coor', objectName, objectsCoor[objectName])
            coor=objectsCoor[objectName]
            objimgcoor.append(coor[0])
            objimgcoor.append(coor[2])

            imageSectors=w*Sectors/y
            startSector=objimgcoor[0]/50
            endSector=objimgcoor[1]/50
            if imageSectors< Sectors:
                print('please upload a valid image with the right measurements')
                var=False
                return
            else:
                x=imageSectors/Sectors
                startSector=int(startSector/x)
                endSector=int(endSector/x)
                print('start end sectors ', startSector, endSector)
                #first element is the wall number, second one is the start sector, the third is the end
                objects[objectName]=[i,startSector,endSector]
                
    return objects, numofWindows, numofDoors

In [ ]:
import matplotlib.pyplot as plt

def inputImages(sizeOfRoom=sizeOfRoom):
    import time
    model=loadingModel()
    i=0
    numofWindows=0
    numofDoors=0
    height=sizeOfRoom[0]
    width=sizeOfRoom[1]
    heightSectors=int(height/50)
    widthSectors=int(width/50)
    objectSectors=[]
    print('please upload the corresponding wall image')
    #we need four images, one for each wall
    while i<4: 
        #the image of the first wall
        if i==0:
            print(f'upload the image of the {i}th wall')
            x1 = [10,20]
            y1 = [20,20]
            plt. plot(x1, y1, color='green')
            x2=[10,10]
            y2=[10,20]
            plt.plot(x2,y2,linestyle='dashed',color='red')
            x3 = [10,20]
            y3 = [10,10]
            plt. plot(x3, y3, linestyle='dashed',color='red')
            x4 = [20,20]
            y4 = [10,20]
            plt. plot(x4, y4, linestyle='dashed',color='red')
            plt.axis('off')
            while True:
                lst=extractingSectors(i,model,widthSectors,width,numofWindows,numofDoors) 
                if lst is not None:
                    break
            #the objects and their sectors, and the number of windows and walls
            objects=lst[0]
            num1=lst[1]
            num2=lst[2]
            
            objectSectors.append(objects)
            if num1!= numofWindows:
                numofWindows+=num1
            if num2!=numofDoors:
                numofDoors+=num2
            i+=1
        if i==1:
            print(f'upload the image of the {i}th wall')
            x1 = [10,20]
            y1 = [20,20]
            plt. plot(x1, y1,linestyle='dashed',color='red' )
            x2=[10,10]
            y2=[10,20]
            plt.plot(x2,y2,color='green')
            x3 = [10,20]
            y3 = [10,10]
            plt. plot(x3, y3, linestyle='dashed',color='red')
            x4 = [20,20]
            y4 = [10,20]
            plt. plot(x4, y4, linestyle='dashed',color='red')
            plt.axis('off')
            while True:
                lst=extractingSectors(i,model,widthSectors,width,numofWindows,numofDoors) 
                if lst is not None:
                    break
            objects=lst[0]
            num1=lst[1]
            num2=lst[2]
            objectSectors.append(objects)
            if num1!= numofWindows:
                numofWindows+=num1
            if num2!=numofDoors:
                numofDoors+=num2
            i+=1
        if i==2:
            print(f'upload the image of the {i}th wall')
            x1 = [10,20]
            y1 = [20,20]
            plt. plot(x1, y1,linestyle='dashed',color='red' )
            x2=[10,10]
            y2=[10,20]
            plt.plot(x2,y2,linestyle='dashed',color='red')
            x3 = [10,20]
            y3 = [10,10]
            plt.plot(x3, y3, color='green')
            x4 = [20,20]
            y4 = [10,20]
            plt. plot(x4, y4, linestyle='dashed',color='red')
            plt.axis('off')
            while True:
                lst=extractingSectors(i,model,widthSectors,width,numofWindows,numofDoors) 
                if lst is not None:
                    break
            objects=lst[0]
            num1=lst[1]
            num2=lst[2]
            objectSectors.append(objects)
            if num1!= numofWindows:
                numofWindows+=num1
            if num2!=numofDoors:
                numofDoors+=num2
            i+=1
        if i==3:
            print(f'upload the image of the {i}th wall')
            x1 = [10,20]
            y1 = [20,20]
            plt. plot(x1, y1,linestyle='dashed',color='red' )
            x2=[10,10]
            y2=[10,20]
            plt.plot(x2,y2,linestyle='dashed',color='red')
            x3 = [10,20]
            y3 = [10,10]
            plt. plot(x3, y3, linestyle='dashed', color='red')
            x4 = [20,20]
            y4 = [10,20]
            plt. plot(x4, y4, color='green')
            plt.axis('off')
            while True:
                lst=extractingSectors(i,model,widthSectors,width,numofWindows,numofDoors) 
                if lst is not None:
                    break
            objects=lst[0]
            num1=lst[1]
            num2=lst[2]
            objectSectors.append(objects)
            if num1!= numofWindows:
                numofWindows+=num1
            if num2!=numofDoors:
                numofDoors+=num2
            i+=1
    return objectSectors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x=inputImages(sizeOfRoom)

TypeError: ignored

In [ ]:
print(x)

[{}, {'window1': [1, 4, 9], 'door1': [1, 4, 9]}, {'window2': [2, 4, 9], 'door2': [2, 4, 9]}, {}]


In [ ]:
def final(x,sizeOfRoom):
    numOfwin=1
    numOfdoor=1
    room=[]
    lengthSectors=int(sizeOfRoom[0]/50)
    widthSectors=int(sizeOfRoom[1]/50)

    #true indicates window
    dw=True
    win=[]
    door=[]
    temp=[]
    for dic in x:
        for key in dic.keys():
            wallnum=dic[key][0]
            for i in range(dic[key][1],dic[key][2]):
                if 'window' in key:
                    print(key)
                    
                    name='window'+str(numOfwin)
                    if wallnum==0:
                        rows=[1,None,None]
                        columns=[i+1,None,None]
                        if [wallnum,rows,columns] not in temp:
                            win.append([name,rows,columns])
                            temp.append([wallnum,rows,columns])
                            numOfwin+=1
                    if wallnum==1:
                        rows=[i+1,None,None]
                        columns=[1,None,None]
                        if [wallnum,rows,columns] not in temp:
                            win.append([name,rows,columns])
                            temp.append([wallnum,rows,columns])
                            numOfwin+=1
                    if wallnum==2:
                        rows=[lengthSectors,None,None]
                        columns=[i+1,None,None]
                        if [wallnum,rows,columns] not in temp:
                            win.append([name,rows,columns])
                            temp.append([wallnum,rows,columns])
                            numOfwin+=1
                    if wallnum==3:
                        rows=[i+1,None,None]
                        columns=[widthSectors,None,None]
                        if [wallnum,rows,columns] not in temp:
                            win.append([name,rows,columns])
                            temp.append([wallnum,rows,columns])
                            numOfwin+=1
                else:
                    print("g")
                    name='door'+str(numOfdoor)
                    if wallnum==0:
                        rows=[1,None,None]
                        columns=[i+1,None,None]
                        if [rows,columns] not in temp:
                            door.append([name,rows,columns])
                            temp.append([rows,columns])
                            numOfdoor+=1
                    if wallnum==1:
                        rows=[i+1,None,None]
                        columns=[1,None,None]
                        if [rows,columns] not in temp:
                            door.append([name,rows,columns])
                            temp.append([rows,columns])
                            numOfdoor+=1
                    if wallnum==2:
                        rows=[lengthSectors,None,None]
                        columns=[i+1,None,None]
                        if [rows,columns] not in temp:
                            door.append([name,rows,columns])
                            temp.append([rows,columns])
                            numOfdoor+=1
                    if wallnum==3:
                        rows=[i+1,None,None]
                        columns=[widthSectors,None,None]
                        if [rows,columns] not in temp:
                            door.append([name,rows,columns])
                            temp.append([rows,columns])
                            numOfdoor+=1
    return win, door

In [ ]:
w, d=final(x,sizeOfRoom)

window1
window1
window1
window1
window1
g
g
g
g
g
window2
window2
window2
window2
window2
g
g
g
g
g


In [ ]:
print(w,d)

[['window1', [5, None, None], [1, None, None]], ['window2', [6, None, None], [1, None, None]], ['window3', [7, None, None], [1, None, None]], ['window4', [8, None, None], [1, None, None]], ['window5', [9, None, None], [1, None, None]], ['window6', [14, None, None], [5, None, None]], ['window7', [14, None, None], [6, None, None]], ['window8', [14, None, None], [7, None, None]], ['window9', [14, None, None], [8, None, None]], ['window10', [14, None, None], [9, None, None]]] [['door1', [5, None, None], [1, None, None]], ['door2', [6, None, None], [1, None, None]], ['door3', [7, None, None], [1, None, None]], ['door4', [8, None, None], [1, None, None]], ['door5', [9, None, None], [1, None, None]], ['door6', [14, None, None], [5, None, None]], ['door7', [14, None, None], [6, None, None]], ['door8', [14, None, None], [7, None, None]], ['door9', [14, None, None], [8, None, None]], ['door10', [14, None, None], [9, None, None]]]


In [ ]:
import csv
with open('objects.csv','w',newline='') as f:
    spamwriter= csv.writer(f,delimiter=',')
    for win in w:
        spamwriter.writerow([win[0]]+[win[1]]+[win[2]])
    for door in d:
        spamwriter.writerow([door[0]]+[door[1]]+[door[2]])

In [ ]:
import argparse
from PIL import Image, ImageDraw
import ast
import PIL

def grid(sizeOfRoom,f):
    step_horizontaly= int(sizeOfRoom[1]/50)
    step_verticaly= int(sizeOfRoom[0]/50)
    height = sizeOfRoom[0]
    width = sizeOfRoom[1]
    image = Image.new(mode='L', size=(width,height), color=255)

    # Draw a grid
    draw = ImageDraw.Draw(image)
    y_start = 0
    y_end = height
    x_start = 0
    x_end = width
    step_size1 = int(width / step_horizontaly)
    
    step_size2= int(height / step_verticaly )
    print('steps are :',step_size1,step_size2 )

    for x in range(0, height+2, step_size1):
        line = ((x, y_start), (x, y_end))
        draw.line(line, fill=128)
    
    
    for y in range(0, width+1,step_size2):
        line = ((x_start, y), (x_end, y))
        draw.line(line, fill=128)
    
    rows=[]
    columns=[]
    with open(f,'r',newline='') as ff:
        read= csv.reader(ff,delimiter=',')

        for row in read:
            rows=[]
            columns=[]
            obj=row[0]
            row1=ast.literal_eval(row[1])
            row2=ast.literal_eval(row[2])
            for j in row2:
                if j not in rows and j is not None:
                    rows.append(j)
            for j in row1:
                if j not in columns and j is not None:
                    columns.append(j)
            for r in rows:
                for c in columns:
                    x=(int(r)*50)-50
                    y=(int(c)*50)-50
                    draw.text((x,y),obj)
            # x=(int(row[1])*50)-50
            # y=(int(row[2])*50)-50
            # print(x,y)
            # coor=[]
            # coor.append(x)
            # coor.append(y)
    #         draw.text((x,y),obj)
    filename = "grid-{}-{}.png".format(width, height)
    print("Saving {}".format(filename))
    image.save(filename)

In [ ]:
grid(sizeOfRoom,'objects.csv')

steps are : 50 50
Saving grid-500-700.png


In [ ]:
print("please enter your description based on the following image")

please enter your description based on the following image


In [ ]:
pip install speechrecognition

In [ ]:
#import speech_recognition as sr 
from scipy.io import wavfile
from google.colab import files
import spacy
import nltk
import csv
import ast


In [ ]:
def speechRecognition0():
    uploaded = files.upload()
    for fn in uploaded.keys():
        s='{name}'.format(name=fn)
    r = sr.Recognizer()
    harvard = sr.AudioFile(s)
    with harvard as source:
        audio = r.record(source)
    str0=r.recognize_google(audio)
    return str0

In [ ]:
nlp=spacy.load('en_core_web_sm')

In [ ]:
numbers={'one':1,
         'two':2,
         'three':3,
         'four':4,
         'five':5,
         'six':6,
         'seven':7,
         'eight':8,
         'nine':9,
         'first':1,
         'second':2,
         'third':3,
         'fourth':4,
         'fifth':5,
         'sixth':6,
         'seventh':7,
         'eighth':8,
         'ninth':9,
         'tenth':10,
         'eleventh':11,
         'twelveth':12,
         'thirteenth':13,
         'fourteenth':14}

objects={ "chair" : 0,
         "desk" : 1,
         "carpet" : 2 ,
         "bed" : 3 ,
         "closet" : 4  ,
         "tables" : 5,
         "table" :7,
         "screen" : 6,
         "dc":76}

directions={ "left": [-1] ,
            "right" : [1] ,
            "infront" : [1] ,
            "behind" : [-1] ,
            "under" : 4 ,
            "above" : 5,
            "near" : [2],
            "next" :[1],
            "around":["left","right","infront","behind"],
            "between":9,
            "against":10,
            "opposite":11,
            "below":12,
            "over":13,
            "middle":14,
            'beside':15,
            'UL':55,
            'UR':422}

locations={ "wall" : 0 ,
           "corner" : 1 ,
           "window" : 2 ,
           "door" : 3,
           "center":4,
           'room':74}

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# s=speechRecognition0()
# s=s.replace(",","")
# s=input()
# sentences=nltk.sent_tokenize(s)

In [ ]:
with open('input.csv','w',newline='') as f:
     theWriter=csv.writer(f,delimiter=',')

In [ ]:
sentences=['put a table in the middle of the room',
'put four chairs around the table',
'put a closet in the UL corner',
'let the desk be near the second window',
'put a dc behind to the desk']

In [ ]:
numOfSentence=0

for sent in sentences:
    numOfSentence+=1
    objectslist=[]
    directionslist=[]
    locationlist=[]
    doc=nlp(sent)
    number=[]
    
    for token in doc:
        
        if token.text=='it' or token.text=='them':
            with open('input.csv','r',newline='') as f:
                reader=csv.reader(f,delimiter=',')
                num=0
                for row in reader:
                    num+=1
                    print('num sentence ', num, numOfSentence)
                    if num==numOfSentence-1:
                        row0=ast.literal_eval(row[2])
                        print('r',row0[len(row0)-1])
                        locationlist.append(row0[len(row0)-1])
               
    filename= 'input'+str(numOfSentence)+'.csv'            
    with open('input.csv','a',newline='') as f:
        theWriter=csv.writer(f,delimiter=',')
        order=[]
        for token in doc:
            
            if token.tag_=='NNS' :
                if token.lemma_ in objects.keys():
                    if len(number)!=0:
                        t=token.text+number[0]
                        objectslist.append(t)
                    else:
                        objectslist.append(token.text)
            if token.tag_ =='NN' and token.text in objects.keys():
                objectslist.append(token.text)
            if token.tag_ =='NN' and token.text in directions.keys():
                directionslist.append(token.text)
            if token.tag_=='IN' and token.text in directions.keys():
                directionslist.append(token.text)
            if token.tag_ =='RB' and token.text in directions.keys():
                directionslist.append(token.text)
            if token.tag_=='NN' or token.tag_=='NNS' :
                if token.text in objects.keys():
                    if len(directionslist)!=0:
                        locationlist.append(token.text)
                        if token.text in objectslist:
                            objectslist.remove(token.text)
            if token.tag_=='NN' or token.tag_=='NNS':
                if token.text in locations.keys():
                    if len(order)>0:
                        loc=token.text+str(order[0])
                        print('loc order',loc)
                        locationlist.append(loc)
                    else:
                        locationlist.append(token.text)
            if token.tag_=='CD':
                number.append(str(numbers[token.text]))
            if token.tag_=='JJ' and token.text in numbers.keys():
                order.append(numbers[token.text])
                print('order',order)
            if token.tag_=='JJ' and token.text in directions.keys():
                directionslist.append(token.text)
            if token.tag_=='NNP' and token.text in directions.keys():
                directionslist.append(token.text)
            if token.tag_=='NNP' and token.text in objects.keys():
              objectslist.append(token.text)
        theWriter.writerow([objectslist]+[directionslist]+[locationlist])

        print(objectslist, directionslist, locationlist)

['table'] ['middle'] ['room']
['chairs4'] ['around'] ['table']
['closet'] ['UL'] ['corner']
order [2]
loc order window2
['desk'] ['near'] ['window2']
['dc'] ['behind'] ['desk']


In [ ]:
with open('input.csv','r',newline='') as f:
    reader=csv.reader(f,delimiter=',')
    objects=[]
    dire=[]
    loc=[]
    for row in reader:
        row0=ast.literal_eval(row[0])
        row1=ast.literal_eval(row[1])
        row2=ast.literal_eval(row[2])
        
        print(row0)
        if row0[0] not in objects and 'chairs' not in row0[0]:
            objects.append(row0[0])
            dire.append(row1)
            loc.append(row2)

        for i in range(1,10):
            if str(i) in row0[0]:
                for j in range(1,i+1):
                    r='chair'+str(j)
                    if r not in objects:
                        objects.append(r)
                        dire.append(row1)
                        loc.append(row2)

['table']
['chairs4']
['closet']
['desk']
['dc']


In [ ]:
loca=[]
for l in loc:
  loca.append(l[0])

direc=[]
for l in dire:
  direc.append(l[0])
loca ,direc

(['room', 'table', 'table', 'table', 'table', 'corner', 'window2', 'desk'],
 ['middle', 'around', 'around', 'around', 'around', 'UL', 'near', 'behind'])

In [ ]:
def empty_room (rows,colmns):
  #this function will create an empty room represented by a list of lists filled with zero values 
  room = []
  for i in range(rows):
    temp=[]
    for j in range(colmns):
      temp.append(0)
    room.append(temp)
  return room

In [ ]:
def print_room(room):
  for item in room:
    print(item)

In [ ]:
def add_objects_to_room (room,coordinates):
  rows=len(room)
  colmn=len(room[0])
  for coor in coordinates :
    room[coor[0]][coor[1]]=1
  return room 

In [ ]:
def is_ver_horz (location ,door_wall):
  rows =[]
  colmns =[]
  direction =""
  if location != None:
      for loc in location :
          rows.append(loc[0])
          colmns.append(loc[1])
      if (door_wall==1 or door_wall == 3 ) and rows[0]==rows[1]:
         direction = "vertical"
      elif (door_wall==1 or door_wall == 3 ) and colmns[0]==colmns[1]:
          direction = "horizantol"
      elif (door_wall==2 or door_wall == 4 ) and rows[0]==rows[1]:
          direction = "horizantol"
      elif (door_wall==2 or door_wall == 4 ) and colmns[0]==colmns[1]:
          direction = "vertical"
  rows.sort()
  colmns.sort()
  return rows , colmns , direction

In [ ]:
import random
def add_item_to_room(item,room): 
  rows = len(room)-1
  colmns=len(room[0])-1
  coordinates=[]
  x=random.randint(1,rows)
  y=random.randint(1,colmns)

  while True :
    if room[x][y]!=0:
      x=random.randint(1,rows)
      y=random.randint(1,colmns)
    else :
      break 

  if"chair" in item or "dc" in item  :
    coordinates.append([x,y])

  if "closet" in item or "table" in item: 
    coordinates.append([x,y])
    if x-1>=0 and room[x-1][y]==0 :
      coordinates.append([x-1,y])
    elif x+1 <= rows  and room[x+1][y]==0 :
      coordinates.append([x+1,y])
    elif y+1<= colmns and room[x][y+1]==0 :
      coordinates.append([x,y+1])
    elif y-1>=0 and room[x][y-1]==0 :
      coordinates.append([x,y-1]) 

  if "desk" in item : 
      if x==0 and room[x+1][y]==0 and room[x+2][y]==0:
        coordinates.append([x,y])
        coordinates.append([x+1,y])
        coordinates.append([x+2,y])
      elif x==rows and room[x-1][y]==0 and room[x-2][y]==0:
        coordinates.append([x-2,y])
        coordinates.append([x-1,y])
        coordinates.append([x,y])
      elif 0<x<rows and room[x-1][y]==0 and room[x-2][y]==0:
        coordinates.append([x-1,y])
        coordinates.append([x,y])
        coordinates.append([x+1,y])
      if len(coordinates)==0:
        if y==0:
          if room[x][y+1]==0 and room[x][y+2]==0:
            coordinates.append([x,y])
            coordinates.append([x,y+1])
            coordinates.append([x,y+2])
        elif 0< y < colmns:
          if y-1 >=0 and y+1 <= colmns and room[x][y-1]==0 and room[x][y+1]==0  :
            coordinates.append([x,y-1])
            coordinates.append([x,y])
            coordinates.append([x,y+1])
        elif y == colmns :
          if room[x][y-1]==0 and room[x][y-2]==0:
            coordinates.append([x,y-2])
            coordinates.append([x,y-1])
            coordinates.append([x,y])
  add_objects_to_room (room,coordinates)
  return coordinates , room

In [ ]:
def location_wall1(item,room):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 
  if "chair" in item or "dc" in item :
    for i in range(0,rows+1):
      if room[i][0]==0 :
        coordinates.append([i,0])
        break
  if "closet" in item or "table" in item :
    for i in range(0,rows+1):
      if i+1<=rows and room[i][0]==0 and room[i+1][0]==0 :
        coordinates.append([i,0])
        coordinates.append([i+1,0])
        break
  if "desk" in item :
    for i in range(0,rows+1):
      if i+2<=rows and room[i][0]==0 and room[i+1][0]==0 and room[i+2][0]==0 :
        coordinates.append([i,0])
        coordinates.append([i+1,0])
        coordinates.append([i+2,0])
        break    
  room=add_objects_to_room(room,coordinates)
  return coordinates , room

In [ ]:
def location_wall3(item,room):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 
  if "chair" in item or "dc" in item :
    for i in range(0,rows+1):
      if room[i][colmns]==0 :
        coordinates.append([i,colmns])
        break
  if "closet" in item or "table" in item :
    for i in range(0,rows+1):
      if i+1<=rows and room[i][colmns]==0 and room[i+1][colmns]==0 :
        coordinates.append([i,colmns])
        coordinates.append([i+1,colmns])
        break
  if "desk" in item :
    for i in range(0,rows+1):
      if i+2<=rows and room[i][colmns]==0 and room[i+1][colmns]==0 and room[i+2][colmns]==0 :
        coordinates.append([i,colmns])
        coordinates.append([i+1,colmns])
        coordinates.append([i+2,colmns])
        break    
  room=add_objects_to_room(room,coordinates)
  return coordinates , room

In [ ]:
def location_wall2(item,room):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 
  if "chair" in item or "dc" in item :
    for i in range(0,colmns+1):
      if room[0][i]==0 :
        coordinates.append([0,i])
        break
  if "closet" in item or "table" in item :
    for i in range(0,colmns+1):
      if i+1<=colmns and room[0][i]==0 and room[0][i+1]==0 :
        coordinates.append([0,i])
        coordinates.append([0,i+1])
        break
  if "desk" in item :
    for i in range(0,colmns+1):
      if i+2<=colmns and room[0][i]==0 and room[0][i+1]==0 and room[0][i+2]==0:
        coordinates.append([0,i])
        coordinates.append([0,i+1])
        coordinates.append([0,i+2])
        break    
  room=add_objects_to_room(room,coordinates)
  return coordinates , room

In [ ]:
def location_wall4(item,room):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 
  if "chair" in item or "dc" in item :
    for i in range(0,colmns+1):
      if room[rows][i]==0 :
        coordinates.append([0,i])
        break
  if "closet" in item or "table" in item :
    for i in range(0,colmns+1):
      if i+1<=colmns and room[rows][i]==0 and room[rows][i+1]==0 :
        coordinates.append([rows,i])
        coordinates.append([rows,i+1])
        break
  if "desk" in item :
    for i in range(0,colmns+1):
      if i+2<=colmns and room[rows][i]==0 and room[rows][i+1]==0 and room[rows][i+2]==0:
        coordinates.append([rows,i])
        coordinates.append([rows,i+1])
        coordinates.append([rows,i+2])
        break    
  room=add_objects_to_room(room,coordinates)
  return coordinates , room

In [ ]:
def location_corners (item ,corner_num,room,door_wall,windows={}):
  #windows is a dictionary where the key is the wall number and the value is the window's coordinates 
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[]

  if (corner_num==1 and door_wall==2)or(corner_num==4 and door_wall==3)or(corner_num==3 and door_wall==4)or(corner_num==2 and door_wall==1):
    corner_row =0
    corner_colmn = colmns
    if "chair" in item or "dc" in item:
      if room[corner_row][corner_colmn]==0 :
        coordinates.append([corner_row,corner_colmn])
      elif room[corner_row][corner_colmn-1]==0 :
        coordinates.append([corner_row,corner_colmn-1]) 
      elif room[corner_row+1][corner_colmn]==0 :
        coordinates.append([corner_row+1,corner_colmn]) 
      elif room[corner_row+1][corner_colmn-1]==0 :
        coordinates.append([corner_row+1,corner_colmn-1]) 
    
    if "table" in item:
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn-1]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row,corner_colmn-1])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row+1][corner_colmn]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row+1,corner_colmn])
      #horizantol
      elif room[corner_row+1][corner_colmn] ==0 and room[corner_row+1][corner_colmn-1]==0:
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+1,corner_colmn-1])
      #vertical
      elif room[corner_row][corner_colmn-1] ==0 and room[corner_row+1][corner_colmn-1]==0:
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row+1,corner_colmn-1])
      #horizantol
      elif room[corner_row][corner_colmn-1]==0 and room[corner_row][corner_colmn-2] ==0:
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row,corner_colmn-2])
      #vertical
      elif room[corner_row+1][corner_colmn]==0 and room[corner_row+2][corner_colmn] ==0 :
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+2,corner_colmn])
      #horizantol
      elif room[corner_row+1][corner_colmn-1]==0 and room[corner_row+1][corner_colmn-2] ==0:
        coordinates.append([corner_row+1,corner_colmn-1])
        coordinates.append([corner_row+1,corner_colmn-2])
      #vertical
      elif room[corner_row+1][corner_colmn-1]==0 and room[corner_row+2][corner_colmn-1] ==0 :
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+2,corner_colmn])

    if "closet" in item :
      #windows locations in wall number 2
      wall2_win = []
      #windows locations in wall number 3
      wall3_win = [] 
      for k in windows :
        if k ==2:
          wall2_win.append(windows[k])
        if k == 3 :
          wall3_win.append(windows[k])

      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn-1]==0:
        #if there are no windows on wall 2 then we can put the closet directly
        if len(wall2_win) == 0 :
          coordinates.append([corner_row,corner_colmn])
          coordinates.append([corner_row,corner_colmn-1])
        #check window locations on wall number 2 so the closet wont block any window 
        elif len(wall2_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall2_win :
            if list(corner_row,corner_colmn)in window or list(corner_row,corner_colmn-1)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row,corner_colmn-1])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row+1][corner_colmn]==0:
        #if there are no windows on wall 3 then we can put the closet directly
        if len(wall3_win) == 0 :
          coordinates.append([corner_row,corner_colmn])
          coordinates.append([corner_row+1,corner_colmn])
        #check window locations on wall number 3 so the closet wont block any window 
        elif len(wall3_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall3_win :
            if list(corner_row,corner_colmn)in window or list(corner_row+1,corner_colmn)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row+1,corner_colmn])
      #horizantol
      elif room[corner_row][corner_colmn-1]==0 and room[corner_row][corner_colmn-2] ==0:
        #if there are no windows on wall 2 then we can put the closet directly
        if len(wall2_win) == 0 :
          coordinates.append([corner_row,corner_colmn-1])
          coordinates.append([corner_row,corner_colmn-2])
        #check window locations on wall number 2 so the closet wont block any window 
        elif len(wall2_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall2_win :
            if list(corner_row,corner_colmn-1)in window or list(corner_row,corner_colmn-2)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn-1])
            coordinates.append([corner_row,corner_colmn-2])
      #vertical
      elif room[corner_row+1][corner_colmn]==0 and room[corner_row+2][corner_colmn] ==0 :
        #if there are no windows on wall 3 then we can put the closet directly
        if len(wall3_win) == 0 :
          coordinates.append([corner_row+1,corner_colmn])
          coordinates.append([corner_row+2,corner_colmn])
        #check window locations on wall number 3 so the closet wont block any window 
        elif len(wall3_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall3_win :
            if list(corner_row+1,corner_colmn)in window or list(corner_row+2,corner_colmn)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row+1,corner_colmn])
            coordinates.append([corner_row+2,corner_colmn])

    if "desk" in item :
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn-1]==0 and room[corner_row][corner_colmn-2]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row,corner_colmn-2])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row+1][corner_colmn]==0 and room[corner_row+2][corner_colmn]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+2,corner_colmn])
      #horizantol
      elif room[corner_row+1][corner_colmn] ==0 and room[corner_row+1][corner_colmn-1]==0 and room[corner_row+1][corner_colmn-2]==0:
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+1,corner_colmn-1])
        coordinates.append([corner_row+1,corner_colmn-2])
      #vertical
      elif room[corner_row][corner_colmn-1] ==0 and room[corner_row+1][corner_colmn-1]==0 and room[corner_row+2][corner_colmn-1]==0:
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row+1,corner_colmn-1])
        coordinates.append([corner_row+2,corner_colmn-1])
      #horizantol
      elif room[corner_row][corner_colmn-1]==0 and room[corner_row][corner_colmn-2]==0 and room[corner_row][corner_colmn-3] ==0:
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row,corner_colmn-2])
        coordinates.append([corner_row,corner_colmn-3])
      #vertical
      elif room[corner_row+1][corner_colmn]==0 and room[corner_row+2][corner_colmn]==0 and room[corner_row+3][corner_colmn] ==0:
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+2,corner_colmn])
        coordinates.append([corner_row+3,corner_colmn])
      #horizantol
      elif room[corner_row+1][corner_colmn-1]==0 and room[corner_row+1][corner_colmn-2]==0 and room[corner_row+1][corner_colmn-3] ==0:
        coordinates.append([corner_row+1,corner_colmn-1])
        coordinates.append([corner_row+1,corner_colmn-2])
        coordinates.append([corner_row+1,corner_colmn-3])
      #vertical
      elif room[corner_row+1][corner_colmn-1]==0 and room[corner_row+2][corner_colmn-1]==0 and room[corner_row+3][corner_colmn-1] ==0:
        coordinates.append([corner_row+1,corner_colmn-1])
        coordinates.append([corner_row+2,corner_colmn-1])
        coordinates.append([corner_row+3,corner_colmn-1])

  if (corner_num==1 and door_wall==1) or (corner_num==2 and door_wall==4) or (corner_num==3 and door_wall==3 )or (corner_num==4 and door_wall==2 ):
    corner_row =0
    corner_colmn = 0
    if "chair" in item or "dc" in item: 
      if room[corner_row][corner_colmn]==0 :
        coordinates.append([corner_row,corner_colmn])
      elif room[corner_row][corner_colmn+1]==0 :
        coordinates.append([corner_row,corner_colmn+1]) 
      elif room[corner_row+1][corner_colmn]==0 :
        coordinates.append([corner_row+1,corner_colmn]) 
      elif room[corner_row+1][corner_colmn+1]==0 :
        coordinates.append([corner_row+1,corner_colmn+1]) 
    
    if "table" in item:
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn+1]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row,corner_colmn+1])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row+1][corner_colmn]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row+1,corner_colmn])
      #horizantol
      elif room[corner_row+1][corner_colmn] ==0 and room[corner_row+1][corner_colmn+1]==0:
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+1,corner_colmn+1])
      #vertical
      elif room[corner_row][corner_colmn+1] ==0 and room[corner_row+1][corner_colmn+1]==0:
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row+1,corner_colmn+1])
      #horizantol
      elif room[corner_row][corner_colmn+1]==0 and room[corner_row][corner_colmn+2] ==0:
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row,corner_colmn+2])
      #vertical
      elif room[corner_row+1][corner_colmn]==0 and room[corner_row+2][corner_colmn] ==0 :
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+2,corner_colmn])
      #horizantol
      elif room[corner_row+1][corner_colmn+1]==0 and room[corner_row+1][corner_colmn+2] ==0:
        coordinates.append([corner_row+1,corner_colmn+1])
        coordinates.append([corner_row+1,corner_colmn+2])
      #vertical
      elif room[corner_row+1][corner_colmn+1]==0 and room[corner_row+2][corner_colmn+1] ==0 :
        coordinates.append([corner_row+1,corner_colmn+1])
        coordinates.append([corner_row+2,corner_colmn+1])

    if "closet" in item:
      #windows locations in wall number 2
      wall2_win = []
      #windows locations in wall number 1
      wall1_win = [] 
      for k in windows :
        if k ==2:
          wall2_win.append(windows[k])
        if k == 1 :
          wall1_win.append(windows[k])
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn+1]==0:
        #if there are no windows on wall 2 then we can put the closet directly
        if len(wall2_win) == 0 :
          coordinates.append([corner_row,corner_colmn])
          coordinates.append([corner_row,corner_colmn+1])
        #check window locations on wall number 2 so the closet wont block any window 
        elif len(wall2_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall2_win :
            if list(corner_row,corner_colmn)in window or list(corner_row,corner_colmn+1)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row,corner_colmn+1])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row+1][corner_colmn]==0:
        #if there are no windows on wall 1 then we can put the closet directly
        if len(wall1_win) == 0 :
          coordinates.append([corner_row,corner_colmn])
          coordinates.append([corner_row+1,corner_colmn])
        #check window locations on wall number 1 so the closet wont block any window 
        elif len(wall1_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall1_win :
            if list(corner_row,corner_colmn)in window or list(corner_row+1,corner_colmn)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row+1,corner_colmn])
      #horizantol
      elif room[corner_row][corner_colmn+1]==0 and room[corner_row][corner_colmn+2] ==0:
        #if there are no windows on wall 2 then we can put the closet directly
        if len(wall2_win) == 0 :
          coordinates.append([corner_row,corner_colmn+1])
          coordinates.append([corner_row,corner_colmn+2])
        #check window locations on wall number 2 so the closet wont block any window 
        elif len(wall2_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall2_win :
            if list(corner_row,corner_colmn+1)in window or list(corner_row,corner_colmn+2)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn+1])
            coordinates.append([corner_row,corner_colmn+2])
      #vertical
      elif room[corner_row+1][corner_colmn]==0 and room[corner_row+2][corner_colmn] ==0 :
        #if there are no windows on wall 1 then we can put the closet directly
        if len(wall1_win) == 0 :
          coordinates.append([corner_row+1,corner_colmn])
          coordinates.append([corner_row+2,corner_colmn])
        #check window locations on wall number 1 so the closet wont block any window 
        elif len(wall1_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall1_win :
            if list(corner_row+1,corner_colmn)in window or list(corner_row+2,corner_colmn)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row+1,corner_colmn])
            coordinates.append([corner_row+2,corner_colmn])

    if "desk" in item :
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn+1]==0 and room[corner_row][corner_colmn+2]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row,corner_colmn+2])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row+1][corner_colmn]==0 and room[corner_row+2][corner_colmn]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+2,corner_colmn])
      #horizantol
      elif room[corner_row+1][corner_colmn] ==0 and room[corner_row+1][corner_colmn+1]==0 and room[corner_row+1][corner_colmn+2]==0:
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+1,corner_colmn+1])
        coordinates.append([corner_row+1,corner_colmn+2])
      #vertical
      elif room[corner_row][corner_colmn+1] ==0 and room[corner_row+1][corner_colmn+1]==0 and room[corner_row+2][corner_colmn+1]==0:
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row+1,corner_colmn+1])
        coordinates.append([corner_row+2,corner_colmn+1])
      #horizantol
      elif room[corner_row][corner_colmn+1]==0 and room[corner_row][corner_colmn+2]==0 and room[corner_row][corner_colmn+3] ==0:
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row,corner_colmn+2])
        coordinates.append([corner_row,corner_colmn+3])
      #vertical
      elif room[corner_row+1][corner_colmn]==0 and room[corner_row+2][corner_colmn]==0 and room[corner_row+3][corner_colmn] ==0:
        coordinates.append([corner_row+1,corner_colmn])
        coordinates.append([corner_row+2,corner_colmn])
        coordinates.append([corner_row+3,corner_colmn])
      #horizantol
      elif room[corner_row+1][corner_colmn+1]==0 and room[corner_row+1][corner_colmn+2]==0 and room[corner_row+1][corner_colmn+3] ==0:
        coordinates.append([corner_row+1,corner_colmn+1])
        coordinates.append([corner_row+1,corner_colmn+2])
        coordinates.append([corner_row+1,corner_colmn+3])
      #vertical
      elif room[corner_row+1][corner_colmn+1]==0 and room[corner_row+2][corner_colmn+1]==0 and room[corner_row+3][corner_colmn+1] ==0:
        coordinates.append([corner_row+1,corner_colmn+1])
        coordinates.append([corner_row+2,corner_colmn+1])
        coordinates.append([corner_row+3,corner_colmn+1])

  if (corner_num==1 and door_wall==4) or(corner_num==2 and door_wall==3) or (corner_num==3 and door_wall==2 )or (corner_num==4 and door_wall==1) :
    corner_row =rows
    corner_colmn = 0
    if "chair" in item or "dc" in item:
      if room[corner_row][corner_colmn]==0 :
        coordinates.append([corner_row,corner_colmn])
      elif room[corner_row][corner_colmn+1]==0 :
        coordinates.append([corner_row,corner_colmn+1]) 
      elif room[corner_row-1][corner_colmn]==0 :
        coordinates.append([corner_row-1,corner_colmn]) 
      elif room[corner_row-1][corner_colmn+1]==0 :
        coordinates.append([corner_row-1,corner_colmn+1]) 
    
    if "table" in item:
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn+1]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row,corner_colmn+1])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row-1][corner_colmn]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row-1,corner_colmn])
      #horizantol
      elif room[corner_row-1][corner_colmn] ==0 and room[corner_row-1][corner_colmn+1]==0:
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-1,corner_colmn+1])
      #vertical
      elif room[corner_row][corner_colmn+1] ==0 and room[corner_row-1][corner_colmn+1]==0:
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row-1,corner_colmn+1])
      #horizantol
      elif room[corner_row][corner_colmn+1]==0 and room[corner_row][corner_colmn+2] ==0:
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row,corner_colmn+2])
      #vertical
      elif room[corner_row-1][corner_colmn]==0 and room[corner_row-2][corner_colmn] ==0 :
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-2,corner_colmn])
      #horizantol
      elif room[corner_row-1][corner_colmn+1]==0 and room[corner_row-1][corner_colmn+2] ==0:
        coordinates.append([corner_row-1,corner_colmn+1])
        coordinates.append([corner_row-1,corner_colmn+2])
      #vertical
      elif room[corner_row-1][corner_colmn+1]==0 and room[corner_row-2][corner_colmn+1] ==0 :
        coordinates.append([corner_row-1,corner_colmn+1])
        coordinates.append([corner_row-2,corner_colmn+1])

    if "closet" in item:
      #windows locations in wall number 4
      wall4_win = []
      #windows locations in wall number 1
      wall1_win = [] 
      for k in windows :
        if k ==4:
          wall4_win.append(windows[k])
        if k == 1 :
          wall1_win.append(windows[k])
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn+1]==0:
        #if there are no windows on wall 4 then we can put the closet directly
        if len(wall4_win) == 0 :
          coordinates.append([corner_row,corner_colmn])
          coordinates.append([corner_row,corner_colmn+1])
        #check window locations on wall number 4 so the closet wont block any window 
        elif len(wall4_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall4_win :
            if list(corner_row,corner_colmn)in window or list(corner_row,corner_colmn+1)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row,corner_colmn+1])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row-1][corner_colmn]==0:
        #if there are no windows on wall 1 then we can put the closet directly
        if len(wall1_win) == 0 :
          coordinates.append([corner_row,corner_colmn])
          coordinates.append([corner_row-1,corner_colmn])
        #check window locations on wall number 1 so the closet wont block any window 
        elif len(wall1_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall1_win :
            if list(corner_row,corner_colmn)in window or list(corner_row-1,corner_colmn)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row-1,corner_colmn])
      #horizantol
      elif room[corner_row][corner_colmn+1]==0 and room[corner_row][corner_colmn+2] ==0:
      #if there are no windows on wall 4 then we can put the closet directly
        if len(wall4_win) == 0 :
          coordinates.append([corner_row,corner_colmn+1])
          coordinates.append([corner_row,corner_colmn+2])
        #check window locations on wall number 4 so the closet wont block any window 
        elif len(wall4_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall4_win :
            if list(corner_row,corner_colmn)in window or list(corner_row,corner_colmn+1)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row,corner_colmn+1])
      #vertical
      elif room[corner_row-1][corner_colmn]==0 and room[corner_row-2][corner_colmn] ==0 :
        #if there are no windows on wall 1 then we can put the closet directly
        if len(wall1_win) == 0 :
          coordinates.append([corner_row,corner_colmn+1])
          coordinates.append([corner_row,corner_colmn+2])
        #check window locations on wall number 1 so the closet wont block any window 
        elif len(wall1_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall1_win :
            if list(corner_row,corner_colmn+1)in window or list(corner_row,corner_colmn+2)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn+1])
            coordinates.append([corner_row,corner_colmn+2])

    if "desk" in item:
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn+1]==0 and room[corner_row][corner_colmn+2]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row,corner_colmn+2])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row-1][corner_colmn]==0 and room[corner_row-2][corner_colmn]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-2,corner_colmn])
      #horizantol
      elif room[corner_row-1][corner_colmn] ==0 and room[corner_row-1][corner_colmn+1]==0 and room[corner_row-1][corner_colmn+2]==0:
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-1,corner_colmn+1])
        coordinates.append([corner_row-1,corner_colmn+2])
      #vertical
      elif room[corner_row][corner_colmn+1] ==0 and room[corner_row-1][corner_colmn+1]==0 and room[corner_row-2][corner_colmn+1]==0:
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row-1,corner_colmn+1])
        coordinates.append([corner_row-2,corner_colmn+1])
      #horizantol
      elif room[corner_row][corner_colmn+1]==0 and room[corner_row][corner_colmn+2]==0 and room[corner_row][corner_colmn+3] ==0:
        coordinates.append([corner_row,corner_colmn+1])
        coordinates.append([corner_row,corner_colmn+2])
        coordinates.append([corner_row,corner_colmn+3])
      #vertical
      elif room[corner_row-1][corner_colmn]==0 and room[corner_row-2][corner_colmn]==0 and room[corner_row-3][corner_colmn] ==0:
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-2,corner_colmn])
        coordinates.append([corner_row-3,corner_colmn])
      #horizantol
      elif room[corner_row-1][corner_colmn+1]==0 and room[corner_row-1][corner_colmn+2]==0 and room[corner_row-1][corner_colmn+3] ==0:
        coordinates.append([corner_row-1,corner_colmn+1])
        coordinates.append([corner_row-1,corner_colmn+2])
        coordinates.append([corner_row-1,corner_colmn+3])
      #vertical
      elif room[corner_row-1][corner_colmn+1]==0 and room[corner_row-2][corner_colmn+1]==0 and room[corner_row-3][corner_colmn+1] ==0:
        coordinates.append([corner_row-1,corner_colmn+1])
        coordinates.append([corner_row-2,corner_colmn+1])
        coordinates.append([corner_row-3,corner_colmn+1])

  if (corner_num==1 and door_wall==3) or (corner_num==2 and door_wall==2) or (corner_num==3 and door_wall==1) or (corner_num==4 and door_wall==4 ):
    corner_row =rows
    corner_colmn = colmns
    if "chair" in item or "dc" in item: 
      if room[corner_row][corner_colmn]==0 :
        coordinates.append([corner_row,corner_colmn])
      elif room[corner_row][corner_colmn-1]==0 :
        coordinates.append([corner_row,corner_colmn-1]) 
      elif room[corner_row-1][corner_colmn]==0 :
        coordinates.append([corner_row-1,corner_colmn]) 
      elif room[corner_row-1][corner_colmn-1]==0 :
        coordinates.append([corner_row-1,corner_colmn-1]) 
    
    if "table" in item:
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn-1]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row,corner_colmn-1])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row-1][corner_colmn]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row-1,corner_colmn])
      #horizantol
      elif room[corner_row-1][corner_colmn] ==0 and room[corner_row-1][corner_colmn-1]==0:
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-1,corner_colmn-1])
      #vertical
      elif room[corner_row][corner_colmn-1] ==0 and room[corner_row-1][corner_colmn-1]==0:
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row-1,corner_colmn-1])
      #horizantol
      elif room[corner_row][corner_colmn-1]==0 and room[corner_row][corner_colmn-2] ==0:
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row,corner_colmn-2])
      #vertical
      elif room[corner_row-1][corner_colmn]==0 and room[corner_row-2][corner_colmn] ==0 :
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-2,corner_colmn])
      #horizantol
      elif room[corner_row-1][corner_colmn-1]==0 and room[corner_row-1][corner_colmn-2] ==0:
        coordinates.append([corner_row-1,corner_colmn-1])
        coordinates.append([corner_row-1,corner_colmn-2])
      #vertical
      elif room[corner_row-1][corner_colmn-1]==0 and room[corner_row-2][corner_colmn-1] ==0 :
        coordinates.append([corner_row-1,corner_colmn-1])
        coordinates.append([corner_row-2,corner_colmn-1])

    if "closet" in item :
      #windows locations in wall number 4
      wall4_win = []
      #windows locations in wall number 3
      wall3_win = [] 
      for k in windows :
        if k ==4:
          wall4_win.append(windows[k])
        if k == 3 :
          wall3_win.append(windows[k])
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn-1]==0:
        #if there are no windows on wall 4 then we can put the closet directly
        if len(wall4_win) == 0 :
          coordinates.append([corner_row,corner_colmn])
          coordinates.append([corner_row,corner_colmn-1])
        #check window locations on wall number 4 so the closet wont block any window 
        elif len(wall4_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall4_win :
            if list(corner_row,corner_colmn)in window or list(corner_row,corner_colmn-1)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row,corner_colmn-1])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row-1][corner_colmn]==0:
        #if there are no windows on wall 3 then we can put the closet directly
        if len(wall3_win) == 0 :
          coordinates.append([corner_row,corner_colmn])
          coordinates.append([corner_row-1,corner_colmn])
        #check window locations on wall number 3 so the closet wont block any window 
        elif len(wall3_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall3_win :
            if list(corner_row,corner_colmn)in window or list(corner_row-1,corner_colmn)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn])
            coordinates.append([corner_row-1,corner_colmn])
      #horizantol
      elif room[corner_row][corner_colmn-1]==0 and room[corner_row][corner_colmn-2] ==0:
        #if there are no windows on wall 4 then we can put the closet directly
        if len(wall4_win) == 0 :
          coordinates.append([corner_row,corner_colmn-1])
          coordinates.append([corner_row,corner_colmn-2])
        #check window locations on wall number 4 so the closet wont block any window 
        elif len(wall4_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall4_win :
            if list(corner_row,corner_colm-1)in window or list(corner_row,corner_colmn-2)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row,corner_colmn-1])
            coordinates.append([corner_row,corner_colmn-2])
      #vertical
      elif room[corner_row-1][corner_colmn]==0 and room[corner_row-2][corner_colmn] ==0 :
        #if there are no windows on wall 3 then we can put the closet directly
        if len(wall3_win) == 0 :
          coordinates.append([corner_row-1,corner_colmn])
          coordinates.append([corner_row-2,corner_colmn])
        #check window locations on wall number 3 so the closet wont block any window 
        elif len(wall3_win) != 0 :
          # first we assume the closet location is not the same as the window's location
          location = False
          for window in wall3_win :
            if list(corner_row-1,corner_colmn)in window or list(corner_row-2,corner_colmn)in window :
              location == True
              break 
          if location == False :
            coordinates.append([corner_row-1,corner_colmn])
            coordinates.append([corner_row-2,corner_colmn])
    if "desk" in item :
      #horizantol
      if room[corner_row][corner_colmn] ==0 and room[corner_row][corner_colmn-1]==0 and room[corner_row][corner_colmn-2]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row,corner_colmn-2])
      #vertical
      elif room[corner_row][corner_colmn] ==0 and room[corner_row-1][corner_colmn]==0 and room[corner_row-2][corner_colmn]==0:
        coordinates.append([corner_row,corner_colmn])
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-2,corner_colmn])
      #horizantol
      elif room[corner_row-1][corner_colmn] ==0 and room[corner_row-1][corner_colmn-1]==0 and room[corner_row-1][corner_colmn-2]==0:
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-1,corner_colmn-1])
        coordinates.append([corner_row-1,corner_colmn-2])
      #vertical
      elif room[corner_row][corner_colmn-1] ==0 and room[corner_row-1][corner_colmn-1]==0 and room[corner_row-2][corner_colmn-1]==0:
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row-1,corner_colmn-1])
        coordinates.append([corner_row-2,corner_colmn-1])
      #horizantol
      elif room[corner_row][corner_colmn-1]==0 and room[corner_row][corner_colmn-2]==0 and room[corner_row][corner_colmn-3] ==0:
        coordinates.append([corner_row,corner_colmn-1])
        coordinates.append([corner_row,corner_colmn-2])
        coordinates.append([corner_row,corner_colmn-3])
      #vertical
      elif room[corner_row-1][corner_colmn]==0 and room[corner_row-2][corner_colmn]==0 and room[corner_row-3][corner_colmn] ==0:
        coordinates.append([corner_row-1,corner_colmn])
        coordinates.append([corner_row-2,corner_colmn])
        coordinates.append([corner_row-3,corner_colmn])
      #horizantol
      elif room[corner_row-1][corner_colmn-1]==0 and room[corner_row-1][corner_colmn-2]==0 and room[corner_row-1][corner_colmn-3] ==0:
        coordinates.append([corner_row-1,corner_colmn-1])
        coordinates.append([corner_row-1,corner_colmn-2])
        coordinates.append([corner_row-1,corner_colmn-3])
      #vertical
      elif room[corner_row-1][corner_colmn-1]==0 and room[corner_row-2][corner_colmn-1]==0 and room[corner_row-3][corner_colmn-1] ==0:
        coordinates.append([corner_row-1,corner_colmn-1])
        coordinates.append([corner_row-2,corner_colmn-1])
        coordinates.append([corner_row-3,corner_colmn-1])
  room=add_objects_to_room(room,coordinates)
  return coordinates , room

In [ ]:
def location_room_direction_middle(item,room):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 
  mid_row = int(rows/2)
  mid_col=int(colmns/2)

  if "chair" in item or "dc" in item :
    if room[mid_row][mid_col] == 0:
      coordinates.append([mid_row,mid_col])
    elif room[mid_row+1][mid_col] == 0:
      coordinates.append([mid_row+1,mid_col])
    elif room[mid_row-1][mid_col] == 0:
      coordinates.append([mid_row-1,mid_col])
    elif room[mid_row][mid_col+1] == 0:
      coordinates.append([mid_row,mid_col+1])
    elif room[mid_row][mid_col-1] == 0:
      coordinates.append([mid_row,mid_col-1])
    elif room[mid_row+1][mid_col+1] == 0:
      coordinates.append([mid_row+1,mid_col+1])
    elif room[mid_row-1][mid_col+1] == 0:
      coordinates.append([mid_row-1,mid_col+1])
    elif room[mid_row+1][mid_col-1] == 0:
      coordinates.append([mid_row,mid_col+1])
    elif room[mid_row-1][mid_col-1] == 0:
      coordinates.append([mid_row,mid_col-1])
  
  if "table" in item or "closet" in item :
    if room[mid_row][mid_col] == 0 and room[mid_row][mid_col+1] == 0  :
      coordinates.append([mid_row,mid_col])
      coordinates.append([mid_row,mid_col+1])
    elif room[mid_row][mid_col] == 0 and room[mid_row][mid_col-1] == 0  :
      coordinates.append([mid_row,mid_col])
      coordinates.append([mid_row,mid_col-1])
    elif room[mid_row][mid_col] == 0 and room[mid_row+1][mid_col] == 0  :
      coordinates.append([mid_row,mid_col])
      coordinates.append([mid_row+1,mid_col])
    elif room[mid_row][mid_col] == 0 and room[mid_row-1][mid_col] == 0  :
      coordinates.append([mid_row,mid_col])
      coordinates.append([mid_row-1,mid_col])
    elif room[mid_row+1][mid_col] == 0 and room[mid_row+1][mid_col+1] == 0  :
      coordinates.append([mid_row+1,mid_col])
      coordinates.append([mid_row+1,mid_col+1])
    elif room[mid_row+1][mid_col] == 0 and room[mid_row+1][mid_col-1] == 0  :
      coordinates.append([mid_row+1,mid_col])
      coordinates.append([mid_row+1,mid_col-1])
    elif room[mid_row-1][mid_col] == 0 and room[mid_row-1][mid_col+1] == 0  :
      coordinates.append([mid_row-1,mid_col])
      coordinates.append([mid_row-1,mid_col+1])
    elif room[mid_row-1][mid_col] == 0 and room[mid_row-1][mid_col-1] == 0  :
      coordinates.append([mid_row-1,mid_col])
      coordinates.append([mid_row-1,mid_col-1])
    elif room[mid_row][mid_col+1] == 0 and room[mid_row+1][mid_col+1] == 0  :
      coordinates.append([mid_row,mid_col+1])
      coordinates.append([mid_row+1,mid_col+1]) 
    elif room[mid_row][mid_col+1] == 0 and room[mid_row-1][mid_col+1] == 0  :
      coordinates.append([mid_row,mid_col+1])
      coordinates.append([mid_row-1,mid_col+1]) 
    elif room[mid_row][mid_col-1] == 0 and room[mid_row+1][mid_col-1] == 0  :
      coordinates.append([mid_row,mid_col-1])
      coordinates.append([mid_row+1,mid_col-1]) 
    elif room[mid_row][mid_col-1] == 0 and room[mid_row-1][mid_col-1] == 0  :
      coordinates.append([mid_row,mid_col-1])
      coordinates.append([mid_row-1,mid_col-1])
  if "desk" in item :
    if room[mid_row][mid_col-1] == 0 and  room[mid_row][mid_col] == 0 and room[mid_row][mid_col+1] == 0  :
      coordinates.append([mid_row,mid_col-1])
      coordinates.append([mid_row,mid_col])
      coordinates.append([mid_row,mid_col+1])
    elif room[mid_row+1][mid_col-1] == 0 and  room[mid_row+1][mid_col] == 0 and room[mid_row+1][mid_col+1] == 0  :
      coordinates.append([mid_row+1,mid_col-1])
      coordinates.append([mid_row+1,mid_col])
      coordinates.append([mid_row+1,mid_col+1])
    elif room[mid_row-1][mid_col-1] == 0 and  room[mid_row-1][mid_col] == 0 and room[mid_row-1][mid_col+1] == 0  :
      coordinates.append([mid_row-1,mid_col-1])
      coordinates.append([mid_row-1,mid_col])
      coordinates.append([mid_row-1,mid_col+1])
    elif room[mid_row-1][mid_col] == 0 and  room[mid_row][mid_col] == 0 and room[mid_row+1][mid_col] == 0  :
      coordinates.append([mid_row-1,mid_col])
      coordinates.append([mid_row,mid_col])
      coordinates.append([mid_row+1,mid_col])
    elif room[mid_row-1][mid_col+1] == 0 and  room[mid_row][mid_col+1] == 0 and room[mid_row+1][mid_col+1] == 0  :
      coordinates.append([mid_row-1,mid_col+1])
      coordinates.append([mid_row,mid_col+1])
      coordinates.append([mid_row+1,mid_col+1])
    elif room[mid_row-1][mid_col-1] == 0 and  room[mid_row][mid_col-1] == 0 and room[mid_row+1][mid_col-1] == 0  :
      coordinates.append([mid_row-1,mid_col-1])
      coordinates.append([mid_row,mid_col-1])
      coordinates.append([mid_row+1,mid_col-1])
  room = add_objects_to_room(room,coordinates)
  return coordinates, room 

In [ ]:
def location_door_direction_right(item ,room,door_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[]
  
  door_rows  , door_colmns  , _ = is_ver_horz(door_location,door_wall)
  door_row0 = door_rows [0]
  door_colmn0 = door_colmns [0]
  door_rown  = door_rows [-1]
  door_colmnn  = door_colmns [-1]

  if "chair" in item or "dc" in item :
      if door_row0 == 0 and door_wall == 2 :
        for i in range(searchstart,searchrange-1):
          if door_colmn0-i >=0 and room[door_row0][door_colmn0-i] == 0 :
            coordinates.append([door_row0,door_colmn0-i])
            break
      elif door_row0== rows and door_wall == 4:
        for i in range(searchstart,searchrange-1):
          if door_colmnn +i <= colmns and room[door_row0][door_colmnn +i] == 0 :
            coordinates.append([door_row0,door_colmnn +i])
            break
      elif door_colmn0== 0 and door_wall == 1:
        for i in range(searchstart,searchrange-1):
          if door_rown +i <=rows and room[door_rown +i][door_colmn0] == 0 :
            coordinates.append([door_rown +i,door_colmn0])
            break
      elif door_colmn0 == colmns and door_wall == 3:
        for i in range(searchstart,searchrange-1):
          if door_row0-i >=0 and room[door_row0-i][door_colmn0] == 0 :
            coordinates.append([door_row0-i,door_colmn0])
            break
  
  if "closet" in item or "table" in item :
        if door_row0 == 0 and door_wall == 2 :
          for i in range (searchstart,searchrange):
            if door_colmn0-i >0 and door_colmn0-(i+1) >=0 and room[door_row0][door_colmn0-i] == 0 and room[door_row0][door_colmn0-(i+1)] == 0:
              coordinates.append([door_row0,door_colmn0-(i+1)])
              coordinates.append([door_row0,door_colmn0-i])
              break  
        elif door_row0 == rows and door_wall == 4:
          for i in range (searchstart,searchrange):
            if door_colmnn +i <colmns and door_colmnn +(i+1) <= colmns and room[door_row0][door_colmnn +i] == 0 and room[door_row0][door_colmnn +(i+1)] == 0 :
              coordinates.append([door_row0,door_colmnn +(i+1)])
              coordinates.append([door_row0,door_colmnn +i])
              break
        elif door_colmn0== 0 and door_wall == 1:
          for i in range (searchstart,searchrange):
            if door_rown +i <rows and door_rown +(i+1) <=rows and room[door_rown +i][door_colmn0]==0 and room[door_rown +(i+1)][door_colmn0]==0 :
              coordinates.append([door_rown +(i+1),door_colmn0])
              coordinates.append([door_rown +i,door_colmn0])
              break
        elif door_colmn0 == colmns and door_wall == 3:
          for i in range (searchstart,searchrange):
            if door_row0-i >0 and door_row0-(i+1) >=0 and room[door_row0-i][door_colmn0]==0 and room[door_row0-(i+1)][door_colmn0]==0 :
              coordinates.append([door_row0-(i+1),door_colmn0])
              coordinates.append([door_row0-i,door_colmn0])
              break
  if "desk" in item :
      if door_row0== 0 and door_wall == 2 :
        for i in range (searchstart,searchrange):
          if door_colmn0-i >0 and door_colmn0-(i+1) >0 and door_colmn0-(i+2) >=0 and room[door_row0][door_colmn0-i] == 0 and room[door_row0][door_colmn0-(i+1)] == 0 and room[door_row0][door_colmn0-(i+2)] == 0 :
            coordinates.append([door_row0,door_colmn0-i])
            coordinates.append([door_row0,door_colmn0-(i+1)])
            coordinates.append([door_row0,door_colmn0-(i+2)])
            break
      elif door_row0== rows and door_wall == 4:
        for i in range (searchstart,searchrange):
          if door_colmnn +i<colmns and door_colmnn +(i+1)<colmns and door_colmnn +(i+2) <=colmns and room[door_row0][door_colmnn +i] == 0 and room[door_row0][door_colmnn +(i+1)] == 0 and room[door_row0][door_colmnn +(i+2)] == 0 :
            coordinates.append([door_row0,door_colmnn +i])
            coordinates.append([door_row0,door_colmnn +(i+1)])
            coordinates.append([door_row0,door_colmnn +(i+2)])
            break
      elif door_colmn0== 0 and door_wall == 1:
        for i in range (searchstart,searchrange):
          if door_rown +i <rows and door_rown +(i+1) <rows and door_rown +(i+2) <=rows and room[door_rown +i][door_colmn0]==0 and room[door_rown +(i+1)][door_colmn0]==0 and room[door_rown +(i+2)][door_colmn0]==0 :
            coordinates.append([door_rown +i,door_colmn0])
            coordinates.append([door_rown +(i+1),door_colmn0])
            coordinates.append([door_rown +(i+2),door_colmn0])
            break
      elif door_colmn0== colmns and door_wall == 3:
        for i in range (searchstart,searchrange):
          if door_row0-i >0 and door_row0-(i+1) >0 and door_row0-(i+2) >=0 and room[door_row0-i][door_colmn0]==0 and room[door_row0-(i+1)][door_colmn0]==0 and room[door_row0-(i+2)][door_colmn0]==0 :
            coordinates.append([door_row0-i,door_colmn0])
            coordinates.append([door_row0-(i+1),door_colmn0])
            coordinates.append([door_row0-(i+2),door_colmn0])
            break   
  room=add_objects_to_room(room,coordinates)
  return coordinates , room 

In [ ]:
def location_door_direction_left(item ,room,door_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[]
  
  door_rows , door_colmns , _ = is_ver_horz(door_location,door_wall)
  door_row0 = door_rows[0]
  door_colmn0 = door_colmns[0]
  door_rown = door_rows[-1]
  door_colmnn = door_colmns[-1]

  if "chair" in item or "dc" in item :
      if door_row0== 0 and door_wall == 2 : 
        for i in  range (searchstart,searchrange-1):
          if door_colmnn+i <= colmns and room[door_row0][door_colmnn+i] == 0 :
            coordinates.append([door_row0,door_colmnn+i])
            break
      elif door_row0== rows and door_wall == 4 :
        for i in  range (searchstart,searchrange-1):
          if door_colmn0-i >= 0 and room[door_row0][door_colmn0-i] == 0 :
            coordinates.append([door_row0,door_colmn0-i])
            break
      elif door_colmn0== 0 and door_wall == 1 :
        for i in range (searchstart,searchrange-1) :
          if door_row0-i >=0 and room[door_row0-i][door_colmn0] == 0 :
            coordinates.append([door_row0-i,door_colmn0])
            break
      elif door_colmn0== colmns and door_wall == 3 :
        for i in range(searchstart,searchrange-1):
          if door_rown+i <=rows and room[door_rown+i][door_colmn0] == 0 :
            coordinates.append([door_rown+i,door_colmn0])
            break

  if "closet" in item or "table" in item :
      if door_row0== 0 and door_wall == 2 :
        for i in range (searchstart,searchrange):
          if door_colmnn+i < colmns and door_colmnn+(i+1) <= colmns and room[door_row0][door_colmnn+i] == 0 and room[door_row0][door_colmnn+(i+1)] == 0 :
              coordinates.append([door_row0,door_colmnn+i])
              coordinates.append([door_row0,door_colmnn+(i+1)])
              break
      elif door_row0== rows and door_wall == 4:
        for i in range (searchstart,searchrange):
          if door_colmn0-i >0 and door_location[1]-(i+1) >=0 and room[door_row0][door_colmn0-i] == 0 and room[door_row0][door_colmn0-(i+1)] == 0 :
              coordinates.append([door_row0,door_colmn0-i])
              coordinates.append([door_row0,door_colmn0-(i+1)])
              break
      elif door_colmn0== 0 and door_wall == 1:
        for i in range (searchstart,searchrange):
          if door_row0-i >0 and door_row0-(i+1) >=0 and room[door_row0-i][door_colmn0]==0 and room[door_row0-(i+1)][door_colmn0]==0 :
              coordinates.append([door_row0-i,door_colmn0])
              coordinates.append([door_row0-(i+1),door_colmn0])
              break
      elif door_colmn0== colmns and door_wall == 3:
        for i in range (searchstart,searchrange):
          if door_rown+i <rows and door_rown+(i+1) <=rows and room[door_rown+i][door_colmn0]==0 and room[door_rown+(i+1)][door_colmn0]==0 :
              coordinates.append([door_rown+i,door_colmn0])
              coordinates.append([door_rown+(i+1),door_colmn0])
              break  

  if "desk" in item :
      if door_row0== 0 and door_wall == 2 :
        for i in range (searchstart,searchrange):
          if door_colmnn+i < colmns and door_colmnn+(i+1) < colmns and door_colmnn+(i+2) <= colmns and room[door_row0][door_colmnn+i] == 0 and room[door_row0][door_colmnn+(i+1)] == 0 and room[door_row0][door_colmnn+(i+2)] == 0 :
            coordinates.append([door_row0,door_colmnn+i])
            coordinates.append([door_row0,door_colmnn+(i+1)])
            coordinates.append([door_row0,door_colmnn+(i+2)])
            break
      elif door_row0== rows and door_wall == 4:
        for i in range (searchstart,searchrange):
          if door_colmn0-i >0 and door_colmn0-(i+1) >0 and door_colmn0-(i+2) >=0 and room[door_row0][door_colmn0-i] == 0 and room[door_row0][door_colmn0-(i+1)] == 0 and room[door_row0][door_colmn0-(i+2)] == 0 :
            coordinates.append([door_row0,door_colmn0-i])
            coordinates.append([door_row0,door_colmn0-(i+1)])
            coordinates.append([door_row0,door_colmn0-(i+2)])
            break
      elif door_colmn0== 0 and door_wall == 1:
        for i in range (searchstart,searchrange):
          if door_row0-i >0 and door_row0-(i+1) >0 and door_row0-(i+2) >=0 and room[door_row0-i][door_colmn0]==0 and room[door_row0-(i+1)][door_colmn0]==0 and room[door_row0-(i+2)][door_colmn0]==0 :
            coordinates.append([door_row0-i,door_colmn0])
            coordinates.append([door_row0-(i+1),door_colmn0])
            coordinates.append([door_row0-(i+2),door_colmn0])
            break 
      elif door_location[1]== colmns and door_wall == 3:
        for i in range (searchstart,searchrange):
          if door_rown+i <rows and door_rown+(i+1) <rows and door_rown+(i+2) <=rows and room[door_rown+i][door_colmn0]==0 and room[door_rown+(i+1)][door_colmn0]==0 and room[door_rown+(i+2)][door_colmn0]==0 :
            coordinates.append([door_rown+i,door_colmn0])
            coordinates.append([door_rown+(i+1),door_colmn0])
            coordinates.append([door_rown+(i+2),door_colmn0])
            break
  room=add_objects_to_room(room,coordinates)
  return coordinates , room

In [ ]:
def location_door_direction_infront_oppsite(item ,room,door_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[]   

  door_rows , door_colmns , _ = is_ver_horz(door_location,door_wall)
  door_row0 = door_rows[0]
  door_colmn0 = door_colmns[0]
  door_rown = door_rows[-1]
  door_colmnn = door_colmns[-1]

  if "chair" in item :
      if door_row0== 0 and door_wall == 2 :
        for i in range (door_row0+searchstart,door_row0+searchrange):
          for col in door_colmns :
            if room[i][col] == 0 :
              coordinates.append([i,col])
              break
          if len(coordinates)!=0:
            break
      elif door_row0== rows and door_wall == 4:
        for i in range(door_row0-searchstart, door_row0-searchrange ,-1):
          for col in door_colmns :
            if room[i][col] == 0 :
              coordinates.append([i,col])
              break
          if len(coordinates)!=0:
            break
      elif door_colmn0== 0 and door_wall == 1:
        for i in range ( door_colmn0+searchstart,door_colmn0+searchrange):
          for row in door_rows :
            if room[row][i] == 0 :
              coordinates.append([row,i])
              break
          if len(coordinates)!=0:
            break
      elif door_colmn0== colmns and door_wall == 3 :
        for i in range (door_colmn0-searchstart , door_colmn0-searchrange, -1):
          for row in door_rows :
            if room[row][i] == 0 :
              coordinates.append([row,i])
              break
          if len(coordinates)!=0:
            break

  if "closet" in item or "table" in item :
      if door_row0== 0 and door_wall == 2 :
        for i in range (door_row0+searchstart,door_row0+searchrange):
          for col in range(len(door_colmns)) :
            if room[i][door_colmns[col]] == 0 and room[i][door_colmns[col]+1] and door_colmns[col]+1 in door_colmns :
              coordinates.append([i,door_colmns[col]])
              coordinates.append([i,door_colmns[col]+1])
              break
          if len(coordinates)!=0:
            break
      elif door_row0== rows and door_wall == 4:
        for i in range(door_row0-searchstart, door_row0-searchrange ,-1):
          for col in range(len(door_colmns)) :
            if room[i][door_colmns[col]] == 0 and room[i][door_colmns[col]+1] and door_colmns[col]+1 in door_colmns :
              coordinates.append([i,door_colmns[col]])
              coordinates.append([i,door_colmns[col]+1])
              break
          if len(coordinates)!=0:
            break
      elif door_colmn0== 0 and door_wall == 1:
        for i in range ( door_colmn0+searchstart,door_colmn0+searchrange):
          for row in range(len(door_rows)) :
            if room[door_rows[row]][i] == 0 and room[door_rows[row]+1][i] == 0 and door_rows[row]+1 in door_rows :
              coordinates.append([door_rows[row],i])
              coordinates.append([door_rows[row]+1,i])
              break
          if len(coordinates)!=0:
            break
      elif door_colmn0== colmns and door_wall == 3 :
        for i in range (door_colmn0-searchstart , door_colmn0-searchrange, -1):
          for row in range(len(door_rows)) :
            if room[door_rows[row]][i] == 0 and room[door_rows[row]+1][i] == 0 and door_rows[row]+1 in door_rows :
              coordinates.append([door_rows[row],i])
              coordinates.append([door_rows[row]+1,i])
              break
          if len(coordinates)!=0:
            break

  if "desk" in item  :
    if len(door_location)>=3:
      if door_row0== 0 and door_wall == 2 :
        for i in range (door_row0+searchstart,door_row0+searchrange):
          for col in range(len(door_colmns)) :
            if room[i][door_colmns[col]] == 0 and room[i][door_colmns[col]+1] and room[i][door_colmns[col]+2]==0 and door_colmns[col]+1 in door_colmns and door_colmns[col]+2 in door_colmns :
              coordinates.append([i,door_colmns[col]])
              coordinates.append([i,door_colmns[col]+1])
              coordinates.append([i,door_colmns[col]+2])
              break
          if len(coordinates)!=0:
            break
      elif door_row0== rows and door_wall == 4:
        for i in range(door_row0-searchstart, door_row0-searchrange ,-1):
          for col in range (len(door_colmns)) :
            if room[i][door_colmns[col]] == 0 and room[i][door_colmns[col]+1] and room[i][door_colmns[col]+2]==0 and door_colmns[col]+1 in door_colmns and door_colmns[col]+2 in door_colmns :
              coordinates.append([i,door_colmns[col]])
              coordinates.append([i,door_colmns[col]+1])
              coordinates.append([i,door_colmns[col]+2])
          if len(coordinates)!=0:
            break
      elif door_colmn0== 0 and door_wall == 1:
        for i in range ( door_colmn0+searchstart,door_colmn0+searchrange):
          for row in range(len(door_rows) ):
            if room[door_rows[row]][i] == 0 and room[door_rows[row]+1][i] == 0 and room[door_rows[row]+2][i] == 0 and door_rows[row]+1 in door_rows and door_rows[row] in door_rows :
              coordinates.append([door_rows[row],i])
              coordinates.append([door_rows[row]+1,i])
              coordinates.append([door_rows[row]+2,i])
              break
          if len(coordinates)!=0:
            break
      elif door_colmn0== colmns and door_wall == 3 :
        for i in range (door_colmn0-searchstart , door_colmn0-searchrange, -1):
          for row in range(len(door_rows)) :
            if room[door_rows[row]][i] == 0 and room[door_rows[row]+1][i] == 0 and room[door_rows[row]+2][i] == 0 and door_rows[row]+1 in door_rows and door_rows[row] in door_rows :
              coordinates.append([door_rows[row],i])
              coordinates.append([door_rows[row]+1,i])
              coordinates.append([door_rows[row]+2,i])
              break
          if len(coordinates)!=0:
            break
    elif len(door_location)==2:
      if door_row0== 0 and door_wall == 2 :
        for i in range (door_row0+searchstart,door_row0+searchrange):
          for col in range(len(door_colmns)) :
            if room[i][door_colmns[col]] == 0 and room[i][door_colmns[col]+1] and door_colmns[col]+1 in door_colmns and door_colmns[col]+2<=colmns and room[i][door_colmns[col]+2]==0:
              coordinates.append([i,door_colmns[col]])
              coordinates.append([i,door_colmns[col]+1])
              coordinates.append([i,door_colmns[col]+2])
              break
            elif room[i][door_colmns[col]] == 0 and room[i][door_colmns[col]+1] and door_colmns[col]+1 in door_colmns and door_colmns[col]-1>=0 and room[i][door_colmns[col]-1]==0:
              coordinates.append([i,door_colmns[col]-1])
              coordinates.append([i,door_colmns[col]])
              coordinates.append([i,door_colmns[col]+1])
              break
          if len(coordinates)!=0:
            break
      elif door_row0== rows and door_wall == 4:
        for i in range(door_row0-searchstart, door_row0-searchrange ,-1):
          for col in range (len(door_colmns)) :
            if room[i][door_colmns[col]] == 0 and room[i][door_colmns[col]+1] and door_colmns[col]+1 in door_colmns and door_colmns[col]+2<=colmns and room[i][door_colmns[col]+2]==0:
              coordinates.append([i,door_colmns[col]])
              coordinates.append([i,door_colmns[col]+1])
              coordinates.append([i,door_colmns[col]+2])
              break
            elif room[i][door_colmns[col]] == 0 and room[i][door_colmns[col]+1] and door_colmns[col]+1 in door_colmns and door_colmns[col]-1>=0 and room[i][door_colmns[col]-1]==0:
              coordinates.append([i,door_colmns[col]-1])
              coordinates.append([i,door_colmns[col]])
              coordinates.append([i,door_colmns[col]+1])
              break
          if len(coordinates)!=0:
            break
      elif door_colmn0== 0 and door_wall == 1:
        for i in range ( door_colmn0+searchstart,door_colmn0+searchrange):
          for row in range(len(door_rows) ):
            if room[door_rows[row]][i] == 0 and room[door_rows[row]+1][i] == 0 and door_rows[row]+1 in door_rows and door_rows[row]+2<=rows and room[door_rows[row]+2][i] == 0:
              coordinates.append([door_rows[row],i])
              coordinates.append([door_rows[row]+1,i])
              coordinates.append([door_rows[row]+2,i])
              break
            elif room[door_rows[row]][i] == 0 and room[door_rows[row]+1][i] == 0 and door_rows[row]+1 in door_rows and door_rows[row]-1>=0 and room[door_rows[row]-1][i] == 0:
              coordinates.append([door_rows[row]-1,i])
              coordinates.append([door_rows[row],i])
              coordinates.append([door_rows[row]+1,i])
              break
          if len(coordinates)!=0:
            break
      elif door_colmn0== colmns and door_wall == 3 :
        for i in range (door_colmn0-searchstart , door_colmn0-searchrange, -1):
          for row in range(len(door_rows)) :
            if room[door_rows[row]][i] == 0 and room[door_rows[row]+1][i] == 0 and door_rows[row]+1 in door_rows and door_rows[row]+2<=rows and room[door_rows[row]+2][i] == 0:
              coordinates.append([door_rows[row],i])
              coordinates.append([door_rows[row]+1,i])
              coordinates.append([door_rows[row]+2,i])
              break
            elif room[door_rows[row]][i] == 0 and room[door_rows[row]+1][i] == 0 and door_rows[row]+1 in door_rows and door_rows[row]-1>=0 and room[door_rows[row]-1][i] == 0:
              coordinates.append([door_rows[row]-1,i])
              coordinates.append([door_rows[row],i])
              coordinates.append([door_rows[row]+1,i])
              break
          if len(coordinates)!=0:
            break

  room=add_objects_to_room(room,coordinates)
  return coordinates , room

In [ ]:
def location_door_direction_beside_next(item ,room,door_location,door_wall,searchstart,searchrange):
  coordinates , room =location_door_direction_right(item=item ,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates) ==0:
    coordinates , room = location_door_direction_left(item=item ,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  return coordinates , room

In [ ]:
def location_door_direction_near(item ,room,door_location,door_wall,searchstart,searchrange):
  coordinates , room =location_door_direction_right(item=item ,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates) ==0:
    coordinates , room = location_door_direction_left(item=item ,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates) == 0:
    coordinates , room = location_door_direction_infront_oppsite(item=item ,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  return coordinates , room 

In [ ]:

def location_window_direction_right(item ,room,window_location,window_wall,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[]
  
  window_rows , window_colmns , _ = is_ver_horz(window_location,window_wall)
  window_row0 = window_rows[0]
  window_colmn0 = window_colmns[0]
  window_rown  = window_rows[-1]
  window_colmnn  = window_colmns[-1]
  if door_wall==2:
    if "chair" in item or "dc" in item :
      if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
        for i in range(searchstart,searchrange-1):
          if window_colmn0-i >=0 and room[window_row0][window_colmn0-i] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range(searchstart,searchrange-1):
          if window_rown +i <=rows and room[window_rown +i][window_colmn0] == 0 :
            coordinates.append([window_rown +i,window_colmn0])
            break
      elif window_colmn0 == colmns and window_wall == 3:
        for i in range(searchstart,searchrange-1):
          if window_row0-i >=0 and room[window_row0-i][window_colmn0] == 0 :
            coordinates.append([window_row0-i,window_colmn0])
            break
    if "closet" in item or "table" in item :
        if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
          for i in range (searchstart,searchrange):
            if window_colmn0-i >0 and window_colmn0-(i+1) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0:
              coordinates.append([window_row0,window_colmn0-(i+1)])
              coordinates.append([window_row0,window_colmn0-i])
              break  
        elif window_colmn0== 0 and window_wall == 1:
          for i in range (searchstart,searchrange):
            if window_rown +i <rows and window_rown +(i+1) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 :
              coordinates.append([window_rown +(i+1),window_colmn0])
              coordinates.append([window_rown +i,window_colmn0])
              break
        elif window_colmn0 == colmns and window_wall == 3:
          for i in range (searchstart,searchrange):
            if window_row0-i >0 and window_row0-(i+1) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 :
              coordinates.append([window_row0-(i+1),window_colmn0])
              coordinates.append([window_row0-i,window_colmn0])
              break
    if "desk" in item :
      if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
        for i in range (searchstart,searchrange):
          if window_colmn0-i >0 and window_colmn0-(i+1) >0 and window_colmn0-(i+2) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0 and room[window_row0][window_colmn0-(i+2)] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            coordinates.append([window_row0,window_colmn0-(i+1)])
            coordinates.append([window_row0,window_colmn0-(i+2)])
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range (searchstart,searchrange):
          if window_rown +i <rows and window_rown +(i+1) <rows and window_rown +(i+2) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 and room[window_rown +(i+2)][window_colmn0]==0 :
            coordinates.append([window_rown +i,window_colmn0])
            coordinates.append([window_rown +(i+1),window_colmn0])
            coordinates.append([window_rown +(i+2),window_colmn0])
            break
      elif window_colmn0== colmns and window_wall == 3:
        for i in range (searchstart,searchrange):
          if window_row0-i >0 and window_row0-(i+1) >0 and window_row0-(i+2) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 and room[window_row0-(i+2)][window_colmn0]==0 :
            coordinates.append([window_row0-i,window_colmn0])
            coordinates.append([window_row0-(i+1),window_colmn0])
            coordinates.append([window_row0-(i+2),window_colmn0])
            break   
##################################################
  if door_wall==4:
    if "chair" in item or "dc" in item :
      if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
        for i in range(searchstart,searchrange-1):
          if window_colmnn +i <= colmns and room[window_row0][window_colmnn +i] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            break
      elif window_colmn0== colmns and window_wall == 3:
        for i in range(searchstart,searchrange-1):
          if window_rown +i <=rows and room[window_rown +i][window_colmn0] == 0 :
            coordinates.append([window_rown +i,window_colmn0])
            break
      elif window_colmn0 == 0 and window_wall == 1:
        for i in range(searchstart,searchrange-1):
          if window_row0-i >=0 and room[window_row0-i][window_colmn0] == 0 :
            coordinates.append([window_row0-i,window_colmn0])
            break
  
      if "closet" in item or "table" in item :
        if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
          for i in range (searchstart,searchrange):
            if window_colmnn +i <colmns and window_colmnn +(i+1) <= colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 :
              coordinates.append([window_row0,window_colmnn +(i+1)])
              coordinates.append([window_row0,window_colmnn +i])
              break
        elif window_colmn0== colmns and window_wall == 3:
          for i in range (searchstart,searchrange):
            if window_rown +i <rows and window_rown +(i+1) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 :
              coordinates.append([window_rown +(i+1),window_colmn0])
              coordinates.append([window_rown +i,window_colmn0])
              break
        elif window_colmn0 == 0 and window_wall == 1:
          for i in range (searchstart,searchrange):
            if window_row0-i >0 and window_row0-(i+1) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 :
              coordinates.append([window_row0-(i+1),window_colmn0])
              coordinates.append([window_row0-i,window_colmn0])
              break
    if "desk" in item :
      if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
        for i in range (searchstart,searchrange):
          if window_colmnn +i<colmns and window_colmnn +(i+1)<colmns and window_colmnn +(i+2) <=colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 and room[window_row0][window_colmnn +(i+2)] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            coordinates.append([window_row0,window_colmnn +(i+1)])
            coordinates.append([window_row0,window_colmnn +(i+2)])
            break
      elif window_colmn0== colmns and window_wall == 3:
        for i in range (searchstart,searchrange):
          if window_rown +i <rows and window_rown +(i+1) <rows and window_rown +(i+2) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 and room[window_rown +(i+2)][window_colmn0]==0 :
            coordinates.append([window_rown +i,window_colmn0])
            coordinates.append([window_rown +(i+1),window_colmn0])
            coordinates.append([window_rown +(i+2),window_colmn0])
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range (searchstart,searchrange):
          if window_row0-i >0 and window_row0-(i+1) >0 and window_row0-(i+2) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 and room[window_row0-(i+2)][window_colmn0]==0 :
            coordinates.append([window_row0-i,window_colmn0])
            coordinates.append([window_row0-(i+1),window_colmn0])
            coordinates.append([window_row0-(i+2),window_colmn0])
            break   
##################################################
  if door_wall==1:
    if "chair" in item or "dc" in item :
      if window_row0 == rows and window_wall == 4 :
        for i in range(searchstart,searchrange-1):
          if window_colmn0-i >=0 and room[window_row0][window_colmn0-i] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            break
      elif window_row0== 0 and window_wall == 2:
        for i in range(searchstart,searchrange-1):
          if window_colmnn +i <= colmns and room[window_row0][window_colmnn +i] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            break
      elif (window_colmn0== 0 and window_wall == 1) or  (window_colmn0== colmns and window_wall == 3):
        for i in range(searchstart,searchrange-1):
          if window_rown +i <=rows and room[window_rown +i][window_colmn0] == 0 :
            coordinates.append([window_rown +i,window_colmn0])
            break
  
    if "closet" in item or "table" in item :
        if window_row0 == rows and window_wall == 4 :
          for i in range (searchstart,searchrange):
            if window_colmn0-i >0 and window_colmn0-(i+1) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0:
              coordinates.append([window_row0,window_colmn0-(i+1)])
              coordinates.append([window_row0,window_colmn0-i])
              break  
        elif window_row0 == 0 and window_wall == 2:
          for i in range (searchstart,searchrange):
            if window_colmnn +i <colmns and window_colmnn +(i+1) <= colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 :
              coordinates.append([window_row0,window_colmnn +(i+1)])
              coordinates.append([window_row0,window_colmnn +i])
              break
        elif (window_colmn0== 0 and window_wall == 1) or (window_colmn0== colmns and window_wall == 3):
          for i in range (searchstart,searchrange):
            if window_rown +i <rows and window_rown +(i+1) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 :
              coordinates.append([window_rown +(i+1),window_colmn0])
              coordinates.append([window_rown +i,window_colmn0])
              break
    if "desk" in item :
      if window_row0== rows and window_wall == 4 :
        for i in range (searchstart,searchrange):
          if window_colmn0-i >0 and window_colmn0-(i+1) >0 and window_colmn0-(i+2) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0 and room[window_row0][window_colmn0-(i+2)] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            coordinates.append([window_row0,window_colmn0-(i+1)])
            coordinates.append([window_row0,window_colmn0-(i+2)])
            break
      elif window_row0== 0 and window_wall == 2:
        for i in range (searchstart,searchrange):
          if window_colmnn +i<colmns and window_colmnn +(i+1)<colmns and window_colmnn +(i+2) <=colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 and room[window_row0][window_colmnn +(i+2)] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            coordinates.append([window_row0,window_colmnn +(i+1)])
            coordinates.append([window_row0,window_colmnn +(i+2)])
            break
      elif (window_colmn0== 0 and window_wall == 1) or  (window_colmn0== colmns and window_wall == 3):
        for i in range (searchstart,searchrange):
          if window_rown +i <rows and window_rown +(i+1) <rows and window_rown +(i+2) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 and room[window_rown +(i+2)][window_colmn0]==0 :
            coordinates.append([window_rown +i,window_colmn0])
            coordinates.append([window_rown +(i+1),window_colmn0])
            coordinates.append([window_rown +(i+2),window_colmn0])
            break

##################################################
  if door_wall==3:
    if "chair" in item or "dc" in item :
      if window_row0 == rows and window_wall == 4 :
        for i in range(searchstart,searchrange-1):
          if window_colmn0-i >=0 and room[window_row0][window_colmn0-i] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            break
      elif window_row0== 0 and window_wall == 2:
        for i in range(searchstart,searchrange-1):
          if window_colmnn +i <= colmns and room[window_row0][window_colmnn +i] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            break
      elif (window_colmn0 == colmns and window_wall == 3)or (window_colmn0== 0 and window_wall == 1):
        for i in range(searchstart,searchrange-1):
          if window_row0-i >=0 and room[window_row0-i][window_colmn0] == 0 :
            coordinates.append([window_row0-i,window_colmn0])
            break
  
    if "closet" in item or "table" in item :
        if window_row0 == rows and window_wall == 4 :
          for i in range (searchstart,searchrange):
            if window_colmn0-i >0 and window_colmn0-(i+1) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0:
              coordinates.append([window_row0,window_colmn0-(i+1)])
              coordinates.append([window_row0,window_colmn0-i])
              break  
        elif window_row0 == 0 and window_wall == 2:
          for i in range (searchstart,searchrange):
            if window_colmnn +i <colmns and window_colmnn +(i+1) <= colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 :
              coordinates.append([window_row0,window_colmnn +(i+1)])
              coordinates.append([window_row0,window_colmnn +i])
              break
        elif (window_colmn0 == colmns and window_wall == 3)or (window_colmn0== 0 and window_wall == 1):
          for i in range (searchstart,searchrange):
            if window_row0-i >0 and window_row0-(i+1) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 :
              coordinates.append([window_row0-(i+1),window_colmn0])
              coordinates.append([window_row0-i,window_colmn0])
              break
    if "desk" in item :
      if window_row0== rows and window_wall == 4 :
        for i in range (searchstart,searchrange):
          if window_colmn0-i >0 and window_colmn0-(i+1) >0 and window_colmn0-(i+2) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0 and room[window_row0][window_colmn0-(i+2)] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            coordinates.append([window_row0,window_colmn0-(i+1)])
            coordinates.append([window_row0,window_colmn0-(i+2)])
            break
      elif window_row0== 0 and window_wall == 2:
        for i in range (searchstart,searchrange):
          if window_colmnn +i<colmns and window_colmnn +(i+1)<colmns and window_colmnn +(i+2) <=colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 and room[window_row0][window_colmnn +(i+2)] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            coordinates.append([window_row0,window_colmnn +(i+1)])
            coordinates.append([window_row0,window_colmnn +(i+2)])
            break
      elif (window_colmn0 == colmns and window_wall == 3)or (window_colmn0== 0 and window_wall == 1):
        for i in range (searchstart,searchrange):
          if window_row0-i >0 and window_row0-(i+1) >0 and window_row0-(i+2) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 and room[window_row0-(i+2)][window_colmn0]==0 :
            coordinates.append([window_row0-i,window_colmn0])
            coordinates.append([window_row0-(i+1),window_colmn0])
            coordinates.append([window_row0-(i+2),window_colmn0])
            break   


  room=add_objects_to_room(room,coordinates)
  return coordinates , room 

In [ ]:
def location_window_direction_left(item ,room,window_location,window_wall,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[]
  
  window_rows , window_colmns , _ = is_ver_horz(window_location,window_wall)
  window_row0 = window_rows[0]
  window_colmn0 = window_colmns[0]
  window_rown  = window_rows[-1]
  window_colmnn  = window_colmns[-1]
  if door_wall==4:
    if "chair" in item or "dc" in item :
      if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
        for i in range(searchstart,searchrange):
          if window_colmn0-i >=0 and room[window_row0][window_colmn0-i] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range(searchstart,searchrange):
          if window_rown +i <=rows and room[window_rown +i][window_colmn0] == 0 :
            coordinates.append([window_rown +i,window_colmn0])
            break
      elif window_colmn0 == colmns and window_wall == 3:
        for i in range(searchstart,searchrange):
          if window_row0-i >=0 and room[window_row0-i][window_colmn0] == 0 :
            coordinates.append([window_row0-i,window_colmn0])
            break
    if "closet" in item or "table" in item :
        if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
          for i in range (searchstart,searchrange):
            if window_colmn0-i >0 and window_colmn0-(i+1) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0:
              coordinates.append([window_row0,window_colmn0-(i+1)])
              coordinates.append([window_row0,window_colmn0-i])
              break  
        elif window_colmn0== 0 and window_wall == 1:
          for i in range (searchstart,searchrange):
            if window_rown +i <rows and window_rown +(i+1) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 :
              coordinates.append([window_rown +(i+1),window_colmn0])
              coordinates.append([window_rown +i,window_colmn0])
              break
        elif window_colmn0 == colmns and window_wall == 3:
          for i in range (searchstart,searchrange):
            if window_row0-i >0 and window_row0-(i+1) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 :
              coordinates.append([window_row0-(i+1),window_colmn0])
              coordinates.append([window_row0-i,window_colmn0])
              break
    if "desk" in item :
      if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
        for i in range (searchstart,searchrange):
          if window_colmn0-i >0 and window_colmn0-(i+1) >0 and window_colmn0-(i+2) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0 and room[window_row0][window_colmn0-(i+2)] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            coordinates.append([window_row0,window_colmn0-(i+1)])
            coordinates.append([window_row0,window_colmn0-(i+2)])
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range (searchstart,searchrange):
          if window_rown +i <rows and window_rown +(i+1) <rows and window_rown +(i+2) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 and room[window_rown +(i+2)][window_colmn0]==0 :
            coordinates.append([window_rown +i,window_colmn0])
            coordinates.append([window_rown +(i+1),window_colmn0])
            coordinates.append([window_rown +(i+2),window_colmn0])
            break
      elif window_colmn0== colmns and window_wall == 3:
        for i in range (searchstart,searchrange):
          if window_row0-i >0 and window_row0-(i+1) >0 and window_row0-(i+2) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 and room[window_row0-(i+2)][window_colmn0]==0 :
            coordinates.append([window_row0-i,window_colmn0])
            coordinates.append([window_row0-(i+1),window_colmn0])
            coordinates.append([window_row0-(i+2),window_colmn0])
            break   
##################################################
  if door_wall==2:
    if "chair" in item or "dc" in item :
      if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
        for i in range(searchstart,searchrange):
          if window_colmnn +i <= colmns and room[window_row0][window_colmnn +i] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            break
      elif window_colmn0== colmns and window_wall == 3:
        for i in range(searchstart,searchrange):
          if window_rown +i <=rows and room[window_rown +i][window_colmn0] == 0 :
            coordinates.append([window_rown +i,window_colmn0])
            break
      elif window_colmn0 == 0 and window_wall == 1:
        for i in range(searchstart,searchrange):
          if window_row0-i >=0 and room[window_row0-i][window_colmn0] == 0 :
            coordinates.append([window_row0-i,window_colmn0])
            break
  
      if "closet" in item or "table" in item :
        if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
          for i in range (searchstart,searchrange):
            if window_colmnn +i <colmns and window_colmnn +(i+1) <= colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 :
              coordinates.append([window_row0,window_colmnn +(i+1)])
              coordinates.append([window_row0,window_colmnn +i])
              break
        elif window_colmn0== colmns and window_wall == 3:
          for i in range (searchstart,searchrange):
            if window_rown +i <rows and window_rown +(i+1) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 :
              coordinates.append([window_rown +(i+1),window_colmn0])
              coordinates.append([window_rown +i,window_colmn0])
              break
        elif window_colmn0 == 0 and window_wall == 1:
          for i in range (searchstart,searchrange):
            if window_row0-i >0 and window_row0-(i+1) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 :
              coordinates.append([window_row0-(i+1),window_colmn0])
              coordinates.append([window_row0-i,window_colmn0])
              break
    if "desk" in item :
      if (window_row0 == 0 and window_wall == 2) or (window_row0 == rows and window_wall == 4):
        for i in range (searchstart,searchrange):
          if window_colmnn +i<colmns and window_colmnn +(i+1)<colmns and window_colmnn +(i+2) <=colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 and room[window_row0][window_colmnn +(i+2)] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            coordinates.append([window_row0,window_colmnn +(i+1)])
            coordinates.append([window_row0,window_colmnn +(i+2)])
            break
      elif window_colmn0== colmns and window_wall == 3:
        for i in range (searchstart,searchrange):
          if window_rown +i <rows and window_rown +(i+1) <rows and window_rown +(i+2) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 and room[window_rown +(i+2)][window_colmn0]==0 :
            coordinates.append([window_rown +i,window_colmn0])
            coordinates.append([window_rown +(i+1),window_colmn0])
            coordinates.append([window_rown +(i+2),window_colmn0])
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range (searchstart,searchrange):
          if window_row0-i >0 and window_row0-(i+1) >0 and window_row0-(i+2) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 and room[window_row0-(i+2)][window_colmn0]==0 :
            coordinates.append([window_row0-i,window_colmn0])
            coordinates.append([window_row0-(i+1),window_colmn0])
            coordinates.append([window_row0-(i+2),window_colmn0])
            break   
##################################################
  if door_wall==3:
    if "chair" in item or "dc" in item :
      if window_row0== 0 and window_wall == 2 :
        for i in range(searchstart,searchrange):
          if window_colmn0-i >=0 and room[window_row0][window_colmn0-i] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            break
      elif window_row0 == rows and window_wall == 4 :
        for i in range(searchstart,searchrange):
          if window_colmnn +i <= colmns and room[window_row0][window_colmnn +i] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            break
      elif (window_colmn0== 0 and window_wall == 1) or  (window_colmn0== colmns and window_wall == 3):
        for i in range(searchstart,searchrange):
          if window_rown +i <=rows and room[window_rown +i][window_colmn0] == 0 :
            coordinates.append([window_rown +i,window_colmn0])
            break
  
    if "closet" in item or "table" in item :
        if window_row0 == 0 and window_wall == 2 :
          for i in range (searchstart,searchrange):
            if window_colmn0-i >0 and window_colmn0-(i+1) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0:
              coordinates.append([window_row0,window_colmn0-(i+1)])
              coordinates.append([window_row0,window_colmn0-i])
              break  
        elif window_row0 == rows and window_wall == 4:
          for i in range (searchstart,searchrange):
            if window_colmnn +i <colmns and window_colmnn +(i+1) <= colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 :
              coordinates.append([window_row0,window_colmnn +(i+1)])
              coordinates.append([window_row0,window_colmnn +i])
              break
        elif (window_colmn0== 0 and window_wall == 1) or (window_colmn0== colmns and window_wall == 3):
          for i in range (searchstart,searchrange):
            if window_rown +i <rows and window_rown +(i+1) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 :
              coordinates.append([window_rown +(i+1),window_colmn0])
              coordinates.append([window_rown +i,window_colmn0])
              break
    if "desk" in item :
      if window_row0== 0 and window_wall == 2 :
        for i in range (searchstart,searchrange):
          if window_colmn0-i >0 and window_colmn0-(i+1) >0 and window_colmn0-(i+2) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0 and room[window_row0][window_colmn0-(i+2)] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            coordinates.append([window_row0,window_colmn0-(i+1)])
            coordinates.append([window_row0,window_colmn0-(i+2)])
            break
      elif window_row0== rows and window_wall == 4:
        for i in range (searchstart,searchrange):
          if window_colmnn +i<colmns and window_colmnn +(i+1)<colmns and window_colmnn +(i+2) <=colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 and room[window_row0][window_colmnn +(i+2)] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            coordinates.append([window_row0,window_colmnn +(i+1)])
            coordinates.append([window_row0,window_colmnn +(i+2)])
            break
      elif (window_colmn0== 0 and window_wall == 1) or  (window_colmn0== colmns and window_wall == 3):
        for i in range (searchstart,searchrange):
          if window_rown +i <rows and window_rown +(i+1) <rows and window_rown +(i+2) <=rows and room[window_rown +i][window_colmn0]==0 and room[window_rown +(i+1)][window_colmn0]==0 and room[window_rown +(i+2)][window_colmn0]==0 :
            coordinates.append([window_rown +i,window_colmn0])
            coordinates.append([window_rown +(i+1),window_colmn0])
            coordinates.append([window_rown +(i+2),window_colmn0])
            break

##################################################
  if door_wall==1:
    if "chair" in item or "dc" in item :
      if window_row0 == 0 and window_wall == 2 :
        for i in range(searchstart,searchrange):
          if window_colmn0-i >=0 and room[window_row0][window_colmn0-i] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            break
      elif window_row0== rows and window_wall == 2:
        for i in range(searchstart,searchrange):
          if window_colmnn +i <= colmns and room[window_row0][window_colmnn +i] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            break
      elif (window_colmn0 == colmns and window_wall == 3)or (window_colmn0== 0 and window_wall == 1):
        for i in range(searchstart,searchrange):
          if window_row0-i >=0 and room[window_row0-i][window_colmn0] == 0 :
            coordinates.append([window_row0-i,window_colmn0])
            break
  
    if "closet" in item or "table" in item :
        if window_row0 == 0 and window_wall == 2 :
          for i in range (searchstart,searchrange):
            if window_colmn0-i >0 and window_colmn0-(i+1) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0:
              coordinates.append([window_row0,window_colmn0-(i+1)])
              coordinates.append([window_row0,window_colmn0-i])
              break  
        elif window_row0 == rows and window_wall == 4:
          for i in range (searchstart,searchrange):
            if window_colmnn +i <colmns and window_colmnn +(i+1) <= colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 :
              coordinates.append([window_row0,window_colmnn +(i+1)])
              coordinates.append([window_row0,window_colmnn +i])
              break
        elif (window_colmn0 == colmns and window_wall == 3)or (window_colmn0== 0 and window_wall == 1):
          for i in range (searchstart,searchrange):
            if window_row0-i >0 and window_row0-(i+1) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 :
              coordinates.append([window_row0-(i+1),window_colmn0])
              coordinates.append([window_row0-i,window_colmn0])
              break
    if "desk" in item :
      if window_row0== 0 and window_wall == 2 :
        for i in range (searchstart,searchrange):
          if window_colmn0-i >0 and window_colmn0-(i+1) >0 and window_colmn0-(i+2) >=0 and room[window_row0][window_colmn0-i] == 0 and room[window_row0][window_colmn0-(i+1)] == 0 and room[window_row0][window_colmn0-(i+2)] == 0 :
            coordinates.append([window_row0,window_colmn0-i])
            coordinates.append([window_row0,window_colmn0-(i+1)])
            coordinates.append([window_row0,window_colmn0-(i+2)])
            break
      elif window_row0== rows and window_wall == 2:
        for i in range (searchstart,searchrange):
          if window_colmnn +i<colmns and window_colmnn +(i+1)<colmns and window_colmnn +(i+2) <=colmns and room[window_row0][window_colmnn +i] == 0 and room[window_row0][window_colmnn +(i+1)] == 0 and room[window_row0][window_colmnn +(i+2)] == 0 :
            coordinates.append([window_row0,window_colmnn +i])
            coordinates.append([window_row0,window_colmnn +(i+1)])
            coordinates.append([window_row0,window_colmnn +(i+2)])
            break
      elif (window_colmn0 == colmns and window_wall == 3)or (window_colmn0== 0 and window_wall == 1):
        for i in range (searchstart,searchrange):
          if window_row0-i >0 and window_row0-(i+1) >0 and window_row0-(i+2) >=0 and room[window_row0-i][window_colmn0]==0 and room[window_row0-(i+1)][window_colmn0]==0 and room[window_row0-(i+2)][window_colmn0]==0 :
            coordinates.append([window_row0-i,window_colmn0])
            coordinates.append([window_row0-(i+1),window_colmn0])
            coordinates.append([window_row0-(i+2),window_colmn0])
            break   


  room=add_objects_to_room(room,coordinates)
  return coordinates , room 

In [ ]:
def location_window_direction_infront_oppsite_under_below(item ,room,window_location,window_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[]   

  window_rows , window_colmns , _ = is_ver_horz(window_location,window_wall)
  window_row0 = window_rows[0]
  window_colmn0 = window_colmns[0]
  window_rown = window_rows[-1]
  window_colmnn = window_colmns[-1]

  if "chair" in item :
      if window_row0== 0 and window_wall == 2 :
        for i in range (window_row0+searchstart,window_row0+searchrange):
          for col in window_colmns :
            if room[i][col] == 0 :
              coordinates.append([i,col])
              break
          if len(coordinates)!=0:
            break
      elif window_row0== rows and window_wall == 4:
        for i in range(window_row0-searchstart, window_row0-searchrange ,-1):
          for col in window_colmns :
            if room[i][col] == 0 :
              coordinates.append([i,col])
              break
          if len(coordinates)!=0:
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range ( window_colmn0+searchstart,window_colmn0+searchrange):
          for row in window_rows :
            if room[row][i] == 0 :
              coordinates.append([row,i])
              break
          if len(coordinates)!=0:
            break
      elif window_colmn0== colmns and window_wall == 3 :
        for i in range (window_colmn0-searchstart , window_colmn0-searchrange, -1):
          for row in window_rows :
            if room[row][i] == 0 :
              coordinates.append([row,i])
              break
          if len(coordinates)!=0:
            break

  if "closet" in item or "table" in item :
      if window_row0== 0 and window_wall == 2 :
        for i in range (window_row0+searchstart,window_row0+searchrange):
          for col in range(len(window_colmns)) :
            if room[i][window_colmns[col]] == 0 and room[i][window_colmns[col]+1] and window_colmns[col]+1 in window_colmns :
              coordinates.append([i,window_colmns[col]])
              coordinates.append([i,window_colmns[col]+1])
              break
          if len(coordinates)!=0:
            break
      elif window_row0== rows and window_wall == 4:
        for i in range(window_row0-searchstart, window_row0-searchrange ,-1):
          for col in range(len(window_colmns)) :
            if room[i][window_colmns[col]] == 0 and room[i][window_colmns[col]+1] and window_colmns[col]+1 in window_colmns :
              coordinates.append([i,window_colmns[col]])
              coordinates.append([i,window_colmns[col]+1])
              break
          if len(coordinates)!=0:
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range ( window_colmn0+searchstart,window_colmn0+searchrange):
          for row in range(len(window_rows)) :
            if room[window_rows[row]][i] == 0 and room[window_rows[row]+1][i] == 0 and window_rows[row]+1 in window_rows :
              coordinates.append([window_rows[row],i])
              coordinates.append([window_rows[row]+1,i])
              break
          if len(coordinates)!=0:
            break
      elif window_colmn0== colmns and window_wall == 3 :
        for i in range (window_colmn0-searchstart , window_colmn0-searchrange, -1):
          for row in range(len(window_rows)) :
            if room[window_rows[row]][i] == 0 and room[window_rows[row]+1][i] == 0 and window_rows[row]+1 in window_rows :
              coordinates.append([window_rows[row],i])
              coordinates.append([window_rows[row]+1,i])
              break
          if len(coordinates)!=0:
            break

  if "desk" in item  :
    if len(window_location)>=3:
      if window_row0== 0 and window_wall == 2 :
        for i in range (window_row0+searchstart,window_row0+searchrange):
          for col in range(len(window_colmns)) :
            if room[i][window_colmns[col]] == 0 and room[i][window_colmns[col]+1]==0 and room[i][window_colmns[col]+2]==0 and window_colmns[col]+1 in window_colmns and window_colmns[col]+2 in window_colmns :
              coordinates.append([i,window_colmns[col]])
              coordinates.append([i,window_colmns[col]+1])
              coordinates.append([i,window_colmns[col]+2])
              break
          if len(coordinates)!=0:
            break
      elif window_row0== rows and window_wall == 4:
        for i in range(window_row0-searchstart, window_row0-searchrange ,-1):
          for col in range (len(window_colmns)) :
            if room[i][window_colmns[col]] == 0 and room[i][window_colmns[col]+1]==0  and room[i][window_colmns[col]+2]==0 and window_colmns[col]+1 in window_colmns and window_colmns[col]+2 in window_colmns :
              coordinates.append([i,window_colmns[col]])
              coordinates.append([i,window_colmns[col]+1])
              coordinates.append([i,window_colmns[col]+2])
          if len(coordinates)!=0:
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range ( window_colmn0+searchstart,window_colmn0+searchrange):
          for row in range(len(window_rows) ):
            if room[window_rows[row]][i] == 0 and room[window_rows[row]+1][i] == 0 and room[window_rows[row]+2][i] == 0 and window_rows[row]+1 in window_rows and window_rows[row]+2 in window_rows :
              coordinates.append([window_rows[row],i])
              coordinates.append([window_rows[row]+1,i])
              coordinates.append([window_rows[row]+2,i])
              break
          if len(coordinates)!=0:
            break
      elif window_colmn0== colmns and window_wall == 3 :
        for i in range (window_colmn0-searchstart , window_colmn0-searchrange, -1):
          for row in range(len(window_rows)) :
            if room[window_rows[row]][i] == 0 and room[window_rows[row]+1][i] == 0 and room[window_rows[row]+2][i] == 0 and window_rows[row]+1 in window_rows and window_rows[row]+2 in window_rows :
              coordinates.append([window_rows[row],i])
              coordinates.append([window_rows[row]+1,i])
              coordinates.append([window_rows[row]+2,i])
              break
          if len(coordinates)!=0:
            break
    elif len(window_location)==2:
      if window_row0== 0 and window_wall == 2 :
        for i in range (window_row0+searchstart,window_row0+searchrange):
          for col in range(len(window_colmns)) :
            if room[i][window_colmns[col]] == 0 and room[i][window_colmns[col]+1]==0  and window_colmns[col]+1 in window_colmns and window_colmns[col]+2<=colmns and room[i][window_colmns[col]+2]==0:
              coordinates.append([i,window_colmns[col]])
              coordinates.append([i,window_colmns[col]+1])
              coordinates.append([i,window_colmns[col]+2])
              break
            elif room[i][window_colmns[col]] == 0 and room[i][window_colmns[col]+1] ==0 and window_colmns[col]+1 in window_colmns and window_colmns[col]-1>=0 and room[i][window_colmns[col]-1]==0:
              coordinates.append([i,window_colmns[col]-1])
              coordinates.append([i,window_colmns[col]])
              coordinates.append([i,window_colmns[col]+1])
              break
          if len(coordinates)!=0:
            break
      elif window_row0== rows and window_wall == 4:
        for i in range(window_row0-searchstart, window_row0-searchrange ,-1):
          for col in range (len(window_colmns)) :
            if room[i][window_colmns[col]] == 0 and room[i][window_colmns[col]+1]==0  and window_colmns[col]+1 in window_colmns and window_colmns[col]+2<=colmns and room[i][window_colmns[col]+2]==0:
              coordinates.append([i,window_colmns[col]])
              coordinates.append([i,window_colmns[col]+1])
              coordinates.append([i,window_colmns[col]+2])
              break
            elif room[i][window_colmns[col]] == 0 and room[i][window_colmns[col]+1]==0  and window_colmns[col]+1 in window_colmns and window_colmns[col]-1>=0 and room[i][door_colmns[col]-1]==0:
              coordinates.append([i,window_colmns[col]-1])
              coordinates.append([i,window_colmns[col]])
              coordinates.append([i,window_colmns[col]+1])
              break
          if len(coordinates)!=0:
            break
      elif window_colmn0== 0 and window_wall == 1:
        for i in range ( window_colmn0+searchstart,window_colmn0+searchrange):
          for row in range(len(window_rows) ):
            if room[window_rows[row]][i] == 0 and room[window_rows[row]+1][i] == 0 and window_rows[row]+1 in window_rows and window_rows[row]+2<=rows and room[window_rows[row]+2][i] == 0:
              coordinates.append([window_rows[row],i])
              coordinates.append([window_rows[row]+1,i])
              coordinates.append([window_rows[row]+2,i])
              break
            elif room[window_rows[row]][i] == 0 and room[window_rows[row]+1][i] == 0 and window_rows[row]+1 in window_rows and window_rows[row]-1>=0 and room[window_rows[row]-1][i] == 0:
              coordinates.append([window_rows[row]-1,i])
              coordinates.append([window_rows[row],i])
              coordinates.append([window_rows[row]+1,i])
              break
          if len(coordinates)!=0:
            break
      elif window_colmn0== colmns and window_wall == 3 :
        for i in range (window_colmn0-searchstart , window_colmn0-searchrange, -1):
          for row in range(len(window_rows)) :
            if room[window_rows[row]][i] == 0 and room[window_rows[row]+1][i] == 0 and window_rows[row]+1 in window_rows and window_rows[row]+2<=rows and room[window_rows[row]+2][i] == 0:
              coordinates.append([window_rows[row],i])
              coordinates.append([window_rows[row]+1,i])
              coordinates.append([window_rows[row]+2,i])
              break
            elif room[window_rows[row]][i] == 0 and room[window_rows[row]+1][i] == 0 and window_rows[row]+1 in window_rows and window_rows[row]-1>=0 and room[window_rows[row]-1][i] == 0:
              coordinates.append([window_rows[row]-1,i])
              coordinates.append([window_rows[row],i])
              coordinates.append([window_rows[row]+1,i])
              break
          if len(coordinates)!=0:
            break

  room=add_objects_to_room(room,coordinates)
  return coordinates , room

In [ ]:
def location_window_direction_beside_next(item,room,window_location,window_wall,door_wall,searchstart,searchrange):
  coordinates , room =location_window_direction_right(item=item,room=room,window_location=window_location,window_wall=window_wall,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates) ==0:
    coordinates , room = location_door_window_direction_left(item=item,room=room,window_location=window_location,window_wall=window_wall,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  return coordinates , room

In [ ]:
def location_window_direction_near(item ,room,window_location,window_wall,door_wall,searchstart,searchrange):
  coordinates , room =location_window_direction_right(item=item,room=room,window_location=window_location,window_wall=window_wall,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates) ==0:
    coordinates , room = location_window_direction_left(item=item,room=room,window_location=window_location,window_wall=window_wall,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates) == 0:
    coordinates , room = location_window_direction_infront_oppsite_under_below(item=item ,room=room,window_location=window_location,window_wall=window_wall,searchstart=searchstart,searchrange=searchrange)
  return coordinates , room 

In [ ]:
def location_chair_direction_right(item ,room,chair_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 

  if "chair" in item or "dc" in item :
    if door_wall == 3 :
      if 0< chair_location[0]<rows : 
        for i in range(searchstart,searchrange):
          if chair_location[0]-i >=0  and room[chair_location[0]-i][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            break
    if door_wall == 1 :
      if 0< chair_location[0]<rows :
        for i in range(searchstart,searchrange):
          if chair_location[0]+i <=rows  and room[chair_location[0]+i][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]])
            break
    if door_wall==1 or door_wall==3 :
      if chair_location[0] == rows :
        for i in range(searchstart,searchrange):
          if chair_location[1]-i >=0  and room[chair_location[0]][chair_location[1]-i]==0:
            coordinates.append([chair_location[0],chair_location[1]-i]) 
            break
      elif chair_location[0] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[1]+i <=colmns  and room[chair_location[0]][chair_location[1]+i]==0: 
            coordinates.append([chair_location[0],chair_location[1]+i]) 
            break

    if door_wall == 4 :
      if 0< chair_location[1] < colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]+i <=colmns  and room[chair_location[0]][chair_location[1]+i]==0: 
            coordinates.append([chair_location[0],chair_location[1]+i])
            break
    if door_wall == 2 :
      if 0< chair_location[1] < colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]-i>=0  and room[chair_location[0]][chair_location[1]-i]==0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            break

    if door_wall==2 or door_wall==4 :
      if chair_location[1] == colmns :
        for i in range(searchstart,searchrange):
          if chair_location[0]+i <= rows  and room[chair_location[0]+i][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]])
            break
      elif chair_location[1] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[0]-i >=0  and room[chair_location[0]-i][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]]) 
            break 

  if "table" in item or "closet" in item :
    if door_wall == 3 :
      if 0< chair_location[0] <rows :
        for i in range(searchstart,searchrange):
          if chair_location[0]-(i+1) >=0  and room[chair_location[0]-i][chair_location[1]]==0 and room[chair_location[0]-(i+1)][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            coordinates.append([chair_location[0]-(i+1),chair_location[1]])
            break
        if len(coordinates)==0:
          if chair_location[0]-1>=0 and chair_location[1]+1<=colmns and room[chair_location[0]-1][chair_location[1]] ==0and room[chair_location[0]-1][chair_location[1]+1]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
          elif chair_location[0]-1>=0 and chair_location[1]-1>=0 and room[chair_location[0]-1][chair_location[1]] ==0and room[chair_location[0]-1][chair_location[1]-1]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]])
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
    
    if door_wall == 1  :
      if 0< chair_location[0] <rows :
        for i in range(searchstart,searchrange):
          if chair_location[0]+(i+1) <=rows  and room[chair_location[0]+i][chair_location[1]]==0 and room[chair_location[0]+(i+1)][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]])
            coordinates.append([chair_location[0]+(i+1),chair_location[1]])
            break
        if len(coordinates)==0:
          if chair_location[0]+1<=rows and chair_location[1]+1<=colmns and room[chair_location[0]+1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]+1]==0 :
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
          elif chair_location[0]+1<=rows and chair_location[1]-1>=0 and room[chair_location[0]+1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]-1]==0 :
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
    
    if door_wall == 3 or door_wall==1 :
      if chair_location[0] == rows :
        for i in range(searchstart,searchrange):
          if chair_location[1]-(i+1) >=0  and room[chair_location[0]][chair_location[1]-i] == 0 and room[chair_location[0]][chair_location[1]-(i+1)] == 0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            coordinates.append([chair_location[0],chair_location[1]-(i+1)])
            break
      elif chair_location[0] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[1]+(i+1) <=colmns  and room[chair_location[0]][chair_location[1]+i] == 0 and room[chair_location[0]][chair_location[1]+(i+1) ]== 0:
            coordinates.append([chair_location[0],chair_location[1]+i])
            coordinates.append([chair_location[0],chair_location[1]+(i+1)])
            break

    if door_wall == 4  :
      if 0< chair_location[1] <colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]+(i+1) <=colmns and room[chair_location[0]][chair_location[1]+i]==0 and room[chair_location[0]][chair_location[1]+(i+1)]==0:
            coordinates.append([chair_location[0],chair_location[1]+i])
            coordinates.append([chair_location[0],chair_location[1]+(i+1)])
            break
        if len(coordinates)==0:
          if chair_location[1]+1 <=colmns and chair_location[0]+1<=rows and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]+1][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
          elif chair_location[1]+1<=colmns and chair_location[0]-1>=0 and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]-1][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
            
    if door_wall == 2  :
      if 0< chair_location[1] <colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]-(i+1) >=0  and room[chair_location[0]][chair_location[1]-i]==0 and room[chair_location[0]][chair_location[1]-(i+1)]==0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            coordinates.append([chair_location[0],chair_location[1]-(i+1)])
            break
        if len(coordinates)==0:
          if chair_location[1]-1>=0 and chair_location[0]+1<=rows and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]+1][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
          elif chair_location[1]-1>=0 and chair_location[0]-1>=0 and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]-1][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
    
    if door_wall == 4 or door_wall==2 :
      if chair_location[1] == colmns :
        for i in range(searchstart,searchrange):
          if chair_location[0]+(i+1) <=rows  and room[chair_location[0]+i][chair_location[1]]==0 and room[chair_location[0]+(i+1)][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]]) 
            coordinates.append([chair_location[0]+(i+1),chair_location[1]])
            break 
      elif chair_location[1] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[0]-(i+1) >=0  and room[chair_location[0]-i][chair_location[1]]==0 and room[chair_location[0]-(i+1)][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            coordinates.append([chair_location[0]-(i+1),chair_location[1]])
            break

  if "desk" in item :
    if door_wall == 3 :
      if 0< chair_location[0] < rows:
        for i in range(searchstart,searchrange):
          if chair_location[0]-(i+2) >=0  and room[chair_location[0]-i][chair_location[1]]==0 and room[chair_location[0]-(i+1)][chair_location[1]]==0 and room[chair_location[0]-(i+2)][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            coordinates.append([chair_location[0]-(i+1),chair_location[1]])
            coordinates.append([chair_location[0]-(i+2),chair_location[1]])
            break
        if len(coordinates)==0:
          if chair_location[0]-1>=0 and chair_location[1]+1<=colmns and chair_location[1]-1>=0 and room[chair_location[0]-1][chair_location[1]-1] ==0and room[chair_location[0]-1][chair_location[1]]==0 and room[chair_location[0]-1][chair_location[1]+1]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
            coordinates.append([chair_location[0]-1,chair_location[1]])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
          elif chair_location[0]-1>=0 and chair_location[1]+2<=colmns and room[chair_location[0]-1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]-1]==0 and room[chair_location[0]-1][chair_location[1]+2]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
            coordinates.append([chair_location[0]-1,chair_location[1]+2])
          elif chair_location[0]-1>=0 and chair_location[1]-2>=0 and room[chair_location[0]-1][chair_location[1]] ==0and room[chair_location[0]-1][chair_location[1]-1]==0 and room[chair_location[0]-1][chair_location[1]-2]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]-2])
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
            coordinates.append([chair_location[0]-1,chair_location[1]])

    if door_wall == 1 :
      if 0< chair_location[0] < rows:
        for i in range(searchstart,searchrange):
          if chair_location[0]+(i+2)<=rows  and room[chair_location[0]+i][chair_location[1]]==0 and room[chair_location[0]+(i+1)][chair_location[1]]==0 and room[chair_location[0]+(i+2)][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]])
            coordinates.append([chair_location[0]+(i+1),chair_location[1]])
            coordinates.append([chair_location[0]+(i+2),chair_location[1]])
            break
        if len(coordinates)==0:
          if chair_location[0]+1<=rows and chair_location[1]+1<=colmns and chair_location[1]-1>=0  and room[chair_location[0]+1][chair_location[1]-1] ==0and room[chair_location[0]+1][chair_location[1]]==0and room[chair_location[0]+1][chair_location[1]+1]==0 :
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
          elif chair_location[0]+1<=rows and chair_location[1]+2<=colmns and room[chair_location[0]+1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]+1]==0and room[chair_location[0]+1][chair_location[1]+2]==0 :
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
            coordinates.append([chair_location[0]+1,chair_location[1]+2])
          elif chair_location[0]+1<=rows and chair_location[1]-2>=0 and room[chair_location[0]+1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]-1]==0 and room[chair_location[0]+1][chair_location[1]-2]==0:
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]-2])
    
    if door_wall == 3 or door_wall==1 :
      if chair_location[0] == rows :
        for i in range(searchstart,searchrange):
          if chair_location[1]-(i+2) >=0  and room[chair_location[0]][chair_location[1]-i] == 0 and room[chair_location[0]][chair_location[1]-(i+1)] == 0 and room[chair_location[0]][chair_location[1]-(i+2)] == 0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            coordinates.append([chair_location[0],chair_location[1]-(i+1)])
            coordinates.append([chair_location[0],chair_location[1]-(i+2)])
            break
      elif chair_location[0] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[1]+(i+2) <= colmns  and room[chair_location[0]][chair_location[1]+i] == 0 and room[chair_location[0]][chair_location[1]+(i+1)] == 0 and room[chair_location[0]][chair_location[1]+(i+2)] == 0:
            coordinates.append([chair_location[0],chair_location[1]+i])
            coordinates.append([chair_location[0],chair_location[1]+(i+1)])
            coordinates.append([chair_location[0],chair_location[1]+(i+2)])
            break

    
    if door_wall == 4 :
      if 0<chair_location[1] < colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]+(i+2) <=rwos  and room[chair_location[0]][chair_location[1]+i]==0 and room[chair_location[0]][chair_location[1]+(i+1)]==0 and room[chair_location[0]][chair_location[1]+(i+2)]==0:
            coordinates.append([chair_location[0],chair_location[1]+i])
            coordinates.append([chair_location[0],chair_location[1]+(i+1)])
            coordinates.append([chair_location[0],chair_location[1]+(i+2)])
            break
        if len(coordinates)==0:
          if chair_location[1]+1<=colmns and chair_location[0]+1<=rows and chair_location[0]-1>=0 and room[chair_location[0]-1][chair_location[1]+1]==0 and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]+1][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
          elif chair_location[1]+1<=colmns  and chair_location[0]+2<=rows and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]+1][chair_location[1]+1]==0 and room[chair_location[0]+2][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
            coordinates.append([chair_location[0]+2,chair_location[1]+1])
          elif chair_location[1]+1<=colmns  and chair_location[0]-2>=0 and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]-1][chair_location[1]+1]==0 and room[chair_location[0]-2][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
            coordinates.append([chair_location[0]-2,chair_location[1]+1])

    if door_wall == 2:
      if 0<chair_location[1] < colmns  :
        for i in range(searchstart,searchrange):
          if chair_location[1]-(i+2) >=0  and room[chair_location[0]][chair_location[1]-i]==0 and room[chair_location[0]][chair_location[1]-(i+1)]==0 and room[chair_location[0]][chair_location[1]-(i+2)]==0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            coordinates.append([chair_location[0],chair_location[1]-(i+1)])
            coordinates.append([chair_location[0],chair_location[1]-(i+2)])
            break
        if len(coordinates)==0:
          if chair_location[1]-1>=0 and chair_location[0]+1<=rows and chair_location[0]-1>=0 and room[chair_location[0]-1][chair_location[1]-1]==0 and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]+1][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
          elif chair_location[1]-1>=0 and chair_location[0]+2<=rows and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]+1][chair_location[1]-1]==0 and room[chair_location[0]+2][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
            coordinates.append([chair_location[0]+2,chair_location[1]-1])
          elif chair_location[1]-1>=0 and chair_location[0]-2>=0 and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]-1][chair_location[1]-1]==0 and room[chair_location[0]-2][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
            coordinates.append([chair_location[0]-2,chair_location[1]-1])
    
    if door_wall == 4 or door_wall==2:
      if chair_location[1] == colmns :
        for i in range(searchstart,searchrange):
          if chair_location[0]+(i+2) <=rows  and room[chair_location[0]+i][chair_location[1]]==0 and room[chair_location[0]+(i+1)][chair_location[1]]==0  and room[chair_location[0]+(i+2)][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]]) 
            coordinates.append([chair_location[0]+(i+1),chair_location[1]]) 
            coordinates.append([chair_location[0]+(i+2),chair_location[1]]) 
            break
      elif chair_location[1] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[0]-(i+2) >=0  and room[chair_location[0]-i][chair_location[1]]==0 and room[chair_location[0]-(i+1)][chair_location[1]]==0 and room[chair_location[0]-(i+2)][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            coordinates.append([chair_location[0]-(i+1),chair_location[1]])
            coordinates.append([chair_location[0]-(i+2),chair_location[1]])
            break

  room = add_objects_to_room(room,coordinates)
  return coordinates, room 

In [ ]:
def location_chair_direction_left(item ,room,chair_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 

  if "chair" in item or "dc" in item :
    if door_wall == 1 :
      if 0< chair_location[0]<rows : 
        for i in range(searchstart,searchrange):
          if chair_location[0]-i >=0  and room[chair_location[0]-i][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            break
    if door_wall == 3 :
      if 0< chair_location[0]<rows :
        for i in range(searchstart,searchrange):
          if chair_location[0]+i <=rows  and room[chair_location[0]+i][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]])
            break
    if door_wall==1 or door_wall==3 :
      if chair_location[0] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[1]-i >=0  and room[chair_location[0]][chair_location[1]-i]==0:
            coordinates.append([chair_location[0],chair_location[1]-i]) 
            break
      elif chair_location[0] == rows :
        for i in range(searchstart,searchrange):
          if chair_location[1]+i <=colmns  and room[chair_location[0]][chair_location[1]+i]==0: 
            coordinates.append([chair_location[0],chair_location[1]+i]) 
            break

    if door_wall == 2 :
      if 0< chair_location[1] < colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]+i <=colmns  and room[chair_location[0]][chair_location[1]+i]==0: 
            coordinates.append([chair_location[0],chair_location[1]+i])
            break
    if door_wall == 4 :
      if 0< chair_location[1] < colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]-i>=0  and room[chair_location[0]][chair_location[1]-i]==0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            break

    if door_wall==2 or door_wall==4 :
      if chair_location[1] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[0]+i <= rows  and room[chair_location[0]+i][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]])
            break
      elif chair_location[1] == colmns :
        for i in range(searchstart,searchrange):
          if chair_location[0]-i >=0  and room[chair_location[0]-i][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]]) 
            break 

  if "table" in item or "closet" in item :
    if door_wall == 1 :
      if 0< chair_location[0] <rows :
        for i in range(searchstart,searchrange):
          if chair_location[0]-(i+1) >=0  and room[chair_location[0]-i][chair_location[1]]==0 and room[chair_location[0]-(i+1)][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            coordinates.append([chair_location[0]-(i+1),chair_location[1]])
            break
        if len(coordinates)==0:
          if chair_location[0]-1>=0 and chair_location[1]+1<=colmns and room[chair_location[0]-1][chair_location[1]] ==0and room[chair_location[0]-1][chair_location[1]+1]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
          elif chair_location[0]-1>=0 and chair_location[1]-1>=0 and room[chair_location[0]-1][chair_location[1]] ==0and room[chair_location[0]-1][chair_location[1]-1]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]])
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
    
    if door_wall == 3  :
      if 0< chair_location[0] <rows :
        for i in range(searchstart,searchrange):
          if chair_location[0]+(i+1) <=rows  and room[chair_location[0]+i][chair_location[1]]==0 and room[chair_location[0]+(i+1)][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]])
            coordinates.append([chair_location[0]+(i+1),chair_location[1]])
            break
        if len(coordinates)==0:
          if chair_location[0]+1<=rows and chair_location[1]+1<=colmns and room[chair_location[0]+1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]+1]==0 :
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
          elif chair_location[0]+1<=rows and chair_location[1]-1>=0 and room[chair_location[0]+1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]-1]==0 :
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
    
    if door_wall == 3 or door_wall==1 :
      if chair_location[0] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[1]-(i+1) >=0  and room[chair_location[0]][chair_location[1]-i] == 0 and room[chair_location[0]][chair_location[1]-(i+1)] == 0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            coordinates.append([chair_location[0],chair_location[1]-(i+1)])
            break
      elif chair_location[0] == rows :
        for i in range(searchstart,searchrange):
          if chair_location[1]+(i+1) <=colmns  and room[chair_location[0]][chair_location[1]+i] == 0 and room[chair_location[0]][chair_location[1]+(i+1) ]== 0:
            coordinates.append([chair_location[0],chair_location[1]+i])
            coordinates.append([chair_location[0],chair_location[1]+(i+1)])
            break

    if door_wall == 2  :
      if 0< chair_location[1] <colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]+(i+1) <=colmns and room[chair_location[0]][chair_location[1]+i]==0 and room[chair_location[0]][chair_location[1]+(i+1)]==0:
            coordinates.append([chair_location[0],chair_location[1]+i])
            coordinates.append([chair_location[0],chair_location[1]+(i+1)])
            break
        if len(coordinates)==0:
          if chair_location[1]+1 <=colmns and chair_location[0]+1<=rows and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]+1][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
          elif chair_location[1]+1<=colmns and chair_location[0]-1>=0 and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]-1][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
            
    if door_wall == 4  :
      if 0< chair_location[1] <colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]-(i+1) >=0  and room[chair_location[0]][chair_location[1]-i]==0 and room[chair_location[0]][chair_location[1]-(i+1)]==0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            coordinates.append([chair_location[0],chair_location[1]-(i+1)])
            break
        if len(coordinates)==0:
          if chair_location[1]-1>=0 and chair_location[0]+1<=rows and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]+1][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
          elif chair_location[1]-1>=0 and chair_location[0]-1>=0 and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]-1][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
    
    if door_wall == 4 or door_wall==2 :
      if chair_location[1] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[0]+(i+1) <=rows  and room[chair_location[0]+i][chair_location[1]]==0 and room[chair_location[0]+(i+1)][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]]) 
            coordinates.append([chair_location[0]+(i+1),chair_location[1]])
            break 
      elif chair_location[1] == colmns :
        for i in range(searchstart,searchrange):
          if chair_location[0]-(i+1) >=0  and room[chair_location[0]-i][chair_location[1]]==0 and room[chair_location[0]-(i+1)][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            coordinates.append([chair_location[0]-(i+1),chair_location[1]])
            break

  if "desk" in item :
    if door_wall == 1 :
      if 0< chair_location[0] < rows:
        for i in range(searchstart,searchrange):
          if chair_location[0]-(i+2) >=0  and room[chair_location[0]-i][chair_location[1]]==0 and room[chair_location[0]-(i+1)][chair_location[1]]==0 and room[chair_location[0]-(i+2)][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            coordinates.append([chair_location[0]-(i+1),chair_location[1]])
            coordinates.append([chair_location[0]-(i+2),chair_location[1]])
            break
        if len(coordinates)==0:
          if chair_location[0]-1>=0 and chair_location[1]+1<=colmns and chair_location[1]-1>=0 and room[chair_location[0]-1][chair_location[1]-1] ==0and room[chair_location[0]-1][chair_location[1]]==0 and room[chair_location[0]-1][chair_location[1]+1]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
            coordinates.append([chair_location[0]-1,chair_location[1]])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
          elif chair_location[0]-1>=0 and chair_location[1]+2<=colmns and room[chair_location[0]-1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]-1]==0 and room[chair_location[0]-1][chair_location[1]+2]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
            coordinates.append([chair_location[0]-1,chair_location[1]+2])
          elif chair_location[0]-1>=0 and chair_location[1]-2>=0 and room[chair_location[0]-1][chair_location[1]] ==0and room[chair_location[0]-1][chair_location[1]-1]==0 and room[chair_location[0]-1][chair_location[1]-2]==0 :
            coordinates.append([chair_location[0]-1,chair_location[1]-2])
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
            coordinates.append([chair_location[0]-1,chair_location[1]])

    if door_wall == 3 :
      if 0< chair_location[0] < rows:
        for i in range(searchstart,searchrange):
          if chair_location[0]+(i+2)<=rows  and room[chair_location[0]+i][chair_location[1]]==0 and room[chair_location[0]+(i+1)][chair_location[1]]==0 and room[chair_location[0]+(i+2)][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]])
            coordinates.append([chair_location[0]+(i+1),chair_location[1]])
            coordinates.append([chair_location[0]+(i+2),chair_location[1]])
            break
        if len(coordinates)==0:
          if chair_location[0]+1<=rows and chair_location[1]+1<=colmns and chair_location[1]-1>=0  and room[chair_location[0]+1][chair_location[1]-1] ==0and room[chair_location[0]+1][chair_location[1]]==0and room[chair_location[0]+1][chair_location[1]+1]==0 :
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
          elif chair_location[0]+1<=rows and chair_location[1]+2<=colmns and room[chair_location[0]+1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]+1]==0and room[chair_location[0]+1][chair_location[1]+2]==0 :
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
            coordinates.append([chair_location[0]+1,chair_location[1]+2])
          elif chair_location[0]+1<=rows and chair_location[1]-2>=0 and room[chair_location[0]+1][chair_location[1]] ==0and room[chair_location[0]+1][chair_location[1]-1]==0 and room[chair_location[0]+1][chair_location[1]-2]==0:
            coordinates.append([chair_location[0]+1,chair_location[1]])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]-2])
    
    if door_wall == 3 or door_wall==1 :
      if chair_location[0] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[1]-(i+2) >=0  and room[chair_location[0]][chair_location[1]-i] == 0 and room[chair_location[0]][chair_location[1]-(i+1)] == 0 and room[chair_location[0]][chair_location[1]-(i+2)] == 0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            coordinates.append([chair_location[0],chair_location[1]-(i+1)])
            coordinates.append([chair_location[0],chair_location[1]-(i+2)])
            break
      elif chair_location[0] == rows :
        for i in range(searchstart,searchrange):
          if chair_location[1]+(i+2) <= colmns  and room[chair_location[0]][chair_location[1]+i] == 0 and room[chair_location[0]][chair_location[1]+(i+1)] == 0 and room[chair_location[0]][chair_location[1]+(i+2)] == 0:
            coordinates.append([chair_location[0],chair_location[1]+i])
            coordinates.append([chair_location[0],chair_location[1]+(i+1)])
            coordinates.append([chair_location[0],chair_location[1]+(i+2)])
            break

    
    if door_wall == 2 :
      if 0<chair_location[1] < colmns :
        for i in range(searchstart,searchrange):
          if chair_location[1]+(i+2) <=rwos  and room[chair_location[0]][chair_location[1]+i]==0 and room[chair_location[0]][chair_location[1]+(i+1)]==0 and room[chair_location[0]][chair_location[1]+(i+2)]==0:
            coordinates.append([chair_location[0],chair_location[1]+i])
            coordinates.append([chair_location[0],chair_location[1]+(i+1)])
            coordinates.append([chair_location[0],chair_location[1]+(i+2)])
            break
        if len(coordinates)==0:
          if chair_location[1]+1<=colmns and chair_location[0]+1<=rows and chair_location[0]-1>=0 and room[chair_location[0]-1][chair_location[1]+1]==0 and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]+1][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
          elif chair_location[1]+1<=colmns  and chair_location[0]+2<=rows and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]+1][chair_location[1]+1]==0 and room[chair_location[0]+2][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]+1,chair_location[1]+1])
            coordinates.append([chair_location[0]+2,chair_location[1]+1])
          elif chair_location[1]+1<=colmns  and chair_location[0]-2>=0 and room[chair_location[0]][chair_location[1]+1]==0 and room[chair_location[0]-1][chair_location[1]+1]==0 and room[chair_location[0]-2][chair_location[1]+1]==0: 
            coordinates.append([chair_location[0],chair_location[1]+1])
            coordinates.append([chair_location[0]-1,chair_location[1]+1])
            coordinates.append([chair_location[0]-2,chair_location[1]+1])

    if door_wall == 4:
      if 0<chair_location[1] < colmns  :
        for i in range(searchstart,searchrange):
          if chair_location[1]-(i+2) >=0  and room[chair_location[0]][chair_location[1]-i]==0 and room[chair_location[0]][chair_location[1]-(i+1)]==0 and room[chair_location[0]][chair_location[1]-(i+2)]==0:
            coordinates.append([chair_location[0],chair_location[1]-i])
            coordinates.append([chair_location[0],chair_location[1]-(i+1)])
            coordinates.append([chair_location[0],chair_location[1]-(i+2)])
            break
        if len(coordinates)==0:
          if chair_location[1]-1>=0 and chair_location[0]+1<=rows and chair_location[0]-1>=0 and room[chair_location[0]-1][chair_location[1]-1]==0 and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]+1][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
          elif chair_location[1]-1>=0 and chair_location[0]+2<=rows and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]+1][chair_location[1]-1]==0 and room[chair_location[0]+2][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]+1,chair_location[1]-1])
            coordinates.append([chair_location[0]+2,chair_location[1]-1])
          elif chair_location[1]-1>=0 and chair_location[0]-2>=0 and room[chair_location[0]][chair_location[1]-1]==0 and room[chair_location[0]-1][chair_location[1]-1]==0 and room[chair_location[0]-2][chair_location[1]-1]==0: 
            coordinates.append([chair_location[0],chair_location[1]-1])
            coordinates.append([chair_location[0]-1,chair_location[1]-1])
            coordinates.append([chair_location[0]-2,chair_location[1]-1])
    
    if door_wall == 4 or door_wall==2:
      if chair_location[1] == 0 :
        for i in range(searchstart,searchrange):
          if chair_location[0]+(i+2) <=rows  and room[chair_location[0]+i][chair_location[1]]==0 and room[chair_location[0]+(i+1)][chair_location[1]]==0  and room[chair_location[0]+(i+2)][chair_location[1]]==0:
            coordinates.append([chair_location[0]+i,chair_location[1]]) 
            coordinates.append([chair_location[0]+(i+1),chair_location[1]]) 
            coordinates.append([chair_location[0]+(i+2),chair_location[1]]) 
            break
      elif chair_location[1] == colmns :
        for i in range(searchstart,searchrange):
          if chair_location[0]-(i+2) >=0  and room[chair_location[0]-i][chair_location[1]]==0 and room[chair_location[0]-(i+1)][chair_location[1]]==0 and room[chair_location[0]-(i+2)][chair_location[1]]==0:
            coordinates.append([chair_location[0]-i,chair_location[1]])
            coordinates.append([chair_location[0]-(i+1),chair_location[1]])
            coordinates.append([chair_location[0]-(i+2),chair_location[1]])
            break

  room = add_objects_to_room(room,coordinates)
  return coordinates, room 

In [ ]:
def location_chair_direction_beside_next(item ,room,chair_location,door_wall,searchstart,searchrange):
  coordinates , room = location_chair_direction_left(item=item ,room=room,chair_location=chair_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates)==0:
    coordinates , room = location_chair_direction_right(item=item ,room=room,chair_location=chair_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  return coordinates , room 

In [ ]:
def location_table_closet_desk_direction_right(item ,room,item_location,door_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 

  item_rows,item_colmns , direction = is_ver_horz(item_location,door_wall)
  
  if "chair" in item:
    if direction == "horizantol" and door_wall == 2 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3)  and len(room[0:item_rows[0]]) > len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if room [item_rows[0]][item_colmns[0]-i]==0 and item_colmns[0]-i>=0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          break

    if direction == "horizantol" and door_wall == 4 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3)  and len(room[0:item_rows[0]]) <= len(room[item_rows[0]+1:len(room)]) ):
      for i in range(searchstart,searchrange) :
        if room [item_rows[0]][item_colmns[-1]+i]==0 and item_colmns[-1]+i<=colmns :
          coordinates.append ([item_rows[0],item_colmns[-1]+i])
          break

    if direction == "horizantol" and door_wall == 1 or( direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range(searchstart,searchrange) :
        if room[item_rows[-1]+i][item_colmns[0]] and item_rows[-1]+i <= rows :
          coordinates.append ([item_rows[-1]+i,item_colmns[0]])
          break
    if direction == "horizantol" and door_wall == 3 or( direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range(searchstart,searchrange) :
        if room[item_rows[0]-i][item_colmns[0]] and item_rows[0]-i <= 0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          break

  ##########################################################################################################################    
  if "table" in item or "closet" in item :
    if direction == "horizantol" and door_wall == 2 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3) and len(room[0:item_rows[0]]) > len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if item_colmns[0]-(i+1)>=0 and room [item_rows[0]][item_colmns[0]-i]==0 and room [item_rows[0]][item_colmns[0]-(i+1)]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[0],item_colmns[0]-(i+1)])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_colmns[0]-i >=0 and item_rows[0]+1<=rows and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]+1][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]+1,item_colmns[0]-i])
            break
          elif item_colmns[0]-i >=0 and item_rows[0]-1>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]-1][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]-1,item_colmns[0]-i])
            break

    if direction == "horizantol" and door_wall == 4 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3) and len(room[0:item_rows[0]]) <= len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if item_colmns[-1]+(i+1)<=rows and room [item_rows[-1]][item_colmns[-1]+i]==0 and room [item_rows[-1]][item_colmns[-1]+(i+1)]==0 :
          coordinates.append ([item_rows[-1],item_colmns[-1]+i])
          coordinates.append ([item_rows[-1],item_colmns[-1]+(i+1)])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_colmns[-1]+i <=colmns and item_rows[-1]+1<=rows and room[item_rows[-1]][item_colmns[-1]+i]==0 and room[item_rows[-1]+1][item_colmns[-1]+i]==0: 
            coordinates.append([item_rows[-1],item_colmns[-1]+i])
            coordinates.append([item_rows[-1]+1,item_colmns[-1]+i])
            break
          elif item_colmns[-1]+i <=colmns and item_rows[-1]-1>=0 and room[item_rows[-1]][item_colmns[-1]+i]==0 and room[item_rows[-1]-1][item_colmns[-1]+i]==0: 
            coordinates.append([item_rows[-1],item_colmns[-1]+i])
            coordinates.append([item_rows[-1]-1,item_colmns[-1]+i])
            break


    if direction == "horizantol" and door_wall == 1 or (direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range(searchstart,searchrange) :
        if item_rows[-1]+(i+1) <= rows and room[item_rows[-1]+i][item_colmns[-1]] ==0 and room[item_rows[-1]+(i+1)][item_colmns[-1]]==0:
          coordinates.append ([item_rows[-1]+i,item_colmns[-1]])
          coordinates.append ([item_rows[-1]+(i+1),item_colmns[-1]])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_rows[-1]+i<=rows and item_colmns[-1]+1<=colmns and room[item_rows[-1]+i][item_colmns[-1]] ==0and room[item_rows[-1]+1][item_colmns[-1]+i]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]+1])
            break
          elif item_rows[-1]+i<=rows and item_colmns[-1]-1>=0 and room[item_rows[-1]+i][item_colmns[-1]] ==0and room[item_rows[-1]+1][item_colmns[-1]-i]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]-1])
            break
    if direction == "horizantol" and door_wall == 3 or ( direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1])) :
      for i in range(searchstart,searchrange) :
        if item_rows[0]-(i+1) >=0 and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-(i+1)][item_colmns[0]]==0:
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-(i+1),item_colmns[0]])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_rows[0]-i>=0 and item_colmns[0]+1<=colmns and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-i][item_colmns[0]+1]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]+1])
            break
          elif item_rows[0]-i>=0 and item_colmns[0]-1>=0 and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-i][item_colmns[0]-1]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]-1])
            break
  #########################################################################################################################3
  if "desk" in item :
    if direction == "horizantol" and door_wall == 2 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3) and len(room[0:item_rows[0]]) > len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if item_colmns[0]-(i+2)>=0 and room [item_rows[0]][item_colmns[0]-i]==0 and room [item_rows[0]][item_colmns[0]-(i+1)]==0 and  room [item_rows[0]][item_colmns[0]-(i+2)]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[0],item_colmns[0]-(i+1)])
          coordinates.append ([item_rows[0],item_colmns[0]-(i+2)])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_colmns[0]-i >=0 and item_rows[0]+1<=rows and item_rows[0]-1>=0  and room[item_rows[0]-1][item_colmns[0]-i]==0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]+1][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0]-1,item_colmns[0]-i])
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]+1,item_colmns[0]-i])
            break
          elif item_colmns[0]-i >=0 and item_rows[0]+2<=rows and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]+1][item_colmns[0]-i]==0 and room[item_rows[0]+2][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]+1,item_colmns[0]-i])
            coordinates.append([item_rows[0]+2,item_colmns[0]-i])
            break
          elif item_colmns[0]-i >=0 and item_rows[0]-2>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]-1][item_colmns[0]-i]==0 and room[item_rows[0]-2][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]-1,item_colmns[0]-i])
            coordinates.append([item_rows[0]-2,item_colmns[0]-i])
            break

    if direction == "horizantol" and door_wall == 4 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3) and len(room[0:item_rows[0]]) <= len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if item_colmns[-1]+(i+2)<=rows and room [item_rows[-1]][item_colmns[-1]+i]==0 and room [item_rows[-1]][item_colmns[-1]+(i+1)]==0 and room [item_rows[-1]][item_colmns[-1]+(i+2)]==0 :
          coordinates.append ([item_rows[-1],item_colmns[-1]+i])
          coordinates.append ([item_rows[-1],item_colmns[-1]+(i+1)])
          coordinates.append ([item_rows[-1],item_colmns[-1]+(i+2)])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_colmns[0]+i <=colmns and item_rows[0]+1<=rows and item_rows[0]-1>=0  and room[item_rows[0]-1][item_colmns[0]+i]==0 and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[0]+1][item_colmns[0]+i]==0: 
            coordinates.append([item_rows[0]-1,item_colmns[0]-i])
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]+1,item_colmns[0]-i])
            break
          elif item_colmns[0]+i <=colmns and item_rows[0]+2<=rows and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[0]+1][item_colmns[0]+i]==0 and room[item_rows[0]+2][item_colmns[0]+i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]+i])
            coordinates.append([item_rows[0]+1,item_colmns[0]+i])
            coordinates.append([item_rows[0]+2,item_colmns[0]+i])
            break
          elif item_colmns[0]+i <=colmns and item_rows[0]-2>=0 and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[0]-1][item_colmns[0]+i]==0 and room[item_rows[0]-2][item_colmns[0]+i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]+i])
            coordinates.append([item_rows[0]-1,item_colmns[0]+i])
            coordinates.append([item_rows[0]-2,item_colmns[0]+i])
            break


    if direction == "horizantol" and door_wall == 1 or (direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range(searchstart,searchrange) :
        if item_rows[-1]+(i+2) <= rows and room[item_rows[-1]+i][item_colmns[-1]] ==0 and room[item_rows[-1]+(i+1)][item_colmns[-1]]==0 and room[item_rows[-1]+(i+2)][item_colmns[-1]]==0:
          coordinates.append ([item_rows[-1]+i,item_colmns[-1]])
          coordinates.append ([item_rows[-1]+(i+1),item_colmns[-1]])
          coordinates.append ([item_rows[-1]+(i+2),item_colmns[-1]])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_rows[-1]+i<=rows and item_colmns[-1]+1<=colmns and  item_colmns[-1]-1>=0 and room[item_rows[-1]+i][item_colmns[-1]-1] ==0and room[item_rows[-1]+i][item_colmns[-1]]==0 and room[item_rows[-1]+i][item_colmns[-1]+1]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]-1])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]+1])
            break
          elif item_rows[-1]+i<=rows and item_colmns[-1]+2<=colmns and room[item_rows[-1]+i][item_colmns[-1]] ==0and room[item_rows[-1]+i][item_colmns[-1]+1]==0 and room[item_rows[-1]+i][item_colmns[-1]+2]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]+1])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]+2])
            break
          elif item_rows[-1]+i<=rows and item_colmns[-1]-2>=0 and room[item_rows[-1]+i][item_colmns[-1]] ==0 and room[item_rows[-1]+i][item_colmns[-1]-1]==0 and room[item_rows[-1]+i][item_colmns[-1]-2]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]-1])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]-2])
            break
    if direction == "horizantol" and door_wall == 3 or ( direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1])) :
      for i in range(searchstart,searchrange) :
        if item_rows[0]-(i+2) >=0 and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-(i+1)][item_colmns[0]]==0 and room[item_rows[0]-(i+2)][item_colmns[0]]==0:
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-(i+1),item_colmns[0]])
          coordinates.append ([item_rows[0]-(i+2),item_colmns[0]])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_rows[0]-i>=0 and item_colmns[0]+1<=colmns and  item_colmns[0]-1>=0 and room[item_rows[0]-i][item_colmns[0]-1] ==0and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[0]-i][item_colmns[0]+1]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]-1])
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]+1])
            break
          elif item_rows[0]-i>=0 and item_colmns[0]+2<=colmns and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-i][item_colmns[0]+1]==0 and room[item_rows[0]-i][item_colmns[0]+2]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]+1])
            coordinates.append([item_rows[0]-i,item_colmns[0]+2])
            break
          elif item_rows[0]-i>=0 and item_colmns[0]-2>=0 and room[item_rows[0]-i][item_colmns[0]] ==0 and room[item_rows[0]-i][item_colmns[0]-1]==0 and room[item_rows[0]-i][item_colmns[0]-2]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]-1])
            coordinates.append([item_rows[0]-i,item_colmns[0]-2])
            break
    

  room = add_objects_to_room(room,coordinates)
  return coordinates, room 

In [ ]:
def location_table_closet_desk_direction_left(item ,room,item_location,door_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 

  item_rows,item_colmns , direction = is_ver_horz(item_location,door_wall)
  
  if "chair" in item:
    if direction == "horizantol" and door_wall == 4 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3)  and len(room[0:item_rows[0]]) <= len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if room [item_rows[0]][item_colmns[0]-i]==0 and item_colmns[0]-i>=0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          break

    if direction == "horizantol" and door_wall == 2 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3)  and len(room[0:item_rows[0]]) > len(room[item_rows[0]+1:len(room)]) ):
      for i in range(searchstart,searchrange) :
        if room [item_rows[0]][item_colmns[-1]+i]==0 and item_colmns[-1]+i<=colmns :
          coordinates.append ([item_rows[0],item_colmns[-1]+i])
          break

    if direction == "horizantol" and door_wall == 3 or( direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range(searchstart,searchrange) :
        if room[item_rows[-1]+i][item_colmns[0]] and item_rows[-1]+i <= rows :
          coordinates.append ([item_rows[-1]+i,item_colmns[0]])
          break
    if direction == "horizantol" and door_wall == 1 or( direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range(searchstart,searchrange) :
        if room[item_rows[0]-i][item_colmns[0]] and item_rows[0]-i <= 0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          break

  ##########################################################################################################################    
  if "table" in item or "closet" in item :
    if direction == "horizantol" and door_wall == 4 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3) and len(room[0:item_rows[0]]) <= len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if item_colmns[0]-(i+1)>=0 and room [item_rows[0]][item_colmns[0]-i]==0 and room [item_rows[0]][item_colmns[0]-(i+1)]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[0],item_colmns[0]-(i+1)])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_colmns[0]-i >=0 and item_rows[0]+1<=rows and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]+1][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]+1,item_colmns[0]-i])
            break
          elif item_colmns[0]-i >=0 and item_rows[0]-1>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]-1][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]-1,item_colmns[0]-i])
            break

    if direction == "horizantol" and door_wall == 2 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3) and len(room[0:item_rows[0]]) > len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if item_colmns[-1]+(i+1)<=rows and room [item_rows[-1]][item_colmns[-1]+i]==0 and room [item_rows[-1]][item_colmns[-1]+(i+1)]==0 :
          coordinates.append ([item_rows[-1],item_colmns[-1]+i])
          coordinates.append ([item_rows[-1],item_colmns[-1]+(i+1)])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_colmns[-1]+i <=colmns and item_rows[-1]+1<=rows and room[item_rows[-1]][item_colmns[-1]+i]==0 and room[item_rows[-1]+1][item_colmns[-1]+i]==0: 
            coordinates.append([item_rows[-1],item_colmns[-1]+i])
            coordinates.append([item_rows[-1]+1,item_colmns[-1]+i])
            break
          elif item_colmns[-1]+i <=colmns and item_rows[-1]-1>=0 and room[item_rows[-1]][item_colmns[-1]+i]==0 and room[item_rows[-1]-1][item_colmns[-1]+i]==0: 
            coordinates.append([item_rows[-1],item_colmns[-1]+i])
            coordinates.append([item_rows[-1]-1,item_colmns[-1]+i])
            break


    if direction == "horizantol" and door_wall == 3 or (direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range(searchstart,searchrange) :
        if item_rows[-1]+(i+1) <= rows and room[item_rows[-1]+i][item_colmns[-1]] ==0 and room[item_rows[-1]+(i+1)][item_colmns[-1]]==0:
          coordinates.append ([item_rows[-1]+i,item_colmns[-1]])
          coordinates.append ([item_rows[-1]+(i+1),item_colmns[-1]])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_rows[-1]+i<=rows and item_colmns[-1]+1<=colmns and room[item_rows[-1]+i][item_colmns[-1]] ==0and room[item_rows[-1]+1][item_colmns[-1]+i]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]+1])
            break
          elif item_rows[-1]+i<=rows and item_colmns[-1]-1>=0 and room[item_rows[-1]+i][item_colmns[-1]] ==0and room[item_rows[-1]+1][item_colmns[-1]-i]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]-1])
            break
    if direction == "horizantol" and door_wall == 1 or ( direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1])) :
      for i in range(searchstart,searchrange) :
        if item_rows[0]-(i+1) >=0 and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-(i+1)][item_colmns[0]]==0:
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-(i+1),item_colmns[0]])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_rows[0]-i>=0 and item_colmns[0]+1<=colmns and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-i][item_colmns[0]+1]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]+1])
            break
          elif item_rows[0]-i>=0 and item_colmns[0]-1>=0 and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-i][item_colmns[0]-1]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]-1])
            break
  #########################################################################################################################3
  if "desk" in item :
    if direction == "horizantol" and door_wall == 4 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3) and len(room[0:item_rows[0]]) <= len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if item_colmns[0]-(i+2)>=0 and room [item_rows[0]][item_colmns[0]-i]==0 and room [item_rows[0]][item_colmns[0]-(i+1)]==0 and  room [item_rows[0]][item_colmns[0]-(i+2)]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[0],item_colmns[0]-(i+1)])
          coordinates.append ([item_rows[0],item_colmns[0]-(i+2)])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_colmns[0]-i >=0 and item_rows[0]+1<=rows and item_rows[0]-1>=0  and room[item_rows[0]-1][item_colmns[0]-i]==0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]+1][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0]-1,item_colmns[0]-i])
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]+1,item_colmns[0]-i])
            break
          elif item_colmns[0]-i >=0 and item_rows[0]+2<=rows and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]+1][item_colmns[0]-i]==0 and room[item_rows[0]+2][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]+1,item_colmns[0]-i])
            coordinates.append([item_rows[0]+2,item_colmns[0]-i])
            break
          elif item_colmns[0]-i >=0 and item_rows[0]-2>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[0]-1][item_colmns[0]-i]==0 and room[item_rows[0]-2][item_colmns[0]-i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]-1,item_colmns[0]-i])
            coordinates.append([item_rows[0]-2,item_colmns[0]-i])
            break

    if direction == "horizantol" and door_wall == 2 or ( direction == "vertical" and (door_wall == 1 or door_wall == 3) and len(room[0:item_rows[0]]) > len(room[item_rows[0]+1:len(room)])):
      for i in range(searchstart,searchrange) :
        if item_colmns[-1]+(i+2)<=rows and room [item_rows[-1]][item_colmns[-1]+i]==0 and room [item_rows[-1]][item_colmns[-1]+(i+1)]==0 and room [item_rows[-1]][item_colmns[-1]+(i+2)]==0 :
          coordinates.append ([item_rows[-1],item_colmns[-1]+i])
          coordinates.append ([item_rows[-1],item_colmns[-1]+(i+1)])
          coordinates.append ([item_rows[-1],item_colmns[-1]+(i+2)])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_colmns[0]+i <=colmns and item_rows[0]+1<=rows and item_rows[0]-1>=0  and room[item_rows[0]-1][item_colmns[0]+i]==0 and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[0]+1][item_colmns[0]+i]==0: 
            coordinates.append([item_rows[0]-1,item_colmns[0]-i])
            coordinates.append([item_rows[0],item_colmns[0]-i])
            coordinates.append([item_rows[0]+1,item_colmns[0]-i])
            break
          elif item_colmns[0]+i <=colmns and item_rows[0]+2<=rows and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[0]+1][item_colmns[0]+i]==0 and room[item_rows[0]+2][item_colmns[0]+i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]+i])
            coordinates.append([item_rows[0]+1,item_colmns[0]+i])
            coordinates.append([item_rows[0]+2,item_colmns[0]+i])
            break
          elif item_colmns[0]+i <=colmns and item_rows[0]-2>=0 and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[0]-1][item_colmns[0]+i]==0 and room[item_rows[0]-2][item_colmns[0]+i]==0: 
            coordinates.append([item_rows[0],item_colmns[0]+i])
            coordinates.append([item_rows[0]-1,item_colmns[0]+i])
            coordinates.append([item_rows[0]-2,item_colmns[0]+i])
            break


    if direction == "horizantol" and door_wall == 3 or (direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range(searchstart,searchrange) :
        if item_rows[-1]+(i+2) <= rows and room[item_rows[-1]+i][item_colmns[-1]] ==0 and room[item_rows[-1]+(i+1)][item_colmns[-1]]==0 and room[item_rows[-1]+(i+2)][item_colmns[-1]]==0:
          coordinates.append ([item_rows[-1]+i,item_colmns[-1]])
          coordinates.append ([item_rows[-1]+(i+1),item_colmns[-1]])
          coordinates.append ([item_rows[-1]+(i+2),item_colmns[-1]])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_rows[-1]+i<=rows and item_colmns[-1]+1<=colmns and  item_colmns[-1]-1>=0 and room[item_rows[-1]+i][item_colmns[-1]-1] ==0and room[item_rows[-1]+i][item_colmns[-1]]==0 and room[item_rows[-1]+i][item_colmns[-1]+1]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]-1])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]+1])
            break
          elif item_rows[-1]+i<=rows and item_colmns[-1]+2<=colmns and room[item_rows[-1]+i][item_colmns[-1]] ==0and room[item_rows[-1]+i][item_colmns[-1]+1]==0 and room[item_rows[-1]+i][item_colmns[-1]+2]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]+1])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]+2])
            break
          elif item_rows[-1]+i<=rows and item_colmns[-1]-2>=0 and room[item_rows[-1]+i][item_colmns[-1]] ==0 and room[item_rows[-1]+i][item_colmns[-1]-1]==0 and room[item_rows[-1]+i][item_colmns[-1]-2]==0 :
            coordinates.append([item_rows[-1]+i,item_colmns[-1]])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]-1])
            coordinates.append([item_rows[-1]+i,item_colmns[-1]-2])
            break
    if direction == "horizantol" and door_wall == 1 or ( direction == "vertical" and (door_wall == 2 or door_wall == 4) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1])) :
      for i in range(searchstart,searchrange) :
        if item_rows[0]-(i+2) >=0 and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-(i+1)][item_colmns[0]]==0 and room[item_rows[0]-(i+2)][item_colmns[0]]==0:
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-(i+1),item_colmns[0]])
          coordinates.append ([item_rows[0]-(i+2),item_colmns[0]])
          break
      if len(coordinates)==0:
        for i in range(searchstart,searchrange) :
          if item_rows[0]-i>=0 and item_colmns[0]+1<=colmns and  item_colmns[0]-1>=0 and room[item_rows[0]-i][item_colmns[0]-1] ==0and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[0]-i][item_colmns[0]+1]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]-1])
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]+1])
            break
          elif item_rows[0]-i>=0 and item_colmns[0]+2<=colmns and room[item_rows[0]-i][item_colmns[0]] ==0and room[item_rows[0]-i][item_colmns[0]+1]==0 and room[item_rows[0]-i][item_colmns[0]+2]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]+1])
            coordinates.append([item_rows[0]-i,item_colmns[0]+2])
            break
          elif item_rows[0]-i>=0 and item_colmns[0]-2>=0 and room[item_rows[0]-i][item_colmns[0]] ==0 and room[item_rows[0]-i][item_colmns[0]-1]==0 and room[item_rows[0]-i][item_colmns[0]-2]==0 :
            coordinates.append([item_rows[0]-i,item_colmns[0]])
            coordinates.append([item_rows[0]-i,item_colmns[0]-1])
            coordinates.append([item_rows[0]-i,item_colmns[0]-2])
            break
    

  room = add_objects_to_room(room,coordinates)
  return coordinates, room 

In [ ]:
def location_table_desk_direction_behind(item ,location_item,room,item_location,door_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 
  item_rows,item_colmns , direction = is_ver_horz(item_location,door_wall)
  if "dc" in item  and "desk" in location_item :
    if direction == "horizantol" and door_wall == 2 :
      if rows in item_rows and room[item_rows[1]-1][item_colmns[1]]==0 :
        coordinates.append ([item_rows[1]-1,item_colmns[1]])
      else:
        coordinates.append ([item_rows[1]+1,item_colmns[1]])

    if direction == "horizantol" and door_wall == 4 :
      if 0 in item_rows and room[item_rows[1]+1][item_colmns[1]]==0:
        coordinates.append ([item_rows[1]+1,item_colmns[1]]) 
      else:
        coordinates.append ([item_rows[1]-1,item_colmns[1]])

    elif direction == "vertical" and (door_wall == 2 or door_wall == 4) :
      if 0 in item_colmns and room[item_rows[1]][item_colmns[0]+1] ==0:
        coordinates.append([item_rows[1],item_colmns[1]+1])
      elif colmns in item_colmns and room[item_rows[1]][item_colmns[0]-1] ==0:
        coordinates.append([item_rows[1],item_colmns[1]-1])   
      elif len(room[0][0:item_colmns[1]]) <= len(room[0][item_colmns[1]+1:len(room[0])-1]) and 0 not in item_colmns and colmns not in item_colmns :
        if room[item_rows[1]][item_colmns[1]-1] ==0:
          coordinates.append([item_rows[1],item_colmns[1]-1])
      elif len(room[0][0:item_colmns[1]]) > len(room[0][item_colmns[1]+1:len(room[0])-1]) and 0 not in item_colmns and colmns not in item_colmns :
        if room[item_rows[1]][item_colmns[1]+1] ==0:
          coordinates.append([item_rows[1],item_colmns[1]+1]) 

    if direction == "horizantol" and door_wall == 1 :
      if colmns in item_colmns and room[item_rows[1]][item_colmns[1]-1]==0:
        coordinates.append ([item_rows[1],item_colmns[1]-1])
      else :
        coordinates.append ([item_rows[1],item_colmns[1]+1])

    if direction == "horizantol" and door_wall == 3 :
      if 0 in item_colmns and room[item_rows[1]][item_colmns[1]+1]==0  :
        coordinates.append ([item_rows[1],item_colmns[1]+1])
      else :
        coordinates.append ([item_rows[1],item_colmns[1]-1])

    elif direction == "vertical" and (door_wall == 1 or door_wall == 3) :
      if 0 in item_rows and room[item_rows[1]+1][item_colmns[1]] ==0:
        coordinates.append([item_rows[1]+1,item_colmns[1]])
      elif rows in item_rows and room[item_rows[1]-1][item_colmns[1]] ==0:
        coordinates.append([item_rows[1]-1,item_colmns[1]])
      elif len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) and 0 not in item_rows and rows not in item_rows :
        if room[item_rows[1]-1][item_colmns[1]] ==0:
          coordinates.append([item_rows[1]-1,item_colmns[1]])
      elif len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) and 0 not in item_rows and rows not in item_rows :
        if room[item_rows[1]+1][item_colmns[1]] ==0:
          coordinates.append([item_rows[1]+1,item_colmns[1]])
  ##########################################################################################
  if "chair" in item and "table" in location_item:
    if ( direction == "horizantol" and door_wall == 2 )or ((direction == "vertical" and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)])):
      for i in range (searchstart,searchrange) :
        if rows not in item_rows and item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[0]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          break
        elif rows not in item_rows and item_rows[1]+i <=rows  and room[item_rows[1]+i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[1]+i,item_colmns[1]])
          break

    if (direction == "horizantol" and door_wall == 4 )or ((direction == "vertical" and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)])):
      for i in range (searchstart,searchrange) :
        if 0 not in item_rows and item_rows[0]-i>=0 and room[item_rows[0]-i][item_colmns[0]]==0:
          coordinates.append ([item_rows[0]-i,item_colmns[0]]) 
          break
        elif 0 not in item_rows and item_rows[1]-i>=0 and room[item_rows[1]-i][item_colmns[1]]==0:
          coordinates.append ([item_rows[1]-i,item_colmns[1]]) 
          break

    if (direction == "horizantol" and door_wall == 1) or((direction == "vertical" and (door_wall == 2 or door_wall == 4))and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range (searchstart,searchrange) :
        if colmns not in item_colmns and item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          break
        elif colmns not in item_colmns and item_colmns[1]+i<=colmns and room[item_rows[1]][item_colmns[1]+i]==0:
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          break

    if ( direction == "horizantol" and door_wall == 3) or((direction == "vertical" and (door_wall == 2 or door_wall == 4))and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range (searchstart,searchrange) :
        if 0 in item_colmns and item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0  :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          break
        elif 0 in item_colmns and item_colmns[1]-i>=0 and room[item_rows[1]][item_colmns[1]-i]==0  :
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          break

  ############################################################################################################################
  if "desk" in item and "desk" in location_item :
    if (direction == "horizantol" and door_wall == 2) or ((direction == "vertical "and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[1]+i][item_colmns[1]]==0 and  room[item_rows[2]+i][item_colmns[2]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[1]+i,item_colmns[1]])
          coordinates.append ([item_rows[2]+i,item_colmns[2]])
          break

    if (direction == "horizantol" and door_wall == 4 )or ((direction == "vertical "and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]-i >=0 and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[1]-i][item_colmns[1]]==0 and  room[item_rows[2]-i][item_colmns[2]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[1]-i,item_colmns[1]])
          coordinates.append ([item_rows[2]-i,item_colmns[2]])
          break

    if (direction == "horizantol" and door_wall == 1 )or ((direction == "vertical "and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[1]+i]==0 and room[item_rows[2]][item_colmns[2]+i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          coordinates.append ([item_rows[2],item_colmns[2]+i])
          break

    if (direction == "horizantol" and door_wall == 3) or ((direction == "vertical "and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[1]-i]==0 and room[item_rows[2]][item_colmns[2]-i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          coordinates.append ([item_rows[2],item_colmns[2]-i])
          break
  ##################################################################################################333
  if ("table" in item or "closet" in item) and "desk" in location_item :
    if (direction == "horizantol" and door_wall == 2) or ((direction == "vertical" and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[0]+i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[0]+i,item_colmns[1]])
          break
        elif item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[1]]==0 and  room[item_rows[0]+i][item_colmns[2]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[1]])
          coordinates.append ([item_rows[0]+i,item_colmns[2]])
          break

    if (direction == "horizantol" and door_wall == 4) or ((direction == "vertical" and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]-i >=0 and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[0]-i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-i,item_colmns[1]])
          break
        elif item_rows[0]-i >=0 and room[item_rows[0]-i][item_colmns[1]]==0 and  room[item_rows[0]-i][item_colmns[2]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[1]])
          coordinates.append ([item_rows[0]-i,item_colmns[2]])
          break

    if (direction == "horizantol" and door_wall == 1 )or ((direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[0]+i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[0]+i])
          break
        elif item_colmns[0]+i<=colmns  and room[item_rows[1]][item_colmns[0]+i]==0 and room[item_rows[2]][item_colmns[0]+i]==0:
          coordinates.append ([item_rows[1],item_colmns[0]+i])
          coordinates.append ([item_rows[2],item_colmns[0]+i])
          break

    if (direction == "horizantol" and door_wall == 3) or ((direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[0]-i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[0]-i])
          break
        elif item_colmns[0]-i>=0 and room[item_rows[1]][item_colmns[0]-i]==0 and room[item_rows[2]][item_colmns[0]-i]==0:
          coordinates.append ([item_rows[1],item_colmns[0]-i])
          coordinates.append ([item_rows[2],item_colmns[0]-i])
          break
  #############################################################################################################

  if "desk" in item and "table" in location_item:
    if (direction == "horizantol" and door_wall == 2) or ((direction == "vertical" and (door_wall == 1 or door_wall == 3))and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]+i <=rows and item_colmns[0]-1>=0 and room[item_rows[0]+i][item_colmns[0]-1]==0 and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[1]+i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]-1])
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[0]+i,item_colmns[1]])
          break
        elif item_rows[0]+i <=rows and item_colmns[1]+1<=colmns and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[1]+i][item_colmns[1]]==0 and room[item_rows[1]+i][item_colmns[1]+1]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[0]+i,item_colmns[1]])
          coordinates.append ([item_rows[0]+i,item_colmns[1]+1])
          break
    if (direction == "horizantol" and door_wall == 4) or ((direction == "vertical" and (door_wall == 1 or door_wall == 3))and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]-i >=0 and item_colmns[0]-1>=0 and room[item_rows[0]-i][item_colmns[0]-1]==0 and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[1]-i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]-1])
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-i,item_colmns[1]])
          break
        elif item_rows[0]-i >=0 and item_colmns[1]+1<=colmns and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[1]-i][item_colmns[1]]==0 and room[item_rows[1]-i][item_colmns[1]+1]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-i,item_colmns[1]])
          coordinates.append ([item_rows[0]-i,item_colmns[1]+1])
          break

    if (direction == "horizantol" and door_wall == 1 )or ( (direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[1]]) > len(room[0][item_colmns[1]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]+i<=colmns and item_rows[0]-1>=0 and room[item_rows[0]-1][item_colmns[0]+i]==0 and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[1]+i]==0:
          coordinates.append ([item_rows[0]-1,item_colmns[0]+i])
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          break
        elif item_colmns[0]+i<=colmns and item_rows[1]+1<=rows and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[1]+i]==0 and room[item_rows[1]+1][item_colmns[1]+i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          coordinates.append ([item_rows[1]+1,item_colmns[1]+i])
          break

    if (direction == "horizantol" and door_wall == 3 )or ( (direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[1]]) <= len(room[0][item_colmns[1]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and item_rows[0]-1>=0 and room[item_rows[0]-1][item_colmns[0]-i]==0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[1]-i]==0:
          coordinates.append ([item_rows[0]-1,item_colmns[0]-i])
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          break
        elif item_colmns[0]-i>=0 and item_rows[1]+1<=rows and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[1]-i]==0 and room[item_rows[1]+1][item_colmns[1]-i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          coordinates.append ([item_rows[1]+1,item_colmns[1]-i])
          break

  #########################################################################################################
  if ("table" in item or "closet" in item) and "table" in location_item :
    if (direction == "horizantol" and door_wall == 2) or ((direction == "vertical "and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[1]+i][item_colmns[1]]==0  :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[1]+i,item_colmns[1]])
          break

    if (direction == "horizantol" and door_wall == 4) or ((direction == "vertical "and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]-i >=0 and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[1]-i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[1]-i,item_colmns[1]])
          break

    if (direction == "horizantol" and door_wall == 1 )or ((direction == "vertical "and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[1]+i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          break

    if (direction == "horizantol" and door_wall == 3) or ((direction == "vertical "and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[1]-i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          break
  ##################################################################################
  room = add_objects_to_room(room,coordinates)
  return coordinates, room 


In [ ]:
def location_table_desk_direction_infront(item ,location_item,room,item_location,door_location,door_wall,searchstart,searchrange):
  rows=len(room)-1
  colmns=len(room[0])-1
  coordinates =[] 

  item_rows,item_colmns , direction = is_ver_horz(item_location,door_wall)
  if "chair" in item  and "desk" in location_item :
    if (direction == "horizantol" and door_wall == 2) or((direction == "vertical" and (door_wall == 1 or door_wall == 3))and len(room[0:item_rows[0]]) > len(room[item_rows[0]+1:len(room)]) and 0 not in item_rows and rows not in item_rows) :
      for i in range (searchstart,searchrange):
        if item_rows[0]-i>=0 and room [item_rows[0]-i][item_colmns[0]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          break
        elif item_rows[-1]-i >=0 and room [item_rows[-1]-i][item_colmns[-1]]==0 :
          coordinates.append ([item_rows[-1]-i,item_colmns[-1]])
          break

    if (direction == "horizantol" and door_wall == 4 )or((direction == "vertical" and (door_wall == 1 or door_wall == 3))and len(room[0:item_rows[0]]) <= len(room[item_rows[0]+1:len(room)]) and 0 not in item_rows and rows not in item_rows) : 
      for i in range (searchstart,searchrange):
        if item_rows[0]+i<=rows and room [item_rows[0]+i][item_colmns[0]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          break
        elif item_rows[-1]+i<=rows and room [item_rows[-1]+i][item_colmns[-1]]==0 :
          coordinates.append ([item_rows[-1]+i,item_colmns[-1]])
          break

    if (direction == "horizantol" and door_wall == 1 )or ((direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range (searchstart,searchrange):
        if item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0  :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          break
        elif item_colmns[-1]-i>=0 and room[item_rows[-1]][item_colmns[-1]-i]==0 :
          coordinates.append ([item_rows[-1],item_colmns[-1]-i])
          break
  
    if (direction == "horizantol" and door_wall == 3) or ((direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange):
        if item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          break
        elif item_colmns[-1]+i<=colmns and room[item_rows[-1]][item_colmns[-1]+i] ==0:
          coordinates.append ([item_rows[-1],item_colmns[-1]+i])
          break  
  ##########################################################################################
  if "chair" in item and "table" in location_item:
    if (direction == "horizantol" and door_wall == 4 )or ((direction == "vertical" and (door_wall == 1 or door_wall == 3))and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)])):
      for i in range (searchstart,searchrange) :
        if  item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[0]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          break
        elif item_rows[1]+i <=rows  and room[item_rows[1]+i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[1]+i,item_colmns[1]])
          break

    if (direction == "horizantol" and door_wall == 2) or ((direction == "vertical" and (door_wall == 1 or door_wall == 3))and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)])) :
      for i in range (searchstart,searchrange) :
        if  item_rows[0]-i>=0 and room[item_rows[0]-i][item_colmns[0]]==0:
          coordinates.append ([item_rows[0]-i,item_colmns[0]]) 
          break
        elif item_rows[1]-i>=0 and room[item_rows[1]-i][item_colmns[1]]==0:
          coordinates.append ([item_rows[1]-i,item_colmns[1]]) 
          break

    if (direction == "horizantol" and door_wall == 1 )or ((direction == "vertical" and (door_wall == 2 or door_wall == 4))and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range (searchstart,searchrange) :
        if  item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          break
        elif  item_colmns[1]+i<=colmns and room[item_rows[1]][item_colmns[1]+i]==0:
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          break

    if (direction == "horizantol" and door_wall == 3) or ((direction == "vertical" and (door_wall == 2 or door_wall == 4))and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1])):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0  :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          break
        elif item_colmns[1]-i>=0 and room[item_rows[1]][item_colmns[1]-i]==0  :
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          break

  ############################################################################################################################
  if "desk" in item and "desk" in location_item :
    if (direction == "horizantol" and door_wall == 4 )or ((direction == "vertical "and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[1]+i][item_colmns[1]]==0 and  room[item_rows[2]+i][item_colmns[2]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[1]+i,item_colmns[1]])
          coordinates.append ([item_rows[2]+i,item_colmns[2]])
          break

    if (direction == "horizantol" and door_wall == 2) or ((direction == "vertical "and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]-i >=0 and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[1]-i][item_colmns[1]]==0 and  room[item_rows[2]-i][item_colmns[2]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[1]-i,item_colmns[1]])
          coordinates.append ([item_rows[2]-i,item_colmns[2]])
          break

    if( direction == "horizantol" and door_wall == 3) or ((direction == "vertical "and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[1]+i]==0 and room[item_rows[2]][item_colmns[2]+i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          coordinates.append ([item_rows[2],item_colmns[2]+i])
          break

    if (direction == "horizantol" and door_wall == 1) or ((direction == "vertical "and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[1]-i]==0 and room[item_rows[2]][item_colmns[2]-i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          coordinates.append ([item_rows[2],item_colmns[2]-i])
          break
  ##################################################################################################333
  if ("table" in item or "closet" in item) and "desk" in location_item :
    if (direction == "horizantol" and door_wall == 4 )or ((direction == "vertical" and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[0]+i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[0]+i,item_colmns[1]])
          break
        elif item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[1]]==0 and  room[item_rows[0]+i][item_colmns[2]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[1]])
          coordinates.append ([item_rows[0]+i,item_colmns[2]])
          break

    if (direction == "horizantol" and door_wall == 2 )or ((direction == "vertical" and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]-i >=0 and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[0]-i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-i,item_colmns[1]])
          break
        elif item_rows[0]-i >=0 and room[item_rows[0]-i][item_colmns[1]]==0 and  room[item_rows[0]-i][item_colmns[2]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[1]])
          coordinates.append ([item_rows[0]-i,item_colmns[2]])
          break

    if (direction == "horizantol" and door_wall == 3 )or ((direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[0]+i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[0]+i])
          break
        elif item_colmns[0]+i<=colmns  and room[item_rows[1]][item_colmns[0]+i]==0 and room[item_rows[2]][item_colmns[0]+i]==0:
          coordinates.append ([item_rows[1],item_colmns[0]+i])
          coordinates.append ([item_rows[2],item_colmns[0]+i])
          break

    if (direction == "horizantol" and door_wall == 1) or ((direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[0]-i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[0]-i])
          break
        elif item_colmns[0]-i>=0 and room[item_rows[1]][item_colmns[0]-i]==0 and room[item_rows[2]][item_colmns[0]-i]==0:
          coordinates.append ([item_rows[1],item_colmns[0]-i])
          coordinates.append ([item_rows[2],item_colmns[0]-i])
          break
  #############################################################################################################

  if "desk" in item and "table" in location_item:
    if (direction == "horizantol" and door_wall == 4 )or ((direction == "vertical" and (door_wall == 1 or door_wall == 3))and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]+i <=rows and item_colmns[0]-1>=0 and room[item_rows[0]+i][item_colmns[0]-1]==0 and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[1]+i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]-1])
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[0]+i,item_colmns[1]])
          break
        elif item_rows[0]+i <=rows and item_colmns[1]+1<=colmns and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[1]+i][item_colmns[1]]==0 and room[item_rows[1]+i][item_colmns[1]+1]==0 :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[0]+i,item_colmns[1]])
          coordinates.append ([item_rows[0]+i,item_colmns[1]+1])
          break
    if (direction == "horizantol" and door_wall == 2) or ((direction == "vertical" and (door_wall == 1 or door_wall == 3))and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]-i >=0 and item_colmns[0]-1>=0 and room[item_rows[0]-i][item_colmns[0]-1]==0 and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[1]-i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]-1])
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-i,item_colmns[1]])
          break
        elif item_rows[0]-i >=0 and item_colmns[1]+1<=colmns and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[1]-i][item_colmns[1]]==0 and room[item_rows[1]-i][item_colmns[1]+1]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[0]-i,item_colmns[1]])
          coordinates.append ([item_rows[0]-i,item_colmns[1]+1])
          break

    if (direction == "horizantol" and door_wall == 3 )or ( (direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[1]]) <= len(room[0][item_colmns[1]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]+i<=colmns and item_rows[0]-1>=0 and room[item_rows[0]-1][item_colmns[0]+i]==0 and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[1]+i]==0:
          coordinates.append ([item_rows[0]-1,item_colmns[0]+i])
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          break
        elif item_colmns[0]+i<=colmns and item_rows[1]+1<=rows and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[1]+i]==0 and room[item_rows[1]+1][item_colmns[1]+i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          coordinates.append ([item_rows[1]+1,item_colmns[1]+i])
          break

    if (direction == "horizantol" and door_wall == 1) or ( (direction == "vertical" and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[1]]) > len(room[0][item_colmns[1]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and item_rows[0]-1>=0 and room[item_rows[0]-1][item_colmns[0]-i]==0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[1]-i]==0:
          coordinates.append ([item_rows[0]-1,item_colmns[0]-i])
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          break
        elif item_colmns[0]-i>=0 and item_rows[1]+1<=rows and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[1]-i]==0 and room[item_rows[1]+1][item_colmns[1]-i]==0:
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          coordinates.append ([item_rows[1]+1,item_colmns[1]-i])
          break

  #########################################################################################################
  if ("table" in item or "closet" in item) and "table" in location_item :
    if( direction == "horizantol" and door_wall == 4) or ((direction == "vertical "and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) <= len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]+i <=rows and room[item_rows[0]+i][item_colmns[0]]==0 and room[item_rows[1]+i][item_colmns[1]]==0  :
          coordinates.append ([item_rows[0]+i,item_colmns[0]])
          coordinates.append ([item_rows[1]+i,item_colmns[1]])
          break

    if (direction == "horizantol" and door_wall == 2) or ((direction == "vertical "and (door_wall == 1 or door_wall == 3)) and len(room[0:item_rows[1]]) > len(room[item_rows[1]+1:len(room)]) ):
      for i in range (searchstart,searchrange) :
        if item_rows[0]-i >=0 and room[item_rows[0]-i][item_colmns[0]]==0 and room[item_rows[1]-i][item_colmns[1]]==0 :
          coordinates.append ([item_rows[0]-i,item_colmns[0]])
          coordinates.append ([item_rows[1]-i,item_colmns[1]])
          break

    if (direction == "horizantol" and door_wall == 3) or ((direction == "vertical "and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) <= len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]+i<=colmns and room[item_rows[0]][item_colmns[0]+i]==0 and room[item_rows[1]][item_colmns[1]+i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]+i])
          coordinates.append ([item_rows[1],item_colmns[1]+i])
          break

    if ( direction == "horizantol" and door_wall == 1 )or ((direction == "vertical "and (door_wall == 2 or door_wall == 4)) and len(room[0][0:item_colmns[0]]) > len(room[0][item_colmns[0]+1:len(room[0])-1]) ):
      for i in range (searchstart,searchrange) :
        if item_colmns[0]-i>=0 and room[item_rows[0]][item_colmns[0]-i]==0 and room[item_rows[1]][item_colmns[1]-i]==0 :
          coordinates.append ([item_rows[0],item_colmns[0]-i])
          coordinates.append ([item_rows[1],item_colmns[1]-i])
          break
  ##################################################################################
  room = add_objects_to_room(room,coordinates)
  return coordinates, room 


In [ ]:
def location_table_closet_desk_direction_beside_next(item ,room,item_location,door_location,door_wall,searchstart,searchrange):
  coordinates ,room = location_table_closet_desk_direction_right(item=item ,room=room,item_location=item_location,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates)==0:
    coordinates ,room = location_table_closet_desk_direction_left(item=item ,room=room,item_location=item_location,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  
  return coordinates , room

In [ ]:
def location_table_closet_desk_direction_near(item ,location_item,room,item_location,door_location,door_wall,searchstart,searchrange):
  coordinates ,room = location_table_closet_desk_direction_right(item=item ,room=room,item_location=item_location,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates)==0:
    coordinates ,room = location_table_closet_desk_direction_left(item=item ,room=room,item_location=item_location,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates)==0:
    coordinates ,room = location_table_desk_direction_behind(item=item ,location_item=location_item,room=room,item_location=item_location,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  if len(coordinates)==0:
    coordinates ,room = location_table_desk_direction_infront(item=item ,location_item=location_item,room=room,item_location=item_location,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
  return coordinates , room

In [ ]:
def get_door_window_coordinates(door_wall,begin,end,room):
  rows=len(room)-1 
  colmns=len(room[0])-1
  coordinates=[]
  if door_wall == 1 :
    for i in range(begin,end+1):
      coordinates.append([i,0])
  elif door_wall == 3 :
    for i in range(begin,end+1):
      coordinates.append([i,colmns])
  elif door_wall == 2 :
    for i in range(begin,end+1):
      coordinates.append([0,i])
  elif door_wall == 4 :
    for i in range(begin,end+1):
      coordinates.append([rows,i])
    print(555)
  return coordinates
# d=[{"door1":[0,7,12],"door2":[0,0,4]}]


In [ ]:

def get_dw_wall_dw_location(walls,dw_name="door"):
  dw_wall=None
  dw_location=None
  wall1=walls[0]
  wall2=walls[1]
  wall3=walls[2]
  wall4=walls[3]
  for item , location in wall1.items() :
    if dw_name in item:
      dw_wall=location[0]+1
      begin=location[1]
      end=location[2]
      dw_location= get_door_window_coordinates(dw_wall,begin,end,room)
      break
  if dw_wall ==None and wall2!=None:
    for item , location in wall2.items() :
      if dw_name in item:
        dw_wall=location[0]+1
        begin=location[1]
        end=location[2]
        dw_location= get_door_window_coordinates(dw_wall,begin,end,room)
        break
  if dw_wall ==None and wall3!=None:
    for item , location in wall3.items() :
      if dw_name in item:
        dw_wall=location[0]+1
        begin=location[1]
        end=location[2]
        dw_location= get_door_window_coordinates(dw_wall,begin,end,room)
        break 
  if dw_wall ==None and wall4!=None:
    for item , location in wall4.items() :
      if dw_name in item:
        dw_wall=location[0]+1
        begin=location[1]
        end=location[2]
        dw_location= get_door_window_coordinates(dw_wall,begin,end,room)
        break
  return dw_wall , dw_location

In [ ]:
def get_searchstart_searchrange(loc,obj,dir) :
  searchstart , searchrange = 0 ,0 
  if "door" in loc :
    if ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and (dir =="right" or dir == "left" or dir =="beside" or dir =="next"):
      searchstart , searchrange = 1 , 3
    elif ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and (dir =="infront" or dir == "opposite"):
      searchstart , searchrange = 3 , 5
    elif ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and dir =="near" :
      searchstart , searchrange = 1 , 4
  ##############################################
  elif "window" in loc :
    if ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and (dir =="under" or dir =="below"):
      searchstart , searchrange = 0 , 1
    elif ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and (dir =="infront" or dir =="oppsite"):
      searchstart , searchrange = 0 , 2
    elif ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and (dir =="right" or dir =="left" or dir =="beside" or dir =="next"):
      searchstart , searchrange = 1, 3    
    elif ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and dir =="near" :
      searchstart , searchrange = 1 , 4
  ################################################
  elif "chair" in loc :
    if ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and (dir =="right" or dir =="left" or dir =="beside" or dir =="next"):
      searchstart , searchrange = 1, 3    
    elif ("chair" in obj or "dc" in obj or "closet" in obj or "table" in obj or "desk" in obj ) and dir =="near" :
      searchstart , searchrange = 1 , 4
  ###############################################
  elif "table" in loc or "closet" in obj:
    if ("chair" in obj or "dc" in obj) and (dir =="right" or dir =="left" or dir =="infront" or dir =="oppsite" or dir=="behind" or dir =="around" ) :
      searchstart , searchrange = 1 , 2
    elif ("chair" in obj or "dc" in obj )and (dir =="beside" or dir =="near" or dir=="next" ) :
      searchstart , searchrange = 1 , 3
    elif "table" in obj or "closet" in obj or "desk" in obj :
      searchstart , searchrange = 3 ,5 
  ###############################################
  elif "desk" in loc :
    if "dc" in obj and dir =="behind" :
      searchstart , searchrange = 1 , 2
    elif ("chair" in obj or "dc" in obj ) and (dir =="right" or dir =="left" or dir =="infront" or dir =="oppsite" or dir=="behind" or dir =="beside" or dir=="next" or dir == "near" ) :
      searchstart , searchrange = 1 , 3
    elif "table" in obj  and (dir =="infront" or dir =="oppsite"):
      searchstart , searchrange = 1 , 3
    elif "table" in obj and (dir =="right" or dir =="left" or dir=="behind" or dir =="beside" or dir=="next" or dir == "near" ) :
      searchstart , searchrange = 3 ,5 
    elif "closet" in obj and (dir =="right" or dir =="left" or dir =="beside" or dir=="next" or dir == "near"  ):
      searchstart , searchrange = 2 ,4 
    elif "closet" in obj and (dir=="behind" or dir=="infront"):
      searchstart , searchrange = 3 ,5 
    elif "desk" in obj and (dir =="right" or dir =="left" or dir =="infront" or dir =="oppsite" or dir=="behind" or dir =="beside" or dir=="next" or dir == "near" ):
      searchstart , searchrange = 3 ,5 

  return searchstart , searchrange 


In [ ]:
def into_coordinates(walls,locations,directions,objects,room):
  refernce_door_wall , refrence_door_location = get_dw_wall_dw_location(walls)
  layout={}
  for loc,dir,obj in zip(locations,directions,objects):
    coordinates=[]
    if "door" in loc :
      door_wall , door_location = get_dw_wall_dw_location(walls,loc)
      if door_wall == refernce_door_wall and door_location == refrence_door_location:
        if dir=="right":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room =  location_door_direction_right(item=obj,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="left":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_door_direction_left(item=obj,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="infront" or dir=="opposite":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_door_direction_infront_oppsite(item=obj,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="beside" or dir=="next":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_door_direction_beside_next(item=obj,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="near" :
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_door_direction_near(item=obj,room=room,door_location=door_location,door_wall=door_wall,searchstart=searchstart,searchrange=searchrange)
      else:
        if dir=="right":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room =  location_window_direction_right(item=obj ,room=room,window_location=door_location,window_wall=door_wall,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="left":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_window_direction_left(item=obj ,room=room,window_location=door_location,window_wall=door_wall,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="infront" or dir=="opposite":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room =location_window_direction_infront_oppsite_under_below(item=obj ,room=room,window_location=door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="beside" or dir=="next":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_window_direction_beside_next(item=obj ,room=room,window_location=door_location,window_wall=door_wall,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="near" :
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room =  location_window_direction_near(item=obj ,room=room,window_location=door_location,window_wall=door_wall,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)        
    #################################################################################################
    elif "window" in loc :
      window_wall , window_location = get_dw_wall_dw_location(walls,loc)
      if dir=="right":
        searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
        coordinates , room =  location_window_direction_right(item=obj ,room=room,window_location=window_location,window_wall=window_wall,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
      elif dir=="left":
        searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
        coordinates , room = location_window_direction_left(item=obj ,room=room,window_location=window_location,window_wall=window_wall,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
      elif dir=="infront" or dir=="opposite":
        searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
        coordinates , room =location_window_direction_infront_oppsite_under_below(item=obj ,room=room,window_location=window_location,window_wall=window_wall,searchstart=searchstart,searchrange=searchrange)
      elif dir=="under" or dir=="below":
        searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
        coordinates , room =location_window_direction_infront_oppsite_under_below(item=obj ,room=room,window_location=window_location,window_wall=window_wall,searchstart=searchstart,searchrange=searchrange)
      elif dir=="beside" or dir=="next":
        searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
        coordinates , room = location_window_direction_beside_next(item=obj ,room=room,window_location=window_location,window_wall=window_wall,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
      elif dir=="near" :
        searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
        print(window_location,window_wall)
        coordinates , room =  location_window_direction_near(item=obj ,room=room,window_location=window_location,window_wall=window_wall,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
    ######################################################################################
    elif "room" in loc:
      if dir=="middle":
        coordinates , room =location_room_direction_middle(item=obj,room=room)
    ###################################################################################
    elif "wall" in loc :
      if door_wall == 1:
        if dir=="front":
          coordinates , room = location_wall3(item=obj,room=room)
        elif dir=="back":
          coordinates , room = location_wall1(item=obj,room=room)
        elif dir=="left":
          coordinates , room = location_wall2(item=obj,room=room)
        elif dir=="right":
          coordinates , room = location_wall4(item=obj,room=room)
      elif door_wall == 3:
        if dir=="front":
          coordinates , room = location_wall1(item=obj,room=room)
        elif dir=="back":
          coordinates , room = location_wall3(item=obj,room=room)
        elif dir=="left":
          coordinates , room = location_wall4(item=obj,room=room)
        elif dir=="right":
          coordinates , room = location_wall2(item=obj,room=room)
      elif door_wall == 2:
        if dir=="front":
          coordinates , room = location_wall4(item=obj,room=room)
        elif dir=="back":
          coordinates , room = location_wall2(item=obj,room=room)
        elif dir=="left":
          coordinates , room = location_wall3(item=obj,room=room)
        elif dir=="right":
          coordinates , room = location_wall1(item=obj,room=room)
      elif door_wall == 4:
        if dir=="front":
          coordinates , room = location_wall2(item=obj,room=room)
        elif dir=="back":
          coordinates , room = location_wall4(item=obj,room=room)
        elif dir=="left":
          coordinates , room = location_wall1(item=obj,room=room)
        elif dir=="right":
          coordinates , room = location_wall3(item=obj,room=room)
    #######################################################################################################3
    elif "corner" in loc:
      if dir=="LR":
        coordinates , room =  location_corners (item=obj ,corner_num=4,room=room,door_wall=refernce_door_wall)
      elif dir=="LL":
        coordinates , room = location_corners (item=obj ,corner_num=1,room=room,door_wall=refernce_door_wall)
      elif dir=="UR":
        coordinates , room = location_corners (item=obj ,corner_num=3,room=room,door_wall=refernce_door_wall)
      elif dir=="UL":
        coordinates , room = location_corners (item=obj ,corner_num=2 ,room=room,door_wall=refernce_door_wall)

    # if len(coordinates)==0 :
    #     coordinates , room = add_item_to_room(item=obj,room=room)
    layout[obj]=coordinates

  ############################################################################################
  ###################################################################################################
  for loc,dir,obj in zip(locations,directions,objects):
    coordinates=[]
    if "chair" in loc or "dc" in loc:
      if len(layout[loc]) !=0:
        chair_location = layout[loc][0]
        if dir=="right":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_chair_direction_right(item=obj,room=room,chair_location=chair_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="left":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_chair_direction_left(item=obj,room=room,chair_location=chair_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="beside" or dir=="next":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_chair_direction_beside_next(item=obj,room=room,chair_location=chair_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        layout[obj]=coordinates
     #########################################################################################################3   
    elif "table" in loc or "closet" in loc : 
      if len(layout[loc])!=0:
        table_location = layout[loc]
        if dir=="right":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_closet_desk_direction_right(item=obj,room=room,item_location=table_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="left":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_closet_desk_direction_left(item=obj,room=room,item_location=table_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="infront":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_desk_direction_infront(item=obj,location_item=loc,room=room,item_location=table_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="behind":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_desk_direction_behind(item=obj,location_item=loc,room=room,item_location=table_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="beside" or dir=="next":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_closet_desk_direction_beside_next(item=obj ,room=room,item_location=table_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="around" or dir=="near":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room =location_table_closet_desk_direction_near(item=obj,location_item=loc,room=room,item_location=table_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)       
        layout[obj]=coordinates

    ######################################################################################################3    
    elif "desk" in loc:
      if len(layout[loc])!=0:
        desk_location = layout[loc]
        if dir=="right":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_closet_desk_direction_right(item=obj,room=room,item_location=desk_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="left":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_closet_desk_direction_left(item=obj,room=room,item_location=desk_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="infront":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_desk_direction_infront(item=obj,location_item=loc,room=room,item_location=desk_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="behind":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_desk_direction_behind(item=obj,location_item=loc,room=room,item_location=desk_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="beside" or dir=="next":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room = location_table_closet_desk_direction_beside_next(item=obj,room=room,item_location=desk_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)
        elif dir=="around" or dir=="near":
          searchstart , searchrange = get_searchstart_searchrange(loc,obj,dir)
          coordinates , room =location_table_closet_desk_direction_near(item=obj,location_item=loc,room=room,item_location=table_location,door_location=refrence_door_location,door_wall=refernce_door_wall,searchstart=searchstart,searchrange=searchrange)       
        layout[obj]=coordinates

  return layout ,room
rows=int(sizeOfRoom[0]/50)
colmns=int(sizeOfRoom[1]/50)
print(x)
print(loca)
print(direc)
print(objects)
room=empty_room(rows,colmns)
# walls=[{"door1":[0,4,6]},{"window1":[1,1,3],"window2":[1,6,8]},{"door2":[2,3,5]},None]
# locations= ["door1", "window1","window2","window1","door1",  "wall4", "wall3","room",  "corner","chair2","door2", "table1", "table2","table2","table3","table3", "table3","desk1"     ,"desk2",     "desk3",  "desk3" ,    "desk4" ,    "desk5"     ]
# directions=["beside","under",  "infront","right",  "infront","right", "front","middle","UR",    "right", "right", "infront","around","around","behind","infront","right",  "behind" ,   "behind" ,   "infront","behind",    "behind" ,   "behind" ]
# objects=   ["chair1","desk1",  "desk2",  "table1", "desk3",  "table2","desk4","table3","chair2","desk5", "table4","chair3", "chair4","chair5","chair6","chair7", "chair8","deskchair1","deskchair2","chair10","deskchair3","deskchair4","deskchair5"]
layout ,room= into_coordinates(walls=x,locations=loca,directions=direc,objects=objects,room=room)
for item in layout:
  print(item ,"= " ,layout[item])
# print_room(room)



[{}, {'window1': [1, 4, 9], 'door1': [1, 4, 9]}, {'window2': [2, 4, 9], 'door2': [2, 4, 9]}, {}]
['room', 'table', 'table', 'table', 'table', 'corner', 'window2', 'desk']
['middle', 'around', 'around', 'around', 'around', 'UL', 'near', 'behind']
['table', 'chair1', 'chair2', 'chair3', 'chair4', 'closet', 'desk', 'dc']
[[4, 9], [5, 9], [6, 9], [7, 9], [8, 9], [9, 9]] 3
table =  [[6, 4], [6, 5]]
chair1 =  [[6, 3]]
chair2 =  [[6, 6]]
chair3 =  [[7, 4]]
chair4 =  [[7, 5]]
closet =  [[13, 9], [13, 8]]
desk =  [[3, 9], [2, 9], [1, 9]]
dc =  [[2, 8]]


In [ ]:
def writeTocsv(layout):
  import csv
  with open('objects.csv','a',newline='') as f:
    spamwriter= csv.writer(f,delimiter=',')
    for i in layout.keys():
      obj=i
      if len(layout[i])==1:
        spamwriter.writerow([obj]+[[layout[i][0][0]+1,None,None]]+[[layout[i][0][1]+1,None,None]])
      elif len(layout[i])==2:
        if layout[i][0][0]== layout[i][1][0]:
          spamwriter.writerow([obj]+[[layout[i][0][0]+1,None,None]]+[[layout[i][0][1]+1,layout[i][1][1]+1,None]])
        else:
          spamwriter.writerow([obj]+[[layout[i][0][0]+1,layout[i][1][0]+1,None]]+[[layout[i][0][1]+1,None,None]])
      else:
        if layout[i][0][0]== layout[i][1][0]:
          spamwriter.writerow([obj]+[[layout[i][0][0]+1,None,None]]+[[layout[i][0][1]+1,layout[i][1][1]+1,layout[i][2][1]+1]])
        else:
          spamwriter.writerow([obj]+[[layout[i][0][0]+1,layout[i][1][0]+1,layout[i][2][0]+1]]+[[layout[i][0][1]+1,None,None]])


In [ ]:
writeTocsv(layout)

In [ ]:
def initializeChromosomes(f,sizeOfRoom):
    import random
    import ast
    heightSectors=int(sizeOfRoom[0]/50)
    widthSectors=int(sizeOfRoom[1]/50)
    chromos=[]
    with open(f,'r',newline='') as ff:
        chromo1={}
        reader=csv.reader(ff,delimiter=',')
        for row in reader:
            
            row1=ast.literal_eval(row[1])
            row2=ast.literal_eval(row[2])
            
            chromo1[row[0]] = [[row1[0],row1[1],row1[2]],[row2[0],row2[1],row2[2]]]
    # print("chromo1",chromo1)
    chromos.append(chromo1)
    for i in range(300):
        chromo={}
        for j in chromo1.keys():
            if 'chair' in j or 'dc' in j:
                chromo[j]=[[random.randrange(1,heightSectors+1),None,None],[random.randrange(1,widthSectors+1),None,None]]
            if 'window' in j or 'door' in j :
                chromo[j] = [[chromo1[j][0][0],chromo1[j][0][1],chromo1[j][0][2]],[chromo1[j][1][0],chromo1[j][1][1],chromo1[j][1][2]]]
            if j=='closet' or j=='table':
                x=random.randrange(1,heightSectors+1)
                if x+1 > heightSectors:
                    
                    x1=random.randint(x-1,x)
                else :
                    if x-1 < 1 :
                        x1=random.randint(x,x+1)
                    else: 
                        n=random.sample([x-1,x+1],1)
                        x1=n[0]
                y=random.randrange(1,widthSectors+1)
                if x1==x :
                    if y+1 > widthSectors:
                        y1=y-1
                    else :
                        if y-1 < 1 :
                            y1=y+1
                        else: 
                            n=random.sample([y-1,y+1],1)
                            y1=n[0]
                    if y+2>= widthSectors :
                        y2=y-2
                    else :
                        y2=y+2
                    x1=None
                else:
                    y1=None
                chromo[j]=[[x,x1,None],[y,y1,None]]
            if j== 'desk' :
                x=random.randrange(1,heightSectors+1)
                if x+1 > heightSectors:
                    
                    x1=random.randint(x-1,x)
                else :
                    if x-1 < 1 :
                        x1=random.randint(x,x+1)
                    else: 
                        n=random.sample([x-1,x+1],1)
                        x1=n[0]
                y=random.randrange(1,widthSectors+1)
                if x1==x :
                    if y+1 > widthSectors:
                        y1=y-1
                    else :
                        if y-1 < 1 :
                            y1=y+1
                        else: 
                            n=random.sample([y-1,y+1],1)
                            y1=n[0]
                    if y+2>= widthSectors :
                        y2=y-2
                    else :
                        y2=y+2
                    x1=None
                    x2=None
                else:
                    y1=None
                    y2=None
                    if x+2>= heightSectors :
                        x2=x-2
                    else:
                        x2=x+2
                chromo[j]=[[x,x1,x2],[y,y1,y2]]
            
        chromos.append(chromo)
    # print(chromos)
    with open(f,'w',newline='') as ff:
        spamwriter=csv.writer(ff,delimiter=',')
        for i in range(len(chromos)):
            spamwriter.writerow(['chromo']+[i])
            for j in chromos[i].keys():
                x=chromos[i][j][0]
                y=chromos[i][j][1]

                spamwriter.writerow([j]+[x]+[y])

In [ ]:
def get_key(val,dic): 
    for key, value in dic.items(): 
         if val == value: 
             return key 

In [ ]:
def accesibleObjects(chromo,sizeOfRoom):
    heightSectors = int(sizeOfRoom[0]/50)
    widthSectors = int(sizeOfRoom[1]/50)
    
    #initializing nodes of graph based on size of the room
    all_nodes = []
    for x in range(1,heightSectors+1):
        for y in range(1,widthSectors+1):
            all_nodes.append([x, y])
    #creating dictionay to map each node to a number
    i=1
    nodes={}
    for n in all_nodes :
        nodes[str(i)] = n
        i=i+1
        
    #when we have multiple door we have to choose only one as a start node to find other objects
    doorCount=0
    for j in chromo:
        if 'door' in j:
            doorCount+=1
    numOfchosenDoor=int(doorCount/2)
    chosenDoor='door'+str(numOfchosenDoor)
    #print('chosen door',chosenDoor)
    # print('nodes are  ', nodes)
    #creating list of object nodes with the starting point node(the door node)
    objectNodes=[]
    startingNode=[]
    for j in chromo:
        #if j != 'door':
        if 'door' not in j:
            objectNodes.append([chromo[j][0][0],chromo[j][1][0]])
            for i in range(1,3) :
                if chromo[j][0][i] is None and chromo[j][1][i] is None :
                    continue
                if chromo[j][0][i] is None and chromo[j][1][i] is not None and chromo[j][0][i-1] is not None: 
                    objectNodes.append([chromo[j][0][i-1],chromo[j][1][i]])
                    continue
                if chromo[j][0][i] is None and chromo[j][1][i] is not None and chromo[j][0][i-2] is not None:
                     objectNodes.append([chromo[j][0][i-2],chromo[j][1][i]])
                     continue
                if chromo[j][0][i] is not None and chromo[j][1][i] is None and chromo[j][1][i-1] is not None: 
                    objectNodes.append([chromo[j][0][i],chromo[j][1][i-1]])
                    continue
                if chromo[j][0][i] is not None and chromo[j][1][i] is None and chromo[j][1][i-2] is not None:
                    objectNodes.append([chromo[j][0][i],chromo[j][1][i-2]])
                    continue
                objectNodes.append([chromo[j][0][i],chromo[j][1][i]])
           
        else:
            if j==chosenDoor:
                for i in range(3) :
                    if chromo[j][0][i] is None and chromo[j][1][i] is None :
                        continue
                    if chromo[j][0][i] is None and chromo[j][1][i] is not None : 
                        startingNode.append([chromo[j][0][i-1],chromo[j][1][i]])
                        continue
                    if chromo[j][0][i] is not None and chromo[j][1][i] is None : 
                        startingNode.append([chromo[j][0][i],chromo[j][1][i-1]])
                        continue
                    startingNode.append([chromo[j][0][i],chromo[j][1][i]]) 
           
    # print('objectNodes ', objectNodes)
    #mapping object nodes to their indecies
    objectNodesIndecies=[]
    for i in objectNodes:
        objectNodesIndecies.append(get_key(i,nodes))
        
    s=get_key(startingNode[0],nodes)
    # print('starting point is ',s, startingNode)
    #declaring the direction that we can move in
    #down, right, up, left, down-left, down-right, up-left, up-right
    dirs = [[1, 0], [0, 1], [-1, 0], [0, -1], [1, -1],[1, 1], [-1, -1], [-1, 1]]
    #generating neighbors only for none-object nodes
    neighbors={}
    for i in nodes:
        result = []
        if nodes.get(i) in objectNodes :
            continue
        for dir in dirs:
            neighbor = [nodes[i][0] + dir[0], nodes[i][1] + dir[1]]
            if neighbor in all_nodes:
                result.append(neighbor)
        neighbors[i]=result
    # print('neighbors',neighbors)
    #generating edges for every node with neighbors
    edges={}
    for i in neighbors:
        valueOfKey=[]
        valueOfKey.append(neighbors.get(i))
        edge=[]
        for j in valueOfKey :
            for k in j:
                edge.append(get_key(k, nodes))
        edges[i]=edge

    #BFS traversal
    visitedNodes=[]
    visited = [False] * (len(all_nodes))
    queue = [] 
    queue.append(s) 
    visited[int(s)-1] = True
    parents=[]
    while queue: 
        s = queue.pop(0) 
        parents.append(s)
        children=edges.get(s)
        # #checking whether this node has children or not
        # if children is not None :
        #     #checking that all its children have children also
        #     for i in children:
        #         #if one child doesn't have children this means it is a node we can only access, but it won't lead to any other node
        #         if edges.get(i) is None:
        #             #so we mark it as visited then we remove it
        #             if i not in visitedNodes:
        #                 visitedNodes.append(i)
        #             children.remove(i)
        #             if visited[int(i)-1]== False:
        #                 visited[int(i)-1] = True
        # else :
        #     break
        # print ('parent s ',s, 'has children ' , children) 
        visitedNodes.append(s)
        #implementing BFS 
        if edges.get(s) is not None:
            for i in edges.get(s):
                if visited[int(i)-1] == False: 
                    queue.append(i) 
                    visited[int(i)-1] = True
    # print('v',visitedNodes)
    #counting how many object nodes have been accessed
    num=0
    numOfObjectNodes=len(objectNodesIndecies)
    #print('v',visitedNodes,'objNodes',objectNodesIndecies,'numofobjectNodes',numOfObjectNodes)
    for n in visitedNodes :
        if n in objectNodesIndecies:
            num=num+1
        
    #print('result of accessible objects', numOfObjectNodes-num)
    
    return numOfObjectNodes-num
    # print('num of object nodes is: ', len(objectNodes),' we accessed ', num, ' nodes')

    # if len(objectNodes) == num :
    #     print('all objects can be accessed from every direction')
    

In [ ]:
#we use it in area function to verify that there is enough space around desk and closet
def twoSectorsOrMoreObjects(objectNodes,objects,heightSectors,widthSectors):
    objectResult=0
    if objectNodes[0][0]==objectNodes[1][0] :
        #horizontal
        horzvert=False
    else:
        #vertical
        horzvert=True
    #when the object is vertical:
    if horzvert:
        rows=[]
        for i in range(len(objectNodes)):
            if objectNodes[i][0] not in rows:
                rows.append(objectNodes[i][0])
        rows=sorted(rows)
        # print(rows)
        first=rows[0]
        last=rows[len(rows)-1]
        objectUpperSector=first-1
        objectLowerSector=last+1
        if objectUpperSector==0:
            objectResult+=5
        else:
            for obj in objects:
                if objectUpperSector in objects[obj]:
                    objectResult+=10
                else:
                    objectResult+=0
        if objectLowerSector==heightSectors+1:
            objectResult+=5
        else:
            for obj in objects:
                if objectLowerSector in objects[obj]:
                    objectResult+=10
                else:
                    objectResult+=0
    else:
        columns=[]
        for i in range(len(objectNodes)):
            if objectNodes[i][1] not in columns:
                columns.append(objectNodes[i][1])
        # print(columns)
        columns=sorted(columns)
        # print('columns: ',columns)
        first=columns[0]
        last=columns[len(columns)-1]
        objectLeftSector=first-1
        objectRightSector=last+1
        if objectLeftSector==0:
            objectResult+=5
        else:
            for obj in objects:
                if objectLeftSector in objects[obj]:
                    objectResult+=10
                else:
                    objectResult+=0
        if objectRightSector==widthSectors+1:
            objectResult+=5
        else:
            for obj in objects:
                if objectRightSector in objects[obj]:
                    objectResult+=10
                else:
                    objectResult+=0
    return horzvert, objectResult

In [ ]:
def extractingObjectsNodes(chromo):
    objects={}
    for j in chromo:
        objectNodes=[]
        objectNodes.append([chromo[j][0][0],chromo[j][1][0]])
        for i in range(1,3) :
            if chromo[j][0][i] is None and chromo[j][1][i] is None :
                continue
            if chromo[j][0][i] is None and chromo[j][1][i] is not None and chromo[j][0][i-1] is not None: 
                objectNodes.append([chromo[j][0][i-1],chromo[j][1][i]])
                continue
            if chromo[j][0][i] is None and chromo[j][1][i] is not None and chromo[j][0][i-2] is not None:
                objectNodes.append([chromo[j][0][i-2],chromo[j][1][i]])
                continue
            if chromo[j][0][i] is not None and chromo[j][1][i] is None and chromo[j][1][i-1] is not None: 
                objectNodes.append([chromo[j][0][i],chromo[j][1][i-1]])
                continue
            if chromo[j][0][i] is not None and chromo[j][1][i] is None and chromo[j][1][i-2] is not None:
                objectNodes.append([chromo[j][0][i],chromo[j][1][i-2]])
                continue
            objectNodes.append([chromo[j][0][i],chromo[j][1][i]])
        objects[j]=objectNodes
    return objects

In [ ]:
def area(chromo,sizeOfRoom):
    heightSectors = int(sizeOfRoom[0]/50)
    widthSectors = int(sizeOfRoom[1]/50)
    objects= extractingObjectsNodes(chromo)
    # print('objects ',objects)

    deskNodes=objects['desk']
    closetNodes=objects['closet']
    tableNodes=objects['table']

    dhv, deskResult=twoSectorsOrMoreObjects(deskNodes,objects,heightSectors,widthSectors)
    chv, closetResult=twoSectorsOrMoreObjects(closetNodes,objects,heightSectors,widthSectors)
    thv, tableResult=twoSectorsOrMoreObjects(tableNodes,objects,heightSectors,widthSectors)

    # chairNodes=objects['chair']
    # chairResult=0
    # if chairNodes[0][0]-1==0:
    #     chairResult-=10
    # else:
    #     chairResult+=10
    # if chairNodes[0][0]+1==heightSectors+1:
    #     chairResult-=10
    # else:
    #     chairResult+=10
    # if chairNodes[0][1]-1==0:
    #     chairResult-=10
    # else:
    #     chairResult+=10
    # if chairNodes[0][1]-1==widthSectors:
    #     chairResult-=10
    # else:
    #     chairResult+=10

    total=deskResult+closetResult+tableResult
    #print('total result of area function is: ',total)

    return total
    # deskResult=0
    # #False is horizantal
    # horzvert=False
    # print(deskNodes)
    
    # print('desk result: ',deskResult)
    # closetNodes=objects['closet']


In [ ]:
def deskChair(chromo,sizeOfRoom):
    lengthSectors=int(sizeOfRoom[0]/50)
    widthSectors=int(sizeOfRoom[1]/50)
    deskChairResult=0
    #to make sure that every chair has a parent
    objects = extractingObjectsNodes(chromo)
    chairNodes= objects['dc']
    chairResult=0
    #find out if the desk is horizontal or vertical acoording to the door
    deskNodes= objects['desk']
    deskcolumns=[]
    deskRows=[]
    for i in range(len(deskNodes)):
        if deskNodes[i][1] not in deskcolumns:
            deskcolumns.append(deskNodes[i][1])
        if deskNodes[i][0] not in deskRows:
            deskRows.append(deskNodes[i][0])
    deskcolumns=sorted(deskcolumns)
    deskRows=sorted(deskRows)
    # print(f'desk columns:{deskcolumns} , desk rows: {deskRows}')
    doorCount=0
    for j in chromo:
        if 'door' in j:
            doorCount+=1
    numOfchosenDoor=int(doorCount/2)
    chosenDoor='door'+str(numOfchosenDoor)
    doorNodes= objects[chosenDoor]
    if deskNodes[0][0]==deskNodes[1][0] :
        #horizontal
        deskhorzvert=False
    else:
        #vertical
        deskhorzvert=True

    if doorNodes[0][0]==1 :
        if not deskhorzvert :
            if chairNodes[0][0]==deskNodes[0][0]+1 :
                if chairNodes[0][1]==deskcolumns[1]:
                    chairResult+=0
                else:
                    chairResult+=5
            else:
                chairResult+=10
        else:
            if deskcolumns[0]>doorNodes[0][1] or deskcolumns[0]==doorNodes[0][1]:
                if chairNodes[0][1]==deskcolumns[0]+1:
                    if chairNodes[0][0]==deskRows[1]:
                        chairResult+=0
                    else:
                        chairResult+=5
                else:
                    chairResult+=10
            else:
                if chairNodes[0][1]==deskcolumns[0]-1:
                    if chairNodes[0][0]==deskRows[1]:
                        chairResult+=0
                    else:
                        chairResult+=5
                else:
                    chairResult+=10

    if doorNodes[0][0]==lengthSectors :
        if not deskhorzvert :
            if chairNodes[0][0]==deskNodes[0][0]-1 :
                if chairNodes[0][1]==deskcolumns[1]:
                    chairResult+=0
                else:
                    chairResult+=5
            else:
                chairResult+=10
        else:
            if deskcolumns[0]>doorNodes[0][1] or deskcolumns[0]==doorNodes[0][1]:
                if chairNodes[0][1]==deskcolumns[0]+1:
                    if chairNodes[0][0]==deskRows[1]:
                        chairResult+=0
                    else:
                        chairResult+=5
                else:
                    chairResult+=10
            else:
                if chairNodes[0][1]==deskcolumns[0]-1:
                    if chairNodes[0][0]==deskRows[1]:
                        chairResult+=0
                    else:
                        chairResult+=5
                else:
                    chairResult+=10

    if doorNodes[0][1]==1:
        if  deskhorzvert:
            if chairNodes[0][1]==deskcolumns[0]+1:
                if chairNodes[0][0]==deskRows[1]:
                    chairResult+=0
                else:
                    chairResult+=5
            else:
                chairResult+=10
        else:
            if deskRows[0]<doorNodes[0][0] or deskRows[0]==doorNodes[0][0]:
                if chairNodes[0][0]==deskRows[0]-1:
                    if chairNodes[0][1]==deskcolumns[1]:
                        chairResult+=0
                    else:
                        chairResult+=5
                else:
                    chairResult+=10
            else:
                if chairNodes[0][0]==deskRows[0]+1:
                    if chairNodes[0][1]==deskcolumns[1]:
                        chairResult+=0
                    else:
                        chairResult+=5
                else:
                    chairResult+=10
    if doorNodes[0][1]==widthSectors:
        if  deskhorzvert:
            if chairNodes[0][1]==deskcolumns[0]-1:
                if chairNodes[0][0]==deskRows[1]:
                    chairResult+=0
                else:
                    chairResult+=5
            else:
                chairResult+=10
        else:
            if deskRows[0]<doorNodes[0][0] or deskRows[0]==doorNodes[0][0]:
                if chairNodes[0][0]==deskRows[0]-1:
                    if chairNodes[0][1]==deskcolumns[1]:
                        chairResult+=0
                    else:
                        chairResult+=5
                else:
                    chairResult+=10
            else:
                if chairNodes[0][0]==deskRows[0]+1:
                    if chairNodes[0][1]==deskcolumns[1]:
                        chairResult+=0
                    else:
                        chairResult+=5
                else:
                    chairResult+=10
    #print('desk chair result is', chairResult)
    return chairResult
    #if the door is located in the last or first row, and the desk is horizontal, then it is orthogonal to it
    # if doorNodes[0][0]==1 or doorNodes[0][0]==lengthSectors :
    #     if not deskhorzvert :
    #         deskisOrthogonaltoDoor=True
    #     else:
    #         deskisOrthogonaltoDoor=False
    
    # if doorNodes[0][1]==1 or doorNodes[0][1]==widthSectors :
    #     if  deskhorzvert :
    #         deskisOrthogonaltoDoor=True
    #     else:
    #         deskisOrthogonaltoDoor=False

    
        

In [ ]:
def objectsIntheDoorPass(chromo,sizeOfRoom):
    widthSectors=int(sizeOfRoom[1]/50)
    lengthSectors=int(sizeOfRoom[0]/50)
    numOfObjects=0
    objects=extractingObjectsNodes(chromo)
    doorCount=0
    for j in chromo:
        if 'door' in j:
            doorCount+=1
    numOfchosenDoor=int(doorCount/2)
    chosenDoor='door'+str(numOfchosenDoor)
    doorNodes= objects[chosenDoor]
    #if door in the first row
    if doorNodes[0][0]==1 :
        #if there is an object directly infront of the door, then the value of the chromo should be increased
        penalty=lengthSectors+1
        for i in range(lengthSectors+1):
            penalty-=1
            for j in objects.keys():
                if [1,i] in objects[j]:
                    numOfObjects+=penalty

    if doorNodes[0][0]==lengthSectors:
        penalty=lengthSectors+1
        for i in range(lengthSectors,1,-1):
            penalty-=1
            for j in objects.keys():
                if [1,i] in objects[j]:
                    numOfObjects+=penalty

    if doorNodes[0][1]==1:
        penalty=widthSectors+1
        for i in range(widthSectors+1):
            penalty-=1
            for j in objects.keys():
                if [1,i] in objects[j]:
                    numOfObjects+=penalty
        
    if doorNodes[0][1]==widthSectors:
        penalty=widthSectors+1
        for i in range(widthSectors,1,-1):
            penalty-=1
            for j in objects.keys():
                if [1,i] in objects[j]:
                    numOfObjects+=penalty
    #print('objects in the door pass',numOfObjects)
    return numOfObjects

In [ ]:
def objectsInfrontOfWindows(chromo):
    objects = extractingObjectsNodes(chromo)
    countOfWindows=0
    result=0
    windowsNodes=[]
    objectsWithoutWindows=[]
    for j in objects:
        if 'window' in j:
            countOfWindows+=1
            obj='window'+str(countOfWindows) 
            windowsNodes.append(objects[obj])
        else:
            #it is ok for a desk chair to be in the same sector of a window
            if 'dc' not in j:
                objectsWithoutWindows.append(objects[j])
    # print(f'windows nodes are {windowsNodes} and other objects are {objectsWithoutWindows}')
    if windowsNodes not in objectsWithoutWindows:
        result+=0
    else:
        result+=10
    #print('objects infront of window', result)
    return result

In [ ]:
def chairs(chromo,sizeOfRoom):
    lengthSectors=int(sizeOfRoom[0]/50)
    widthSectors=int(sizeOfRoom[1]/50)
    
    objects = extractingObjectsNodes(chromo)
    tableNodes=objects['table']
    if tableNodes[0][0]==tableNodes[1][0] :
        #horizontal
        horzvert=False
    else:
        #vertical
        horzvert=True
    tablecolumns=[]
    tableRows=[]
    for i in range(len(tableNodes)):
        if tableNodes[i][1] not in tablecolumns:
            tablecolumns.append(tableNodes[i][1])
        if tableNodes[i][0] not in tableRows:
            tableRows.append(tableNodes[i][0])
    tablecolumns=sorted(tablecolumns)
    tableRows=sorted(tableRows)

    chairResults=0
    tableResult=0
    chairNodes=[]
    countOfChairs=0
    for j in objects:
        if 'chair' in j and j != 'dc':
            countOfChairs+=1
            obj='chair'+str(countOfChairs) 
            chairNodes.append(objects[obj])

    if horzvert:
        #check if there is space around the table
        if tablecolumns[0]+1 <= widthSectors:
            rightDir1=[tableRows[0],tablecolumns[0]+1]
            rightDir2=[tableRows[1],tablecolumns[0]+1]
            if rightDir1 in chairNodes:
                chairResults+=0
            else:
                chairResults+=5
            if rightDir2 in chairNodes:
                chairResults+=0
            else:
                chairResults+=5
        else:
            tableResult+=10

        if tablecolumns[0]-1>= 1 :
            leftDir1=[tableRows[0],tablecolumns[0]-1]
            leftDir2=[tableRows[1],tablecolumns[0]-1]
            if leftDir1 in chairNodes:
                chairResults+=0
            else:
                chairResults+=5
            if leftDir2 in chairNodes:
                chairResults+=0
            else:
                chairResults+=5
        else:
            tableResult+=10
    else:
        if tableRows[0]+1 <= lengthSectors:
            rightDir1=[tableRows[0]+1,tablecolumns[0]]
            rightDir2=[tableRows[0]+1,tablecolumns[1]]
            if rightDir1 in chairNodes:
                chairResults+=0
            else:
                chairResults+=5
            if rightDir2 in chairNodes:
                chairResults+=0
            else:
                chairResults+=5
        else:
            tableResult+=10

        if tableRows[0]-1>= 1 :
            leftDir1=[tableRows[0]-1,tablecolumns[0]]
            leftDir2=[tableRows[0]-1,tablecolumns[1]]
            if leftDir1 in chairNodes:
                chairResults+=0
            else:
                chairResults+=5
            if leftDir2 in chairNodes:
                chairResults+=0
            else:
                chairResults+=5
        else:
            tableResult+=10
    #print('chairs result', chairResults, tableResult)
    return chairResults, tableResult

In [ ]:
def fitness(chromo,sizeOfRoom):
    acc=accesibleObjects(chromo,sizeOfRoom)
    ar=area(chromo,sizeOfRoom)
    de=deskChair(chromo,sizeOfRoom)
    x=objectsIntheDoorPass(chromo,sizeOfRoom)
    y=objectsInfrontOfWindows(chromo)
    z, n=chairs(chromo,sizeOfRoom)
    return acc+ar+de+x+y+z+n

In [ ]:
def get_object_coordinates(random_object_location):
  object_coordinates=[]
  if random_object_location[0][1] == None and random_object_location[1][1] == None :
    object_coordinates.append([random_object_location[0][0],random_object_location[1][0]])
  elif random_object_location[0][1] == None :
    object_coordinates.append([random_object_location[0][0],random_object_location[1][0]])
    object_coordinates.append([random_object_location[0][0],random_object_location[1][1]])
    if random_object_location[1][2] != None:
      object_coordinates.append([random_object_location[0][0],random_object_location[1][2]])
  
  elif random_object_location[1][1] == None :
    object_coordinates.append([random_object_location[0][0],random_object_location[1][0]])
    object_coordinates.append([random_object_location[0][1],random_object_location[1][0]])
    if random_object_location[0][2] != None:
      object_coordinates.append([random_object_location[0][2],random_object_location[1][0]])

  return object_coordinates

In [ ]:
def room_representation (chromo,rows,colmns):
  room = []
  for i in range(rows):
    temp=[]
    for j in range(colmns):
      temp.append(0)
    room.append(temp)
  object_names = list(chromo)
  object_location_in_chromo = list(chromo.values())
  
  for i in range(len(object_location_in_chromo )):
    if "window" in object_names[i] :
      continue
    else :
      object_coordinates=get_object_coordinates(object_location_in_chromo[i])
      for coor in object_coordinates :
        room[coor[0]-1][coor[1]-1] = object_location_in_chromo.index (object_location_in_chromo[i])
      
  return room

In [ ]:
def print_room(room):
  for item in room:
    print(item)

In [ ]:
def print_chromo(chromo):
  for item in chromo :
    print(item,"location is : " ,chromo[item])


In [ ]:
import random 
"""Tournament selection technique.
		   How it works: The algorithm choose randomly five
		   individuals from the population and returns the fittest one"""
def tournament_selection(old_population ,pop_fitness, k= 30):
  #k is the number of chromos selected randomly in each round ( which is now 6 )
  
  #when we combine this with the fitness function , we call it so we can calculate the fitness of the passed popluation  
  # population={0:34,1:45,2:32,3:55,4:12,5:120,6:3,7:33,8:14,9:21} #just an example to see results
  # population = {}
  # for chromo in old_population :
  #   population[old_population.index(chromo)]=fitness(chromo,room_dimensions)
  
  
  #population is a dictionary : key is a number indicating the index of the chromo in the population , value is the fitness of that chromo
  #population = fitness(old_population)
  # 70% of the new generation is selected using tournament selection 
  population = pop_fitness.copy()
  selection_size=int(len(population)*0.7) #we set the selection sive to foruty percent because after few iterations there ont be enough chromos to start a new round but originally it will be (*0.7)
  
  selected_chromos_indexes = []
  selected_chromos=[]
  for i in range (selection_size) :
    #random_chromos is a dictionary will store the randomly selected chromos in each round (key->index of chromo , value -> fitness of chromo)
    random_chromos={}
    j=0
    while j < k :
      #we need to generate six random indexes to select them from the population 
      index=random.choice(list(population.keys()))
      #if the generated number doesn't exsit in the keys of random_chromos then this chromo has not benn selected yet and we take it 
      if index not in list(random_chromos.keys() ):
        random_chromos[index]=population[index] 
        j+=1
    
    #after we select the chromos for this round we order them in a descending order to select the first chromo with the highest fitness
    #round_sort_chromos is a list of tuples each tuples contains tow items the first one is the index of the chromo the second is its fitness
    round_sort_chromos = sorted(random_chromos.items(), key=lambda x: x[1])#reverse=True (descending)
    
    #add the index of the chosen chromo in this round 
    selected_chromos_indexes.append(round_sort_chromos[0][0])

    #delete this chromo we selected from the old population so it's not selected again 
    del population[round_sort_chromos[0][0]]
  # print(selected_chromos_indexes)
  for i in selected_chromos_indexes:
    selected_chromos.append(old_population[i])
  return selected_chromos

In [ ]:
import random
def change_object_location (chromo,rows,colmns,random_object=""):
  #if random_object is passed to this function then the loction of the passed object will be changed
  #if random_object is not passed (empty string) then the function will randomly choose an object from the objects in the passed chromo and search 
  #the space surrounding it and place it in the first empty space found

  object_names_in_chromo=list(chromo) #list contains the objects in the passed chromo
  object_location_in_chromo = list(chromo.values())#list of lists each list is an object location in the passed chromo

  if random_object =="":
    #choosing a random object
    random_object = random.choice(object_names_in_chromo)
    while True :
      #checking if the randomly selected object is a window or a door , if so we will keep choosing until the selected object is no longer one of them
      #because the location of windows and doors acn not be changed 
      if "window" in random_object or "door" in random_object :
        random_object = random.choice(object_names_in_chromo)
      else :
        break
  
  # print("random object is :",random_object)
  #show the room before changing any thing  
  room = room_representation(chromo,rows,colmns)
  
  #generating the coordinates of the new location for the randoomly selected object
  x=random.randint(1,rows)
  y=random.randint(1,colmns)
  new_object_coordinates = (x,y)
  # print(x , " ", y)

  while True :
    #(-1) because the room indexes start from zero
    r=new_object_coordinates[0]-1
    c=new_object_coordinates[1]-1
    #here we are checking if the randomly generated location is not empty , if so we will keep generating new coordinates until we find an empty one
    if room[r][c]!=0:
      x=random.randint(1,rows)
      y=random.randint(1,colmns)
      new_object_coordinates = (x,y)
    else :
      break 

  #if the randomly selected object is a chair or a deskChair this means it takes only one section and we can put it immediately  in that location
  if ("chair" in random_object or  "dc" in random_object) :
    chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1],None,None]]

  #if the randomly selected object is a table or a closet this means it takes two sections and we have to search the space surrounding that location
  #since the object takes only two sections we search for another one empty location 
  #we will search for that empty location in four directions : up , down , left , right
  if "closet" in random_object or "table" in random_object : 
    up=[None,None]
    down=[None,None]
    right=[None,None]
    left=[None,None]

    r=new_object_coordinates[0]-1
    c=new_object_coordinates[1]-1
    
    #the values stored in up (or down , right , left ) , will help us know if the current location is empty
    #if the two values are zeroes this means the location is empty and we can change the object location to that location , if not this means that
    #location is not empty and we move on to the 
    if r-1 > -1 :
      #the condition (r-1 > -1) is when the row value is zero , if so we can't search in up direction 
      up[0] = room[r-1][c]
      up[1]= room[r][c]
      #check if the location is empty
      if up[0]==0and up[1]==0 :
        chromo[random_object]=[[new_object_coordinates[0]-1,new_object_coordinates[0],None],[new_object_coordinates[1],None,None]]

    if r+1 != rows :
      #the condition (r+1 != rows) is when the row value is the value of the last row , if so we can't search in down direction 
      down[0] = room[r][c]
      down[1]= room[r+1][c]
      #check if the location is empty
      if down[0]==0and down[1]==0 :
        chromo[random_object]=[[new_object_coordinates[0],new_object_coordinates[0]+1,None],[new_object_coordinates[1],None,None]]

    if c+1 != colmns :
      #the condition (c+1 != colmns) is when the colmn value is the value of the last colmn , if so we can't search in right direction 
      right[0] =room[r][c]
      right[1]=room[r][c+1]
      #check if the location is empty
      if right[0]==0 and right[1]==0:
        chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1],new_object_coordinates[1]+1,None]]

    if c-1 >-1:
      #the condition (c-1 >-1) is when the colmn value is zero , if so we can't search in left direction 
      left[0]=room[r][c-1]
      left[1]=room[r][c]
      #check if the location is empty
      if left[0]==0and left[1]==0 :
        chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1]-1,new_object_coordinates[1],None]]
    


  #if the randomly selected object is a desk this means it takes three sections and we have to search the space surrounding that location
  #we will search for that empty location in two directions : verticaly and horizontaly
  #since the object takes three sections we search for another two empty locations 
  #when we search in the vertical direction we look at the location above the one we genertated randomly and the location below it
  #when we search in the horizontal direction we look at the location on the right of the one we genertated randomly and the location on the left of it
  if "desk" in random_object : 
    vertical =[None,None,None]  
    horizontal=[None,None,None] 
    #the values stored in vertical (or horizontal) , will help us know if the current location is empty
    #if all values are zeroes this means the location is empty and we can change the object location to that location , if not this means that
    #location is not empty and we move on to the 
    r=new_object_coordinates[0]-1
    c=new_object_coordinates[1]-1
    
    #if the row value is zero (we can't search above the randomly generated location so we have to look for two locations below it )
    if r==0:
      #if the colmn value is zero (we can't search to the left of the randomly generated location so we have to look for two locations on its right it )
      if c ==0 :
        vertical[0] = room[r][c]
        vertical[1] = room[r+1][c]
        vertical[2] = room[r+2][c]
        horizontal[0] = room[r][c]
        horizontal[1] = room[r][c+1]
        horizontal[2] = room[r][c+2]
        #checking the vertical and horizontal directions and place the object in the first empty location we find
        if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],new_object_coordinates[0]+1,new_object_coordinates[0]+2],[new_object_coordinates[1],None,None]]
        elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1],new_object_coordinates[1]+1,new_object_coordinates[1]+2]]

      #if the colmn value is the value of the last colmn (we can't search to the right of the randomly generated location so we have to look 
      #for two locations on its left it )
      elif c == colmns-1 :
        vertical[0] = room[r][c]
        vertical[1] = room[r+1][c]
        vertical[2] = room[r+2][c]
        horizontal[0] = room[r][c-2]
        horizontal[1] = room[r][c-1]
        horizontal[2] = room[r][c]

        if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],new_object_coordinates[0]+1,new_object_coordinates[0]+2],[new_object_coordinates[1],None,None]]
        elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1]-2,new_object_coordinates[1]-1,new_object_coordinates[1]]]
      
      #if the colmn value is not zero or value of the last colmn (we can search to the right of the randomly generated location and its left)
      elif 0 < c < colmns-1 :
        vertical[0] = room[r][c]
        vertical[1] = room[r+1][c]
        vertical[2] = room[r+2][c]
        horizontal[0] = room[r][c-1]
        horizontal[1] = room[r][c]
        horizontal[2] = room[r][c+1]

        if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],new_object_coordinates[0]+1,new_object_coordinates[0]+2],[new_object_coordinates[1],None,None]]
        elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1]-1,new_object_coordinates[1],new_object_coordinates[1]+1]]
    
    #if the row value is the value of the last row (we can't search below the randomly generated location so we have to look for two locations above it )
    if r == rows-1:
      #if the colmn value is zero (we can't search to the left of the randomly generated location so we have to look for two locations on its right it )
      if c==0 :
        vertical[0] = room[r-2][c]
        vertical[1] = room[r-1][c]
        vertical[2] = room[r][c]
        horizontal[0] = room[r][c]
        horizontal[1] = room[r][c+1]
        horizontal[2] = room[r][c+2]
        
        if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0]-2,new_object_coordinates[0]-1,new_object_coordinates[0]],[new_object_coordinates[1],None,None]]
        elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1],new_object_coordinates[1]+1,new_object_coordinates[1]+2]]
      
      #if the colmn value is the value of the last colmn (we can't search to the right of the randomly generated location so we have to look 
      #for two locations on its left it )
      elif c== colmns-1 :
        vertical[0] = room[r-2][c]
        vertical[1] = room[r-1][c]
        vertical[2] = room[r][c]
        horizontal[0] = room[r][c-2]
        horizontal[1] = room[r][c-1]
        horizontal[2] = room[r][c]

        if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0]-2,new_object_coordinates[0]-1,new_object_coordinates[0]],[new_object_coordinates[1],None,None]]
        elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1]-2,new_object_coordinates[1]-1,new_object_coordinates[1]]]
      
      #if the colmn value is not zero or value of the last colmn (we can search to the right of the randomly generated location and its left)
      elif 0<c<colmns-1 :
        vertical[0] = room[r-2][c]
        vertical[1] = room[r-1][c]
        vertical[2] = room[r][c]
        horizontal[0] = room[r][c-1]
        horizontal[1] = room[r][c]
        horizontal[2] = room[r][c+1]

        if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0]-2,new_object_coordinates[0]-1,new_object_coordinates[0]],[new_object_coordinates[1],None,None]]
        elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
          chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1]-1,new_object_coordinates[1],new_object_coordinates[1]+1]]

    #if the row value is not zero or value of the last row (we can search above the randomly generated location and below it )
    #and when colmn value is zero (we can't search to the left of the randomly generated location so we have to look for two locations on its right it ) 
    elif 0 < r < rows-1 and c == 0:
      vertical[0] = room[r-1][c]
      vertical[1] = room[r][c]
      vertical[2] = room[r+1][c]
      horizontal[0] = room[r][c]
      horizontal[1] = room[r][c+1]
      horizontal[2] = room[r][c+2]

      if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
        chromo[random_object]=[[new_object_coordinates[0]-1,new_object_coordinates[0],new_object_coordinates[0]+1],[new_object_coordinates[1],None,None]]
      elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
        chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1],new_object_coordinates[1]+1,new_object_coordinates[1]+2]]
    
    #if the row value is not zero or value of the last row (we can search above the randomly generated location and below it )
    #if the colmn value is the value of the last colmn (we can't search to the right of the randomly generated location so we have to look 
    #for two locations on its left it )
    elif 0 < r < rows-1 and c == colmns-1:
      vertical[0] = room[r-1][c]
      vertical[1] = room[r][c]
      vertical[2] = room[r+1][c]
      horizontal[0] = room[r][c-2]
      horizontal[1] = room[r][c-1]
      horizontal[2] = room[r][c]

      if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
        chromo[random_object]=[[new_object_coordinates[0]-1,new_object_coordinates[0],new_object_coordinates[0]+1],[new_object_coordinates[1],None,None]]
      elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
        chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1]-2,new_object_coordinates[1]-1,new_object_coordinates[1]]]
    
    #if the row value is not zero or value of the last row (we can search above the randomly generated location and below it )
    #if the colmn value is not zero or value of the last colmn (we can search to the right of the randomly generated location and its left)
    elif 0 < r < rows-1 and  0 < c < colmns-1 :
      vertical[0] = room[r-1][c]
      vertical[1] = room[r][c]
      vertical[2] = room[r+1][c]
      horizontal[0] = room[r][c-1]
      horizontal[1] = room[r][c]
      horizontal[2] = room[r][c+1]

      if vertical[0]==0 and vertical[1]==0 and vertical[2]==0 :
        chromo[random_object]=[[new_object_coordinates[0]-1,new_object_coordinates[0],new_object_coordinates[0]+1],[new_object_coordinates[1],None,None]]
      elif horizontal[0]==0 and horizontal[1]==0 and horizontal[2]==0 :
        chromo[random_object]=[[new_object_coordinates[0],None,None],[new_object_coordinates[1]-1,new_object_coordinates[1],new_object_coordinates[1]+1]]

  return chromo 

In [ ]:
def cross_over (rows,colmns,old_population, seventy_percent_of_the_new_population):
  # the remaining thirty percent of the new population will be generated by the crossover between two randomly selected parents from seventy_percent_of_the_new_population
  # we passed the old_population so we can find out the number of childern we need to produce in order to mainntain the populaion size
  number_of_childern_to_produce = int(len(old_population)*0.3)

  #these two parents are randomly selected from the chromos we selected by applying the tournament_selection function
  #each two randomly selected parents will give one child
  childern=[]
  

  #seventy_percent_of_the_new_population is a list of dictionaries each dictionary represents a chromo ( or parent ) from whish we will choose two 
  #parents randomly
  for i in range(number_of_childern_to_produce):
    new_chromo={}
    #selecting two random parents
    parents = random.choices(seventy_percent_of_the_new_population,k=2)

    #just checking if the randomly selected parents are not the same if so we will choose another two random parents and keep 
    #choosing until they are not the same
    while True :
      if parents[0] == parents[1] :
        parents = random.choices(seventy_percent_of_the_new_population,k=2)
      else :
        break 

    #the delete function affects the origional population that's why we take a copy of them
    parent1 = parents[0].copy()
    parent2 = parents[1].copy()

    #at first the locations of windows and doors in the room is fixed (can't be changed) in all chromos (parents and children) so windows and doors
    #and their location are passed to the child (this is not part of the cross over)
    # print("before cross over")
    # print(parents[0])
    # print(parents[1])

    # objects = list(parents[0])
    objects = list(parent1)
    for item in objects :
      if "window" in item or "door" in item :
        # new_chromo[item]=parents[0][item]
        new_chromo[item]=parent1[item]
        #then we delete the windows and doors from those parents 
        # print("while deleting")
        # print(parents[0])
        # print(parents[1])

        # parents[0].pop(item)
        # parents[1].pop(item)
        parent1.pop(item)
        parent2.pop(item)
    

    #the crossover : 
    #now we will take the first half of the first parent and put in the new chromosome , for the second parent we will add objects
    #from its second half considering that the objects we take from the second parent will not cause an intersection with any object in the new
    #chromo ( which are taken from the first half of the first parent ) , if such object exist in the second half of the second parent we will
    #change its location so that chromo length stayes the same 

    # half_size_of_first_parent = int(len(parents[0])/2)
    half_size_of_first_parent = int(len(parent1)/2)

    
    #we will take half of the objects in the first parent and put immediately in the new child
    
    # object_names_in_first_parent = list(parents[0])
    # object_locations_in_first_parent = list(parents[0].values())
    object_names_in_first_parent = list(parent1)
    object_locations_in_first_parent = list(parent1.values())

    for i in range(half_size_of_first_parent):
      new_chromo[object_names_in_first_parent[i]]=object_locations_in_first_parent[i]
    
    # print("half")
    # print_chromo(new_chromo)
   
    object_names_in_new_chromo = list(new_chromo) 
    object_locations_in_new_chromo = list(new_chromo.values()) 

    #after that we will loop over the objects in the second half of the second parent and compare the location of that object with all the locations
    #of all objects that are currently in the new child , if that object ( from the second parent ) causes intersection with an object exists in the 
    #new child we will change its locaton , if it doesn't cause any intersection with any object of the objects in the new child it will be added to that child
    
    # half_size_of_second_parent = int(len(parents[1])/2)
    half_size_of_second_parent = int(len(parent2)/2)


    # object_names_in_second_parent = list(parents[1]) 
    # object_locations_in_second_parent = list(parents[1].values())
    object_names_in_second_parent = list(parent1) 
    object_locations_in_second_parent = list(parent1.values())

    x=len(new_chromo)
    # for i in range (half_size_of_second_parent,len(parents[1]) ):
    for i in range (half_size_of_second_parent,len(parent1) ):
      #object_not_found will determine if the selected object from the second parent caused an intersction or not , at first we consider that the
      #object hasn't caused any intersection ( value True ) 
      #after the comparsion with the objects in the new child if the value of object_not_found is False it means the selected object from the second
      #parent caused an intersection and we won't take it 
      object_not_found = True

      for j in range(x) :
        #object_locations_in_new_chromo is a list each element in it is a list representing the location of an object in the child 
        #each location (element) in object_locations_in_new_chromo contains two lists the first has the row values and the second has the colmn values
        #object_locations_in_new_chromo[j][k][l] :
        #index j gives us the location of current object (object_locations_in_new_chromo[j] shape is [[,,],[,,]])
        #index k : if its zero then we have row values of that object (object_locations_in_new_chromo[j][0] shape is [,,])
        #          index l : specifies one row value of object_locations_in_new_chromo[j][0] (object_locations_in_new_chromo[j][0][l] is an integer)
        #          if its one then we have colmn values of that object (object_locations_in_new_chromo[j][1] shape is [,,])
        #          index l : specifies one colmn value of object_locations_in_new_chromo[j][1] (object_locations_in_new_chromo[j][1][l] is an integer)
        
        # we check if any row value from the second parent exist in the row values from the current child's object , if so we have to check the 
        #colmn values , if there is a shared colmn value between them then that object caused an intersection and we change its location
        if (object_locations_in_second_parent[i][0][0] in object_locations_in_new_chromo[j][0]) or ( object_locations_in_second_parent[i][0][1] in object_locations_in_new_chromo[j][0] and object_locations_in_second_parent[i][0][1]!=None)or ( object_locations_in_second_parent[i][0][2] in object_locations_in_new_chromo[j][0] and object_locations_in_second_parent[i][0][2]!=None ):                     
          #we check if any colmn value from the second parent exist in the colmn values from the current child's object , if such value exist then
          #this object caused an intersection and we will change its location
          if (object_locations_in_second_parent[i][1][0] in object_locations_in_new_chromo[j][1]) or ( object_locations_in_second_parent[i][1][1] in object_locations_in_new_chromo[j][1] and object_locations_in_second_parent[i][1][1]!=None)or ( object_locations_in_second_parent[i][1][2] in object_locations_in_new_chromo[j][1] and object_locations_in_second_parent[i][1][2]!=None ):             
            new_chromo=change_object_location(new_chromo,rows,colmns,object_names_in_second_parent[i])
            #the change_object_location function will place the object without making any intersections with the objects in the passed chromo
            #so there is no need to continue comparing its location after it has been changed 
            break
          #if they don't share any colmn value then we add the object to the new chromo
          else :
            new_chromo[object_names_in_second_parent[i]]=object_locations_in_second_parent[i]
                       
        # we check if any colmn value from the second parent exist in the colmn values from the current child's object , if so we have to check the 
        #row values , if there is a shared row value between them then that object caused an intersection and we change its location
        elif (object_locations_in_second_parent[i][1][0] in object_locations_in_new_chromo[j][1]) or ( object_locations_in_second_parent[i][1][1] in object_locations_in_new_chromo[j][1] and object_locations_in_second_parent[i][1][1]!=None)or ( object_locations_in_second_parent[i][1][2] in object_locations_in_new_chromo[j][1] and object_locations_in_second_parent[i][1][2]!=None ):                     
          #we check if any row value from the second parent exist in the row values from the current child's object , if such value exist then
          #this object caused an intersection and we will change its location
          if (object_locations_in_second_parent[i][0][0] in object_locations_in_new_chromo[j][0]) or ( object_locations_in_second_parent[i][0][1] in object_locations_in_new_chromo[j][0] and object_locations_in_second_parent[i][0][1]!=None)or ( object_locations_in_second_parent[i][0][2] in object_locations_in_new_chromo[j][0] and object_locations_in_second_parent[i][0][2]!=None ):                       
            new_chromo=change_object_location(new_chromo,rows,colmns,object_names_in_second_parent[i])
            #the change_object_location function will place the object without making any intersections with the objects in the passed chromo
            #so there is no need to continue comparing its location after it has been changed 
            break
          #if they don't share any row value then we add the object to the new chromo
          else :
            new_chromo[object_names_in_second_parent[i]]=object_locations_in_second_parent[i]
        #if the compared objects locations have completely diffrent
        elif object_locations_in_new_chromo[j][0][0]!=object_locations_in_second_parent[i][0][0] and object_locations_in_new_chromo[j][1][0]!=object_locations_in_second_parent[i][1][0] :
          new_chromo[object_names_in_second_parent[i]]=object_locations_in_second_parent[i]


    # print("new chromo : ")
    # print_chromo(new_chromo)
    #when we finish looping over the second half of the second parent the child is now complete and we add it the childern list 
    childern.append(new_chromo)
  return childern

In [ ]:
import random 
def change_object_orientation (chromo,rows,colmns):
  #this function will change the orientation of objects that only take more than one section in the room
  #the object is selected randomly , if its position vertical it will become horizontal and if its horizontal it will becom vertical
  #if the space around the selected objected is occupied with another objects then the object's orientation will not be changed

  object_names_in_chromo=list(chromo)#list contains the objects in the passed chromo
  object_location_in_chromo = list(chromo.values())#list of lists each list is an object location in the passed chromo

  random_object = random.choice(object_names_in_chromo)

  while True :
    #each item mentioned in this if statement takes one section , so here we are just checking if the randomly selected objesst is among them 
    #if so we keep choosing until its not one of these objects
    if "window" in random_object or "door" in random_object or "chair" in random_object or "dc" in random_object :
      random_object = random.choice(object_names_in_chromo)
    else :
      break
  # print("random_object : ",random_object)
  

  random_object_location = chromo[random_object]
  
  object_coordinates = get_object_coordinates(random_object_location)

  room = room_representation(chromo,rows,colmns)

  if "closet" in random_object or "table" in random_object : 
    #if the chosen object is one them then it takes two sections
    #if the selected object position vertical (colmn coordinates are the same in both sections ) then we search two sections above its location and 
    #two sections below its location
    #if the selected object position horizontal (row coordinates are the same in both sections ) then we search two sections right to its location and 
    #two sections left to its location

    # |0,0|  ,|0,b,0|
    # |b,b|  ,|0,b,0|
    # |0,0|
    # the shape above to illustrate the possible orientation of an object that takes two sections
    # spaces_around_object[0] will store the value of upper left section
    # spaces_around_object[1] will store the value of upper right section
    # spaces_around_object[2] will store the value of lower left section
    # spaces_around_object[3] will store the value of lower right section

    spaces_around_object=[None,None,None,None]
    new_location =[]
    #if the original position of the object is horizontal(row values are the same)
    if object_coordinates[0][0]==object_coordinates[1][0] :
      #if the object in first row (we can only search below its location )
      if object_coordinates[0][0]==1:
        spaces_around_object[2]=(object_coordinates[0][0]+1,object_coordinates[0][1])
        spaces_around_object[3]=(object_coordinates[1][0]+1,object_coordinates[1][1])
        #change the object's orientation to the first empty section 
        if room[spaces_around_object[2][0]-1][spaces_around_object[2][1]-1]==0:
          new_location=[[object_coordinates[0][0],object_coordinates[0][0]+1,None],[object_coordinates[0][1],None,None]]
        elif room[spaces_around_object[3][0]-1][spaces_around_object[3][1]-1]==0:
          new_location=[[object_coordinates[1][0],object_coordinates[1][0]+1,None],[object_coordinates[1][1],None,None]]
      #if the object in last row ( we can only search above it )
      elif object_coordinates[0][0]==rows:
        spaces_around_object[0]=(object_coordinates[0][0]-1,object_coordinates[0][1])
        spaces_around_object[1]=(object_coordinates[1][0]-1,object_coordinates[1][1])

        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0:
          new_location=[[object_coordinates[0][0]-1,object_coordinates[0][0],None],[object_coordinates[0][1],None,None]]
        elif room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[1][0]-1,object_coordinates[1][0],None],[object_coordinates[1][1],None,None]]
      #if the row value is not zero or the value of the last row we can search above and below the location 
      elif 1<object_coordinates[0][0]<rows:
        spaces_around_object[0]=(object_coordinates[0][0]-1,object_coordinates[0][1])
        spaces_around_object[1]=(object_coordinates[1][0]-1,object_coordinates[1][1])
        spaces_around_object[2]=(object_coordinates[0][0]+1,object_coordinates[0][1])
        spaces_around_object[3]=(object_coordinates[1][0]+1,object_coordinates[1][1])
        #(-1) because indexes in room starts from 0
        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0:
          new_location=[[object_coordinates[0][0]-1,object_coordinates[0][0],None],[object_coordinates[0][1],None,None]]
        elif room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[1][0]-1,object_coordinates[1][0],None],[object_coordinates[1][1],None,None]]
        elif room[spaces_around_object[2][0]-1][spaces_around_object[2][1]-1]==0:
          new_location=[[object_coordinates[0][0],object_coordinates[0][0]+1,None],[object_coordinates[0][1],None,None]]
        elif room[spaces_around_object[3][0]-1][spaces_around_object[3][1]-1]==0:
          new_location=[[object_coordinates[1][0],object_coordinates[1][0]+1,None],[object_coordinates[1][1],None,None]]
    
    #if the original position of the object is vertical (colmn values are the same)
    elif object_coordinates[0][1]==object_coordinates[1][1] :
      #if the object in first colmn (we can only search right to its location )
      if object_coordinates[1][1]==1:
        spaces_around_object[1]=(object_coordinates[0][0],object_coordinates[1][1]+1)
        spaces_around_object[3]=(object_coordinates[1][0],object_coordinates[1][1]+1)

        if room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[0][0],None,None],[object_coordinates[0][1],object_coordinates[0][1]+1,None]]
        elif room[spaces_around_object[3][0]-1][spaces_around_object[3][1]-1]==0:
          new_location=[[object_coordinates[1][0],None,None],[object_coordinates[1][1],object_coordinates[1][1]+1,None]]
      #if the object in last colmn (we can only search left to its location )
      elif object_coordinates[1][1]==colmns:
        spaces_around_object[0]=(object_coordinates[0][0],object_coordinates[0][1]-1)
        spaces_around_object[2]=(object_coordinates[1][0],object_coordinates[0][1]-1)

        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0:
          new_location=[[object_coordinates[0][0],None,None],[object_coordinates[0][1]-1,object_coordinates[0][1],None]]
      
      #if the colmn value is not the first colmn or the value of the last colmn we can search right and left to its location 
      elif 1<object_coordinates[1][1]<colmns:
        spaces_around_object[0]=(object_coordinates[0][0],object_coordinates[0][1]-1)
        spaces_around_object[1]=(object_coordinates[0][0],object_coordinates[1][1]+1)
        spaces_around_object[2]=(object_coordinates[1][0],object_coordinates[0][1]-1)
        spaces_around_object[3]=(object_coordinates[1][0],object_coordinates[1][1]+1)
      
        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0:
          new_location=[[object_coordinates[0][0],None,None],[object_coordinates[0][1]-1,object_coordinates[0][1],None]]
        elif room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[0][0],None,None],[object_coordinates[0][1],object_coordinates[0][1]+1,None]]
        elif room[spaces_around_object[2][0]-1][spaces_around_object[2][1]-1]==0:
          new_location=[[object_coordinates[1][0],None,None],[object_coordinates[1][1]-1,object_coordinates[1][1],None]]
        elif room[spaces_around_object[3][0]-1][spaces_around_object[3][1]-1]==0:
          new_location=[[object_coordinates[1][0],None,None],[object_coordinates[1][1],object_coordinates[1][1]+1,None]]
    
    #when len(new_location)=0 that means the space around the object is not empty and it can't be moved so we dont change its orientation
    #if its length not 0 then its orientation can be changed to be new_location
    #if the space around the object is not empty then the variable new_location will stay an empty list , so without this condition the new object's
    #location will be an empty list
    if len(new_location)!=0:
      chromo[random_object]=new_location

  if "desk" in random_object : 
    #if the chosen object is a desk then it takes three sections
    #if the selected object position vertical (colmn coordinates are the same in all sections ) we will search the sections denoted in the letter S :
    #|0,B,0|
    #|S,B,S|
    #|0,B,0|
    # spaces_around_object[0] will store the value of left section
    # spaces_around_object[1] will store the value of right section

    #if the selected object position horizontal (row coordinates are the same in all sections ) we will search the sections denoted in the letter S :
    #|0,S,0|
    #|B,B,B|
    #|0,S,0|
    # spaces_around_object[0] will store the value of upper section
    # spaces_around_object[1] will store the value of upper section

    spaces_around_object=[None,None]
    new_location =[]
    #if the original position of the object is horizantol ( row values are all the same )
    if object_coordinates[0][0]==object_coordinates[1][0]==object_coordinates[2][0]:
      #if object in first row ( we cant search above it so we search two sections below it)
      #|B,B,B|
      #|0,S,0|
      #|0,S,0|
      if object_coordinates[0][0]== 1:
        spaces_around_object[0]=(object_coordinates[1][0]+1,object_coordinates[1][1])
        spaces_around_object[1]=(object_coordinates[1][0]+2,object_coordinates[1][1])
        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0 and room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[1][0],object_coordinates[1][0]+1,object_coordinates[1][0]+2],[object_coordinates[1][1],None,None]]
      #if object in last row ( we cant search below it so we search two sections above it)
      #|0,S,0|
      #|0,S,0|
      #|B,B,B|
      elif object_coordinates[0][0]== rows:
        spaces_around_object[0]=(object_coordinates[1][0]-2,object_coordinates[1][1])
        spaces_around_object[1]=(object_coordinates[1][0]-1,object_coordinates[1][1])
        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0 and room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[1][0]-2,object_coordinates[1][0]-1,object_coordinates[1][0]],[object_coordinates[1][1],None,None]]
      #if the object is not in the first or last row we serch one section above it and below it
      #|0,S,0|
      #|B,B,B|
      #|0,S,0|
      elif 1<object_coordinates[0][0]<rows:
        spaces_around_object[0]=(object_coordinates[1][0]-1,object_coordinates[1][1])
        spaces_around_object[1]=(object_coordinates[1][0]+1,object_coordinates[1][1])
        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0 and room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[1][0]-1,object_coordinates[1][0],object_coordinates[1][0]+1],[object_coordinates[1][1],None,None]]
    
    #if the original position of the object is vertical
    elif object_coordinates[0][1]==object_coordinates[1][1]==object_coordinates[2][1]:
      #if object in first colmn ( we cant search to its left so we search two sections to its right)
      #|B,0,0|
      #|B,S,S|
      #|B,0,0|
      if object_coordinates[0][1]==1:
        spaces_around_object[0]=(object_coordinates[1][0],object_coordinates[1][1]+1)
        spaces_around_object[1]=(object_coordinates[1][0],object_coordinates[1][1]+2)
        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0 and room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[1][0],None,None],[object_coordinates[1][1],object_coordinates[1][1]+1,object_coordinates[1][1]+2]]
      #if object in last colmn ( we cant search to its right so we search two sections to its left)
      #|0,0,B|
      #|S,S,B|
      #|0,0,B|
      if object_coordinates[0][1]==colmns:
        spaces_around_object[0]=(object_coordinates[1][0],object_coordinates[1][1]-2)
        spaces_around_object[1]=(object_coordinates[1][0],object_coordinates[1][1]-1)
        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0 and room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[1][0],None,None],[object_coordinates[1][1]-2,object_coordinates[1][1]-1,object_coordinates[1][1]]]
      #if object not in first or last colmn ( we search one section to its right and one section to its left )
      #|0,B,0|
      #|S,B,S|
      #|0,B,0|
      elif 1<object_coordinates[0][1]<colmns:
        spaces_around_object[0]=(object_coordinates[1][0],object_coordinates[1][1]-1)
        spaces_around_object[1]=(object_coordinates[1][0],object_coordinates[1][1]+1)
        if room[spaces_around_object[0][0]-1][spaces_around_object[0][1]-1]==0 and room[spaces_around_object[1][0]-1][spaces_around_object[1][1]-1]==0:
          new_location=[[object_coordinates[1][0],None,None],[object_coordinates[1][1]-1,object_coordinates[1][1],object_coordinates[1][1]+1]]
    
    #when len(new_location) that means the space around the object is not empty and it can't be moved
    if len(new_location)!=0:
      chromo[random_object]=new_location
  return chromo 



In [ ]:
def mutation (population,rows,colmns):
  number_of_randomly_selected_chromos = int(len(population)*0.7)
  for i in range (number_of_randomly_selected_chromos):
    randomly_selected_chromo = random.choice(population).copy()

    randomly_selected_chromo_index=population.index(randomly_selected_chromo)
    ratio=random.random()
    if ratio >0.5 :
      temp=randomly_selected_chromo.copy()
      new_chromo=change_object_location(randomly_selected_chromo,rows,colmns)
      while True :
        if temp == new_chromo :
          new_chromo=change_object_location(randomly_selected_chromo,rows,colmns)
        else :
          break
    else:
      new_chromo = change_object_orientation(randomly_selected_chromo,rows,colmns)
    # else:
    #     new_chromo=place_dc_behind_desk (chromo,rows,colmns,door_wall)
    population[randomly_selected_chromo_index]=new_chromo

  return population



In [ ]:
initializeChromosomes('objects.csv',sizeOfRoom)

In [ ]:
import ast
chromos=[]
chromo={}
with open('objects.csv','r',newline='') as ff:
    
    reader=csv.reader(ff,delimiter=',')
    for row in reader:
        if len(row)==2:
            if len(chromo)!=0:
                chromos.append(chromo)
            chromo={}
            
        else:
            row1=ast.literal_eval(row[1])
            row2=ast.literal_eval(row[2])
            chromo[row[0]] = [[row1[0],row1[1],row1[2]],[row2[0],row2[1],row2[2]]]
            
    


In [ ]:
# pop=chromos
# round_sort_chromos = sorted(random_chromos.items(), key=lambda x: x[1],reverse=True)
iter=0
bestThroughAllGenerations=[]
for i in range(100):
    print("##################################################################### round ",iter)
    children=[]
    population={}
    for i in range(len(chromos)):
        population[i]=fitness(chromos[i],sizeOfRoom)
    j=min(population, key=population.get)
    print("best chromo is: ",j," and its value is: ",population[j])
    bestThroughAllGenerations.append(population[j])
    # if iter>0:
    #     r=bestThroughAllGenerations[iter]-bestThroughAllGenerations[iter-1]
    #     if r==0 and bestThroughAllGenerations[iter]<0.0011:
    #         break
    #     if iter>1000:
    #         break
            

    seventy_percent_of_the_new_population=tournament_selection(chromos,population)
    print("selection done")
    children=cross_over(rows,colmns,chromos,seventy_percent_of_the_new_population)
    print("crosover done")

    chromos =seventy_percent_of_the_new_population+children
    chromos=mutation(chromos,rows,colmns)
    print("mutation done")
    iter+=1

print("______________________________________")
print(chromos)
pop={}
for i in range(len(chromos)):
  pop[i]=fitness(chromos[i],sizeOfRoom)

round_sort_chromos = sorted(pop.items(), key=lambda x: x[1])
round_sort_chromos = list(round_sort_chromos)
best_chromos_indexes=[]
for i in range (10):
  best_chromos_indexes.append(round_sort_chromos[i])
print(best_chromos_indexes)
    

##################################################################### round  0
best chromo is:  8  and its value is:  46
selection done
crosover done
mutation done
##################################################################### round  1
best chromo is:  8  and its value is:  46
selection done
crosover done
mutation done
##################################################################### round  2
best chromo is:  76  and its value is:  41
selection done
crosover done
mutation done
##################################################################### round  3
best chromo is:  10  and its value is:  41
selection done
crosover done
mutation done
##################################################################### round  4
best chromo is:  13  and its value is:  41
selection done
crosover done
mutation done
##################################################################### round  5
best chromo is:  0  and its value is:  41
selection done
crosover done
mutation done
#############

In [ ]:
print(best_chromos_indexes[0])

(0, 41)
